A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
classdef EQCOMDupireSSSABRRawNormalCLVRoughHestonRSLV < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        impliedVolSurface
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
        paramsIn
        
       %% HestonSLV parameters
        levFuncInfo
        
       %% Heston parameters
        kappa
        theta
        rho
        sigma
        v0
        
       %% Rough Heston params
        H  % Hurst Parameter
        psi
        psiT
        psiDailyTimeStep
        
        quadrature_NumOfWeights
        quadrature_nodesAndWeights
        
        maxIter
        calibrationErrorType
        modelPriceFormula
        strikeUseBound
        
       %% SS SABR parameters
        %dynamic
        alpha_0
        alpha_inf
        rho_SS
        nu
        
        %static
        beta
        lambda
        omega
        time_shift
       
       %% Ornstein-Uhlenbeck
       speed_OU
       level_OU
       vol_OU
       x0_OU
       
       lagrangeOrder
       HermiteNormalXBar
       gridStreching_sigma
       
       BSATMVol
       
       maturityTimes
       mappingTable
       mappingTableHeston
       
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        settleFuturesRef
        atmVols
        
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSSSABRRawNormalCLVRoughHestonRSLV(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SLV modelParams start
                eqCOMDupireSpotGF.levFuncInfo = struct;
              
              %% Heston modelParams Start
                eqCOMDupireSpotGF.kappa = EQModel.modelParams('kappa');
                eqCOMDupireSpotGF.theta = EQModel.modelParams('theta');
                eqCOMDupireSpotGF.rho   = EQModel.modelParams('rho');
                
                eqCOMDupireSpotGF.sigma = EQModel.modelParams('sigma');
                eqCOMDupireSpotGF.v0    = EQModel.modelParams('v0');
                
                eqCOMDupireSpotGF.H    = EQModel.modelParams('H');
                eqCOMDupireSpotGF.psi    = EQModel.modelParams('psi');
                eqCOMDupireSpotGF.psiT   = EQModel.modelParams('psiT');
                eqCOMDupireSpotGF.psiDailyTimeStep   = EQModel.modelParams('psiDailyTimeStep');
                
                
                if strcmp(eqCOMDupireSpotGF.modelParams('quadrature_generateNodesYN'),'false')
                    eqCOMDupireSpotGF.quadrature_nodesAndWeights = eqCOMDupireSpotGF.modelParams('quadrature_nodesAndWeights');
                    eqCOMDupireSpotGF.quadrature_NumOfWeights = size(eqCOMDupireSpotGF.quadrature_nodesAndWeights,1);
                else
                    % quadrature setting
                    if isKey(eqCOMDupireSpotGF.modelParams,'quadrature_NumOfWeights')
                        eqCOMDupireSpotGF.quadrature_NumOfWeights = EQModel.modelParams('quadrature_NumOfWeights');
                    else
                        eqCOMDupireSpotGF.quadrature_NumOfWeights = 32;
                    end
                    
                    eqCOMDupireSpotGF.quadrature_nodesAndWeights = eqCOMDupireSpotGF.GenerateGaussLaguerre(eqCOMDupireSpotGF.quadrature_NumOfWeights);
                    
                end
                
                % quadrature setting
                if isKey(eqCOMDupireSpotGF.modelParams,'maxIter')
                    eqCOMDupireSpotGF.maxIter = EQModel.modelParams('maxIter');
                else
                    eqCOMDupireSpotGF.maxIter = 1000;
                end
                
                if isKey(eqCOMDupireSpotGF.modelParams,'calibrationErrorType')
                    eqCOMDupireSpotGF.calibrationErrorType = EQModel.modelParams('calibrationErrorType');
                else
                    eqCOMDupireSpotGF.calibrationErrorType = 'impVolError';
                end
                
                if isKey(eqCOMDupireSpotGF.modelParams,'modelPriceFormula')
                    eqCOMDupireSpotGF.modelPriceFormula = EQModel.modelParams('modelPriceFormula');
                else
                    eqCOMDupireSpotGF.modelPriceFormula = 'Rouah';
                end
                
                if isKey(eqCOMDupireSpotGF.modelParams,'strikeUseBound')
                    eqCOMDupireSpotGF.strikeUseBound = EQModel.modelParams('strikeUseBound');
                else
                    strikeUseBound = [30	0.8	1.2
                                    91	0.8	1.2
                                    182	0.7	1.3
                                    365	0.5	1.5
                                    730	0.5	1.5
                                    1095	0.5	1.5
                                    1460	0.5	1.5
                                    1825	0.5	1.5
                                    ];
                    eqCOMDupireSpotGF.strikeUseBound = strikeUseBound;
                end
                
                eqCOMDupireSpotGF.impliedVolSurface = eqCOMDupireSpotGF.mktData('impliedVolSurface');
                eqCOMDupireSpotGF.localVolSurface = eqCOMDupireSpotGF.modelParams('localVolSurface');
                eqCOMDupireSpotGF.fwdMoneyness = [];
               
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
                eqCOMDupireSpotGF.paramsIn = [];
              %% SSR modelparams end
                eqCOMDupireSpotGF.alpha_0 = EQModel.modelParams('alpha_0');
                eqCOMDupireSpotGF.alpha_inf = EQModel.modelParams('alpha_inf');
                eqCOMDupireSpotGF.rho_SS = EQModel.modelParams('rho_SS');
                eqCOMDupireSpotGF.nu = EQModel.modelParams('nu');
                
                
                eqCOMDupireSpotGF.beta = EQModel.modelParams('beta');
                eqCOMDupireSpotGF.lambda = EQModel.modelParams('lambda');
                eqCOMDupireSpotGF.omega = EQModel.modelParams('omega');
                eqCOMDupireSpotGF.time_shift = EQModel.modelParams('time_shift');
                
              %% Ornstein Uhlenbeck
                eqCOMDupireSpotGF.speed_OU = EQModel.modelParams('speed_OU');
                eqCOMDupireSpotGF.level_OU = EQModel.modelParams('level_OU');
                eqCOMDupireSpotGF.vol_OU = EQModel.modelParams('vol_OU');
                eqCOMDupireSpotGF.x0_OU = EQModel.modelParams('x0_OU');
                
                eqCOMDupireSpotGF.lagrangeOrder = EQModel.modelParams('lagrangeOrder');
                eqCOMDupireSpotGF.HermiteNormalXBar = EQModel.modelParams('HermiteNormalXBar');
                if length(eqCOMDupireSpotGF.HermiteNormalXBar) ~= eqCOMDupireSpotGF.lagrangeOrder
                    err('size of HermiteNormalXBar should match lagrangeOrder !')
                end
                
                if ~isKey(EQModel.modelParams,'gridStreching_sigma')
                    eqCOMDupireSpotGF.gridStreching_sigma = 1.0;
                else
                    eqCOMDupireSpotGF.gridStreching_sigma = EQModel.modelParams('gridStreching_sigma');
                end
                
                if ~isKey(EQModel.modelParams,'BSATMVol')
                    eqCOMDupireSpotGF.BSATMVol = [0.2];
                else
                    eqCOMDupireSpotGF.BSATMVol = EQModel.modelParams('BSATMVol');
                end
                
                eqCOMDupireSpotGF.maturityTimes = EQModel.modelParams('maturityTimes');
                eqCOMDupireSpotGF.mappingTable = [];
                eqCOMDupireSpotGF.mappingTableHeston = [];
              %% SS SABR model params start
                
              %% SS SABR model params end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                if ~isKey(EQModel.modelParams,'settleFutures')
                    eqCOMDupireSpotGF.settleFutures= [];
                else
                    eqCOMDupireSpotGF.settleFutures= EQModel.modelParams('settleFutures');
                end
                
                if ~isKey(EQModel.modelParams,'settleFuturesRef')
                    eqCOMDupireSpotGF.settleFuturesRef= [];
                else
                    eqCOMDupireSpotGF.settleFuturesRef = EQModel.modelParams('settleFuturesRef');
                end
                
                if ~isKey(EQModel.modelParams,'atmVols')
                    eqCOMDupireSpotGF.atmVols= [];
                else
                    eqCOMDupireSpotGF.atmVols = EQModel.modelParams('atmVols');
                end
                
                if isKey(EQModel.modelParams,'localVolSurface')
                    eqCOMDupireSpotGF.localVolSurface = EQModel.modelParams('localVolSurface');
                end
                
                if ~isKey(EQModel.modelParams,'calendarSpreadDt')
                    eqCOMDupireSpotGF.modelParams('calendarSpreadDt')= 0.5*0.000001/365.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
%                 eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 2 factor stoch vol model
                eqCOMDupireSpotGF.numOfFactors = 2;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out = GenerateGaussLaguerre(eqCOMDupireSpotGF,n)
            % Generate abscissas (x) and weights (w) for Gauss Laguerre integration

            % The Laguerre polynomial
            for k=0:n
                L(k+1) = (-1)^k/factorial(k)*nchoosek(n,k);
            end

            % Need to flip the vector to get the roots in the correct order
            L = fliplr(L);

            % Find the roots.  
            x = flipud(roots(L));

            % Find the weights
            w = zeros(n,1);
            for j=1:n
                % The coefficients of the derivative of the Laguerre polynomial
                for k=1:n
                    dL(k,j) = (-1)^k/factorial(k-1)*nchoosek(n,k)*x(j)^(k-1);
                end
                % The weight w(j)
                w(j) = 1/x(j)/sum(dL(:,j))^2;
                % The total weight w(j)exp(x(j))
                w(j) = w(j)*exp(x(j));
            end
            
            out = zeros(n,2);
            for j=1:n
                out(j,1) = x(j);
                out(j,2) = w(j);
            end
            
        end
        
        function out = HestonProb(eqCOMDupireSpotGF,phi,kappa,theta,rho,sigma,term,spotPrice,strikePrice,...
                                                     impliedRepoRate,v0,pNum,cpxLog)
            
            x  = log(spotPrice);
            sx = log(strikePrice);
            dd = x + impliedRepoRate*term;
            sigma2 = sigma * sigma;
            rsigma = rho * sigma;
             
            if (pNum == 1) 
                t0 = kappa - rho * sigma;
            else
                t0 = kappa;
            end
            
            rpsig = rsigma * phi;
            
            t1 = t0 + complex(0.0,-rpsig);
            if pNum == 1
                d = sqrt(t1*t1 - sigma2*phi*complex(-phi,1));
            else
                d = sqrt(t1*t1 - sigma2*phi*complex(-phi,-1));
            end
            
            ex = exp(-d*term);
            
            if strcmp(cpxLog,'Gatheral')
                if abs(phi) > 1e-8
                    if sigma > 1e-5
                        p = (t1-d)/(t1+d);
                        g = log((1.0-p*ex)/(1.0-p));
                        
                        D = v0*(t1-d)*(1.0-ex)/(sigma2*(1.0-ex*p));
                        C = kappa*theta/sigma2*((t1-d)*term -2.0*g);
                        out = imag(exp(D+C+complex(0.0,phi*(dd - sx))))/phi;
                    else
                         if pNum == 1
                             td = phi/(2.0*t1)*complex(-phi,1);
                         else
                             td = phi/(2.0*t1)*complex(-phi,-1);
                         end
                         
                         p = td*sigma2/(t1+d);
                         g = p*(1.0 -ex);
                         
                         D = v0*td*(1.0-ex)/(1.0-p*ex);
                         C = kappa*theta*(td*term - 2.0*g/sigma2);
                         out = imag(exp(D+C+complex(0.0,phi*(dd - sx))))/phi;
                    end
                else
                    error('not implemented for phi == 0 case !')
                end
            else
                error('notImplemented!')
            end
        end
        
        function out = HestonProbVector(eqCOMDupireSpotGF,phi,kappa,theta,rho,sigma,term,spotPrice,strikePrice,...
                                                     impliedRepoRate,v0,pNum,cpxLog)
            
            x  = log(spotPrice);
            sx = log(strikePrice);
            dd = x + impliedRepoRate*term;
            sigma2 = sigma * sigma;
            rsigma = rho * sigma;
             
            if (pNum == 1) 
                t0 = kappa - rho * sigma;
            else
                t0 = kappa;
            end
            
            rpsig = rsigma * phi;
            
            t1 = t0 + complex(0.0,-rpsig);
            if pNum == 1
                d = sqrt(t1*t1 - sigma2*phi*complex(-phi,1));
            else
                d = sqrt(t1*t1 - sigma2*phi*complex(-phi,-1));
            end
            
            ex = exp(-d*term);
            
            if strcmp(cpxLog,'Gatheral')
                if abs(phi) > 1e-8
                    if sigma > 1e-5
                        p = (t1-d)/(t1+d);
                        g = log((1.0-p*ex)/(1.0-p));
                        
                        D = v0*(t1-d)*(1.0-ex)/(sigma2*(1.0-ex*p));
                        C = kappa*theta/sigma2*((t1-d)*term -2.0*g);
                        out = imag(exp(D+C+complex(0.0,phi*(dd - sx))))/phi;
                    else
                         if pNum == 1
                             td = phi/(2.0*t1)*complex(-phi,1);
                         else
                             td = phi/(2.0*t1)*complex(-phi,-1);
                         end
                         
                         p = td*sigma2/(t1+d);
                         g = p*(1.0 -ex);
                         
                         D = v0*td*(1.0-ex)/(1.0-p*ex);
                         C = kappa*theta*(td*term - 2.0*g/sigma2);
                         out = imag(exp(D+C+complex(0.0,phi*(dd - sx))))/phi;
                    end
                else
                    error('not implemented for phi == 0 case !')
                end
            else
                error('notImplemented!')
            end
        end
        
        function out = HestonPriceGaussLaguerre(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType)
            
%             spotPrice = params('spotPrice');
%             strikePrice = params('strikePrice');
%             termDay = params('termDay');
            term = termDay/365.0;
            
            fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
            
            impliedRepoRate = log(fwd/spotPrice)/term;
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            rho   = eqCOMDupireSpotGF.rho;
            
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            
            x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
            w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
            
            int1 = zeros(length(x),1);
            int2 = zeros(length(x),1);
            
%             for k=length(x):-1:1
                
            for k=1:length(x)                                    
                int1(k) = w(k) * eqCOMDupireSpotGF.HestonProb(x(k),kappa,theta,eqCOMDupireSpotGF.rho,sigma,term,spotPrice,strikePrice,...
                                                    impliedRepoRate,v0,1,'Gatheral');
                int2(k) = w(k) * eqCOMDupireSpotGF.HestonProb(x(k),kappa,theta,rho,sigma,term,spotPrice,strikePrice,...
                                                    impliedRepoRate,v0,2,'Gatheral');
            end
            
            p1 = 1/pi*sum(int1);
            
            p2 = 1/pi*sum(int2);
            
%             optionType = params('optionType');
            
            switch  optionType
                case 'Call'
                    out = fwd*(p1 + 0.5) - strikePrice*(p2 + 0.5);
                case 'Put'
                    out = fwd*(p1 - 0.5) - strikePrice*(p2 - 0.5);
                otherwise
                    error('not supported! only Call or Put')
            end
                
            
        end
        
        function out = HestonPriceGaussLaguerreVector(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType)
            
%             spotPrice = params('spotPrice');
%             strikePrice = params('strikePrice');
%             termDay = params('termDay');
            term = termDay/365.0;
            
            fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
            
            impliedRepoRate = log(fwd/spotPrice)/term;
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            rho   = eqCOMDupireSpotGF.rho;
            
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            
            x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
            w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
            
            int1 = zeros(length(x),length(strikePrice));
            int2 = zeros(length(x),length(strikePrice));
            
%             for k=length(x):-1:1
                
            for k=1:length(x)                                    
                int1(k,:) = w(k) * eqCOMDupireSpotGF.HestonProbVector(x(k),kappa,theta,eqCOMDupireSpotGF.rho,sigma,term,spotPrice,strikePrice,...
                                                    impliedRepoRate,v0,1,'Gatheral');
                int2(k,:) = w(k) * eqCOMDupireSpotGF.HestonProbVector(x(k),kappa,theta,rho,sigma,term,spotPrice,strikePrice,...
                                                    impliedRepoRate,v0,2,'Gatheral');
            end
            
            p1 = 1/pi*sum(int1);
            
            p2 = 1/pi*sum(int2);
            
%             optionType = params('optionType');
            
            switch  optionType
                case 'Call'
                    out = fwd*(p1 + 0.5) - strikePrice.*(p2 + 0.5);
                case 'Put'
                    out = fwd.*(p1 - 0.5) - strikePrice.*(p2 - 0.5);
                otherwise
                    error('not supported! only Call or Put')
            end
                
            
        end
        
        function y = HestonProbRouah(eqCOMDupireSpotGF,phi,kappa,theta,lambda,rho,sigma,tau,K,S,r,q,v,Pnum,Trap)

                % Returns the integrand for the risk neutral probabilities P1 and P2.
                % phi = integration variable
                % Pnum = 1 or 2 (for the probabilities)
                % Heston parameters:
                %    kappa  = volatility mean reversion speed parameter
                %    theta  = volatility mean reversion level parameter
                %    lambda = risk parameter
                %    rho    = correlation between two Brownian motions
                %    sigma  = volatility of variance
                %    v      = initial variance
                % Option features.
                %    PutCall = 'C'all or 'P'ut
                %    K = strike price
                %    S = spot price
                %    r = risk free rate
                %    Trap = 1 "Little Trap" formulation 
                %           0  Original Heston formulation

                % Log of the stock price.
                x = log(S);

                % Parameter "a" is the same for P1 and P2.
                a = kappa*theta;

                % Parameters "u" and "b" are different for P1 and P2.
                if Pnum==1
                    u = 0.5;
                    b = kappa + lambda - rho*sigma;
                else
                    u = -0.5;
                    b = kappa + lambda;
                end

                d = sqrt((rho*sigma*i*phi - b)^2 - sigma^2*(2*u*i*phi - phi^2));
                g = (b - rho*sigma*i*phi + d) / (b - rho*sigma*i*phi - d);

                if Trap==1
                    % "Little Heston Trap" formulation
                    c = 1/g;
                    D = (b - rho*sigma*i*phi - d)/sigma^2*((1-exp(-d*tau))/(1-c*exp(-d*tau)));
                    G = (1 - c*exp(-d*tau))/(1-c);
                    C = (r-q)*i*phi*tau + a/sigma^2*((b - rho*sigma*i*phi - d)*tau - 2*log(G));
                elseif Trap==0
                    % Original Heston formulation.
                    G = (1 - g*exp(d*tau))/(1-g);
                    C = (r-q)*i*phi*tau + a/sigma^2*((b - rho*sigma*i*phi + d)*tau - 2*log(G));
                    D = (b - rho*sigma*i*phi + d)/sigma^2*((1-exp(d*tau))/(1-g*exp(d*tau)));
                end

                % The characteristic function.
                f = exp(C + D*v + i*phi*x);

                % Return the real part of the integrand.
                y = real(exp(-i*phi*log(K))*f/i/phi);
        end
        
        
        
        function y = HestonPriceGaussLaguerreRouahSimple(eqCOMDupireSpotGF,PutCall,S,K,T,r,q,kappa,theta,sigma,lambda,v0,rho,trap,x,w)

                % Heston (1993) call or put price by Gauss-Laguerre Quadrature
                % Uses the original Heston formulation of the characteristic function,
                % or the "Little Heston Trap" formulation of Albrecher et al.
                % INPUTS -------------------------------------------------------
                %   PutCall = 'C' Call or 'P' Put
                %   S = Spot price.
                %   K = Strike
                %   T = Time to maturity.
                %   r = Risk free rate.
                %   kappa  = Heston parameter: mean reversion speed.
                %   theta  = Heston parameter: mean reversion level.
                %   sigma  = Heston parameter: volatility of vol
                %   lambda = Heston parameter: risk.
                %   v0     = Heston parameter: initial variance.
                %   rho    = Heston parameter: correlation
                %   trap:  1 = "Little Trap" formulation
                %          0 = Original Heston formulation
                %   x = Gauss Laguerre abscissas
                %   w = Gauss Laguerre weights
                % OUTPUT -------------------------------------------------------
                %   The Heston call or put price

                % Numerical integration
                for k=1:length(x)
                    int1(k) = w(k)*eqCOMDupireSpotGF.HestonProbRouah(x(k),kappa,theta,lambda,rho,sigma,T,K,S,r,q,v0,1,trap);
                    int2(k) = w(k)*eqCOMDupireSpotGF.HestonProbRouah(x(k),kappa,theta,lambda,rho,sigma,T,K,S,r,q,v0,2,trap);
                end

                % Define P1 and P2
                P1 = 1/2 + 1/pi*sum(int1);
                P2 = 1/2 + 1/pi*sum(int2);

                % The call price
                HestonC = S*exp(-q*T)*P1 - K*exp(-r*T)*P2;

                % The put price by put-call parity
%                 HestonP = HestonC - S*exp(-q*T) + K*exp(-r*T);
                HestonP = S*exp(-q*T)*(P1-1.0) - K*exp(-r*T)*(P2-1.0);

                % Output the option price
                if strcmp(PutCall,'C')
                    y = HestonC;
                else
                    y = HestonP;
                end
            
        end
        
        function out = HestonPriceGaussLaguerreRouah(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType)
       
                term = termDay/365.0;

                fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);

                impliedRepoRate = log(fwd/spotPrice)/term;

                kappa = eqCOMDupireSpotGF.kappa;
                theta = eqCOMDupireSpotGF.theta;
                rho   = eqCOMDupireSpotGF.rho;

                sigma = eqCOMDupireSpotGF.sigma;
                v0    = eqCOMDupireSpotGF.v0;

                x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
                w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
                
                trap = 1;
                switch  optionType
                    case 'Call'
                        out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreRouahSimple('C',spotPrice,strikePrice,term,impliedRepoRate,0.0,kappa,theta,sigma,0.0,v0,rho,trap,x,w);

                    case 'Put'
                        out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreRouahSimple('P',spotPrice,strikePrice,term,impliedRepoRate,0.0,kappa,theta,sigma,0.0,v0,rho,trap,x,w);
                    otherwise
                        error('not supported! only Call or Put')
                end
        end
        
        function out = HestonCharacteristicFcnR(eqCOMDupireSpotGF,phi,kappa,theta,rho,sigma,term,v0)
                                                     
             al  = -phi*phi/2.0 -1i*phi/2.0;
             bet = kappa - rho*sigma*1i*phi;
             gam = sigma*sigma/2.0;
             d   = sqrt(bet*bet -4*al*gam);
             rp  = (bet + d)/(2*gam);
             rm  = (bet - d)/(2*gam);
             g   = rm/rp;
             D   = rm*(1 - exp(-d*term))/(1 - g*exp(-d*term));
             C   = kappa*(rm * term -2/(sigma*sigma) * log((1 - g*exp(-d*term))/(1 - g)));
             out = exp(C*theta + D*v0);
        end
        
        function out = HestonCharacteristicFcnPadeApprox33(eqCOMDupireSpotGF,phi,H,kappa,theta,rho,sigma,term,v0)
                                                     
%             H   = params(1);
%             rho = params(3);
%             nu  = params(2);

            alpha = H + 0.5;
            nu = sigma;

            % we treat 1year as 365 days 
            % we discretize daily time step
            n = ceil(365.0*term);
            
            ti  = (0:(n-1)) / n * term;
            dt  = ti(end) - ti(end-1);
            % initial forward variance curve
            xi = v0*ones(1,length(ti));
            
            y   = nu * ti.^alpha;

            u = phi;
            [rm,rp,p1,p2,p3,q1,q2,q3] = dh_Pade33_coeff(alpha,rho,u);

%             m = size(u,2);
            m = length(phi);
            

            charFun = zeros(1,m);
            for j = 1:m
                dah = dh_Pade33(y,rm(1,j),rp(1,j),p1(1,j),p2(1,j),p3(1,j),q1(1,j),q2(1,j),q3(1,j)); 
                charFun(1,j) = exp(flip(xi) * dah.' * dt);
            end
            
            out = charFun;

%             phi = charFun;
%         
%              al  = -phi*phi/2.0 -1i*phi/2.0;
%              bet = kappa - rho*sigma*1i*phi;
%              gam = sigma*sigma/2.0;
%              d   = sqrt(bet*bet -4*al*gam);
%              rp  = (bet + d)/(2*gam);
%              rm  = (bet - d)/(2*gam);
%              g   = rm/rp;
%              D   = rm*(1 - exp(-d*term))/(1 - g*exp(-d*term));
%              C   = kappa*(rm * term -2/(sigma*sigma) * log((1 - g*exp(-d*term))/(1 - g)));
%              out = exp(C*theta + D*v0);
        end
        
        function out = HestonCharacteristicFcnPadeApprox33Simple(eqCOMDupireSpotGF,addParams,xicurve,phi,term)
            H = eqCOMDupireSpotGF.H;
            rho = eqCOMDupireSpotGF.rho;
            sigma = eqCOMDupireSpotGF.sigma;

            alpha = H + 0.5;
            nu = sigma;

            % we treat 1year as 365 days 
            % we discretize daily time step
%             n = ceil(365.0*term);
%             ti  = (0:(n-1)) / n * term;
%             dt  = ti(end) - ti(end-1);
%             % initial forward variance curve
%             xi = xicurve(ti);

            % we use cached initial forward curve    
            n = addParams.n;
            ti = addParams.ti;
            dt = addParams.dt;
            xi = addParams.xi;
            
            
            y   = nu * ti.^alpha;

            u = phi;
            [rm,rp,p1,p2,p3,q1,q2,q3] = dh_Pade33_coeff(alpha,rho,u);
            
            
            m = size(u,2);

            charFun = zeros(1,m);
            for j = 1:m
                dah = dh_Pade33(y,rm(1,j),rp(1,j),p1(1,j),p2(1,j),p3(1,j),q1(1,j),q2(1,j),q3(1,j)); 
                charFun(1,j) = exp(flip(xi) * dah.' * dt);
            end

            out = charFun;
        
%             out = exp(flip(xi) * dah.' * dt);
            
        end
        
        function out = HestonPriceGaussLaguerreGatheralR(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType)

            term = termDay/365.0;
            
            fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
            
%             impliedRepoRate = log(fwd/spotPrice)/term;
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            rho   = eqCOMDupireSpotGF.rho;
            
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            
            x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
            w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
            
            int1 = zeros(length(x),1);
            
            realPart = 0.0;
            charFcn = 0.0;
            
            logK = log(strikePrice/fwd);
            reducedK = strikePrice/fwd; 
            
            for k=1:length(x)
                charFcn = eqCOMDupireSpotGF.HestonCharacteristicFcnR(x(k) -1i/2.0,kappa,theta,rho,sigma,term,v0);
                realPart = real(exp(-1i*x(k)*logK)*charFcn);
                int1(k) = w(k) * 1.0/(x(k)*x(k) + 0.25)*realPart;                       
            end
            
            p1 = 1/pi*sum(int1);
            
            switch  optionType
                case 'Call'
                    out = fwd*(1.0 - sqrt(reducedK)*p1);
                case 'Put'
                    out = fwd*(reducedK -sqrt(reducedK)*p1);
                otherwise
                    error('not supported! only Call or Put')
            end
                
            
        end
        
        function out = HestonPriceGaussLaguerreGatheraPadeApprox33(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType)

            term = termDay/365.0;
            
            fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
            
%             impliedRepoRate = log(fwd/spotPrice)/term;
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            rho   = eqCOMDupireSpotGF.rho;
            
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            H    = eqCOMDupireSpotGF.H;
            
            
            x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
            w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
            
            int1 = zeros(length(x),1);
            
            realPart = 0.0;
            charFcn = 0.0;
            
            logK = log(strikePrice/fwd);
            reducedK = strikePrice/fwd; 
            
            y = zeros(1,length(x));
            for k=1:length(x)
                y(k) = x(k) -1i/2.0;
            end
            
            charFcn = eqCOMDupireSpotGF.HestonCharacteristicFcnPadeApprox33(y,H, kappa,theta,rho,sigma,term,v0);
            
            for k=1:length(x)
                charFcn_k = charFcn(k);
                realPart = real(exp(-1i*x(k)*logK)*charFcn_k);
                int1(k) = w(k) * 1.0/(x(k)*x(k) + 0.25)*realPart;                       
            end
            
            p1 = 1/pi*sum(int1);
            
            switch  optionType
                case 'Call'
                    out = fwd*(1.0 - sqrt(reducedK)*p1);
                case 'Put'
                    out = fwd*(reducedK -sqrt(reducedK)*p1);
                otherwise
                    error('not supported! only Call or Put')
            end
                
            
        end
        
        function out = XicurveSingle(eqCOMDupireSpotGF,t)
            
            psiT = eqCOMDupireSpotGF.psiT;
            psi = eqCOMDupireSpotGF.psi;
            v0 = eqCOMDupireSpotGF.v0;
            
            alpha = eqCOMDupireSpotGF.H + 0.5;
            invgamma = 1.0/gamma(alpha +1.0);
            
            for i=1:length(psiT)
                if t<= psiT(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            kerf = @(u,s) (u-s).^(alpha);
            for i=1:tidx
                if i< tidx
                    curr_t = psiT(i);
                else
                    curr_t = t;
                end
                sum = sum + psi(i)*(kerf(t,prev_t)-kerf(t,curr_t));

%                 prev_t = psiT(i);
                prev_t = curr_t;
                
            end
            
            out = v0 + invgamma.*sum;
            
            
%             out = 0.0;
            
        end
       
        function out = Xicurve(eqCOMDupireSpotGF,t)
            
            xiCurvep = @(s) (arrayfun(@(s) XicurveSingle(eqCOMDupireSpotGF,s),s));
            out = xiCurvep(t);
            aaa = 1.0;
%             out =  v0*zeros(1,m);
%             zerosIdx = find(abs(t) < 1e-6);
%             nonzerosIdx = find(abs(t) >= 1e-6);
% %             out(zerosIdx) = 0.0;
% %             out(nonzerosIdx) = integral(integrand, 0.0, t(nonzerosIdx),'RelTol',1e-8);
%             
%             for idx=1:length(nonzerosIdx)
% %                 p1 = integral(integrand, 0.0, t(nonzerosIdx(idx)),'RelTol',1e-8);
%                 p1 = integral(integrand, 0.0, t(nonzerosIdx(idx)));
%                 
%                 out(1,nonzerosIdx(idx)) = p1;
%             end
%             
%             out = out.*kappa.*(theta-v0);
%             out = out + v0;
%             bbb = 0.0;
%             out = theta.*(1.0 - exp(-kappa.* t)) + v0.*exp(-kappa.* t);

        end
        
        
        function out = XicurveKappa0(eqCOMDupireSpotGF,t)
            
            m = size(t,2);
            v0 = eqCOMDupireSpotGF.v0;
            out =  v0*ones(1,m);
            
%             out = v0;
        end
        
        
        function out = yAlpha(eqCOMDupireSpotGF,H,tau,xiCurve)
            alpha = H + 0.5;
            integ = @(s) (xiCurve(s)./(tau-s).^alpha);
            res = integral(integ,0.0,tau)/gamma(1.0-alpha);
            out = res;
        end
        function out = FindaAmax(eqCOMDupireSpotGF,xiCurve,k,tau,epsSig)
            rho = eqCOMDupireSpotGF.rho;
            H = eqCOMDupireSpotGF.H;
            sigma = eqCOMDupireSpotGF.sigma;
            yy = sqrt(1-rho*rho)/sigma *eqCOMDupireSpotGF.yAlpha(H,tau,xiCurve);
            c = sqrt(pi/2)*exp(-k/2)*epsSig;
            f = @(a) (exp(-yy*a) -c*yy*(a*a+0.25));
            amax = fzero(f,[0 1.0/sqrt(c*yy)]);
            out = amax;
        end
        
        function out = RoughHestonPriceGatheralPadeApprox33Amax(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType,addParams)
            try 
                term = termDay/365.0;

                fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);

                kappa = eqCOMDupireSpotGF.kappa;
                theta = eqCOMDupireSpotGF.theta;
                rho   = eqCOMDupireSpotGF.rho;

                sigma = eqCOMDupireSpotGF.sigma;
                v0    = eqCOMDupireSpotGF.v0;
                H    = eqCOMDupireSpotGF.H;

                % find Amax
                % xicurve
                xicurve = @(x)  eqCOMDupireSpotGF.Xicurve(x);

%                 logK = log(strikePrice/fwd);
%                 %             FindAmax(eqCOMDupireSpotGF,
%                 x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
%                 w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
% 
%                 int1 = zeros(length(x),1);
% 
%                 realPart = 0.0;
%                 charFcn = 0.0;

                logK = log(strikePrice/fwd);
                reducedK = strikePrice/fwd; 

%                 y = zeros(1,length(x));
%                 for k=1:length(x)
%                     y(k) = x(k) -1i/2.0;
%                 end

                charFcn = @(x,t) eqCOMDupireSpotGF.HestonCharacteristicFcnPadeApprox33Simple(addParams,xicurve,x,t);

%                 aMax =  eqCOMDupireSpotGF.FindaAmax(xicurve,logK,term,1e-4);
    %             dummy = charFcn(0.0,1.0);
                integrand = @(u) (real(exp(-1i.*u.*logK).*charFcn(u-1i/2.0,term)./(u.*u +0.25)));
%                 p1 = integral(integrand,0.0,aMax);
%                 p1 = integral(integrand,0.0,inf,'RelTol',1e-8);
                p1 = integral(integrand,0.0,inf);
                p1 = p1 ./pi;
                
%                 x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
%                 w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
% 
%                 int1 = zeros(length(x),1);
% 
%                 for k=1:length(x)
%                     int1(k) = w(k) * integrand(x(k));
%                 end
% 
%                 p1 = 1/pi*sum(int1);
                
                
%                 p1 = p1 ./pi;

    %             charFcn = eqCOMDupireSpotGF.HestonCharacteristicFcnPadeApprox33Simple(xicurve,y,term);



    %             charFcn = eqCOMDupireSpotGF.HestonCharacteristicFcnPadeApprox33(y,H, kappa,theta,rho,sigma,term,v0);
    %             
    %             for k=1:length(x)
    %                 charFcn_k = charFcn(k);
    %                 realPart = real(exp(-1i*x(k)*logK)*charFcn_k);
    %                 int1(k) = w(k) * 1.0/(x(k)*x(k) + 0.25)*realPart;                       
    %             end

    %             p1 = 1/pi*sum(int1);
    %             p1 = 0.0;

                switch  optionType
                    case 'Call'
                        out = fwd*(1.0 - sqrt(reducedK)*p1);
                    case 'Put'
                        out = fwd*(reducedK -sqrt(reducedK)*p1);
                    otherwise
                        error('not supported! only Call or Put')
                end
              
            catch ME
                rethrow(ME)
            end
            
        end
        
        function out = RoughHestonPriceEER18Sigurdroemer(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,optionType)

            term = termDay/365.0;
            
            fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
            
%             impliedRepoRate = log(fwd/spotPrice)/term;
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            rho   = eqCOMDupireSpotGF.rho;
            
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            H    = eqCOMDupireSpotGF.H;
            
            alpha = H + 0.5;
            
            switch  optionType
                case 'Call'
                    call = true;
                case 'Put'
                    call = false;
                otherwise
                    error('not supported! only Call or Put')
            end
            
            [price] = NumericalIntegrationRoughHestonPrice(fwd,v0,alpha,kappa,...
                                        theta,sigma,rho,call,strikePrice,term,...
                                        'disp_iter',true);
                                    
            out = price;
                                    
%             x = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,1);
%             w = eqCOMDupireSpotGF.quadrature_nodesAndWeights(:,2);
%             
%             int1 = zeros(length(x),1);
%             
%             realPart = 0.0;
%             charFcn = 0.0;
%             
%             logK = log(strikePrice/fwd);
%             reducedK = strikePrice/fwd; 
%             
%             y = zeros(1,length(x));
%             for k=1:length(x)
%                 y(k) = x(k) -1i/2.0;
%             end
%             
%             charFcn = eqCOMDupireSpotGF.HestonCharacteristicFcnPadeApprox33(y,H, kappa,theta,rho,sigma,term,v0);
%             
%             for k=1:length(x)
%                 charFcn_k = charFcn(k);
%                 realPart = real(exp(-1i*x(k)*logK)*charFcn_k);
%                 int1(k) = w(k) * 1.0/(x(k)*x(k) + 0.25)*realPart;                       
%             end
%             
%             p1 = 1/pi*sum(int1);
%             
%             switch  optionType
%                 case 'Call'
%                     out = fwd*(1.0 - sqrt(reducedK)*p1);
%                 case 'Put'
%                     out = fwd*(reducedK -sqrt(reducedK)*p1);
%                 otherwise
%                     error('not supported! only Call or Put')
%             end
                
            
        end
        
        function out = modelOTMPrice(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,vanillayOptionType,addParams)
                fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
                K = strikePrice/fwd;
                if K <= 1.0
                    optionType = 'Put';
                else
                    optionType = 'Call';
                end
                    
%                 if nargin == 4
%                     if K <= 1.0
%                         optionType = 'Put';
%                     else
%                         optionType = 'Call';
%                     end
%                 else
%                     optionType = vanillayOptionType;
%                 end
                
                if strcmp(eqCOMDupireSpotGF.modelPriceFormula,'Rouah')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreRouah(spotPrice,strikePrice,termDay,optionType);
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'Quantlib')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerre(spotPrice,strikePrice,termDay,optionType);
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'GatheralR')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreGatheralR(spotPrice,strikePrice,termDay,optionType);
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'GatheralPadeApprox33')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreGatheraPadeApprox33(spotPrice,strikePrice,termDay,optionType);
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'GatheralPadeApprox33Amax')
                    out = eqCOMDupireSpotGF.RoughHestonPriceGatheralPadeApprox33Amax(spotPrice,strikePrice,termDay,optionType,addParams);    
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'EER18Sigurdroemer')
                    out = eqCOMDupireSpotGF.RoughHestonPriceEER18Sigurdroemer(spotPrice,strikePrice,termDay,optionType);
                    
                else
                    error('unknown modelPriceFormula type')
                end
        end
        
        function out = modelOTMPriceVector(eqCOMDupireSpotGF,spotPrice,strikePrice,termDay,vanillayOptionType)
                fwd = eqCOMDupireSpotGF.Fwd(spotPrice,termDay);
                K = strikePrice/fwd;
                if nargin == 4
                    if K <= 1.0
                        optionType = 'Put';
                    else
                        optionType = 'Call';
                    end
                else
                    optionType = vanillayOptionType;
                end
                
                if strcmp(eqCOMDupireSpotGF.modelPriceFormula,'Rouah')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreRouah(spotPrice,strikePrice,termDay,optionType);
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'Quantlib')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreVector(spotPrice,strikePrice,termDay,optionType);
                elseif strcmp(eqCOMDupireSpotGF.modelPriceFormula,'GatheralR')
                    out = eqCOMDupireSpotGF.HestonPriceGaussLaguerreGatheralR(spotPrice,strikePrice,termDay,optionType);
                else
                    error('unknown modelPriceFormula type')
                end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = initFuturesStates(eqCOMDupireSpotGF,NMC)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            out =  spotPrice*ones(NMC,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            out = nextFuturesStates;
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end

        function localVol = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry;
            
            %find which strip fromTime belongs to 
            idxS = find(expiry > fromTime);
            if isempty(idxS)
                idxStrip = length(expiry);
            else
                idxStrip = min(idxS);
            end
            
            % find exact timeSlice in the given strip
            try
                localVolTimeStep = eqCOMDupireSpotGF.localVolSurface.localVolTimeStep{idxStrip};
                localVolSurfaceSABR = eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR{idxStrip};
                forwardMoneyness = eqCOMDupireSpotGF.localVolSurface.forwardMoneyness{idxStrip};
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            idx = find(localVolTimeStep > fromTime);
            if isempty(idx)
                idxNow = length(localVolTimeStep);
            else
                idxNow = min(idx);
            end
            
            localVolLine = localVolSurfaceSABR(idxNow,:);
%             forwardMoneynessLine = forwardMoneyness(idxNow,:);
            
            localVol =  zeros(length(x),1);
            
            dummyOut = interp1FlatExtrapNew(forwardMoneyness,localVolLine,x','linear');
            dummyOut2 = sqrt(dummyOut);
            localVol = dummyOut2';
            
%             for ii =1:length(x)
%                 dummyVariance = H_interpolation(forwardMoneyness,localVolLine,x(ii),1);
%                 localVol(ii) = sqrt(dummyVariance);
%             end

        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
         function mcmcOut = inductMCForwardNewReducedQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
         end
        
         function mcmcOut = inductMCForwardNewReducedNMonthsQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut1 = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            mcmcOut.nextFuturesStates = mcmcOut1.nextX;
            mcmcOut.idxNM = 1;
        end
        
        function mcmcOut = inductMCForwardNewCLVBSATM(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;

            Dt = (toTime - fromTime)/365.0;
            drift = eqCOMDupireSpotGF.BSATM_Expectation(fromTime,predictor,Dt);
            stdDev = eqCOMDupireSpotGF.BSATM_StdDeviation(fromTime,predictor,Dt);
            
            predictor = (drift - 0.5*stdDev*stdDev) + stdDev*dZ;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewCLV(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;

            Dt = (toTime - fromTime)/365.0;
            drift = eqCOMDupireSpotGF.OU_Expectation(fromTime,predictor,Dt);
            stdDev = eqCOMDupireSpotGF.OU_StdDeviation(fromTime,predictor,Dt);
            
            predictor = drift + stdDev*dZ;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(spotRawData*predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end

        function mcmcOut = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = GetLocalVol(eqCOMDupireSpotGF,fromTime,x)
            
%             lookupTime = max(fromTime-0.01,0);
            out = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime);
            
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             strike = eqCOMDupireSpotGF.localVolSurface.strike';
% %             out = interp1(strike,volProxy,x,'linear','extrap');
% 
%             out = interp1FlatExtrap(strike,volProxy,x);

%             out = zeros(length(x),1);
%             for idx=1:length(x)
%                 out(idx) = H_interpolation(strike,volProxy,x(idx),1);
%             end
              
        end
        
        function out = GetLevFunc(eqCOMDupireSpotGF,fromTime,x)
            
            timeStep = eqCOMDupireSpotGF.levFuncInfo.timeStepLVF;
            
            idx = find(timeStep > fromTime);
            if isempty(idx)
                idxNow = length(timeStep);
            else
                idxNow = min(idx);
            end
            
            levFunc = eqCOMDupireSpotGF.levFuncInfo.levFunc(idxNow,:);
            strike = eqCOMDupireSpotGF.levFuncInfo.vStrikes(idxNow,:);
            
%             out = interp1(strike,levFunc,x,'linear','extrap');
            out = interp1FlatExtrap(strike,levFunc,x);
            
%             out = zeros(length(x),1);
%             for idx=1:length(x)
%                 out(idx) = H_interpolation(strike,levFunc,x(idx),1);
%             end
              
        end
        
        function out = GenerateLeverageFunction(eqCOMDupireSpotGF,valueDate,leverageFunctionParams)
           
          %% initialize leverageFunction and assign leverageFunction to eqCOMDupireSpotGF's property
           numOfBins = leverageFunctionParams.numOfBins;
           numOfPathPerBins = leverageFunctionParams.numOfPathPerBins;
           numOfPath = leverageFunctionParams.numOfPath;
           
           numOfPathPerBins = numOfPath/numOfBins;
           modOfPath = mod(numOfPath,numOfBins);
           
           timeStep = leverageFunctionParams.timeStepLVF;
         %%
           
           expiryDate = leverageFunctionParams.expiryDate;
           expiry = expiryDate.DateDiff(valueDate);
           
           % we add today & expiryDate
           timeStep = [timeStep;0];
           timeStep = sort(union(timeStep,expiry),'ascend');
           
           timeStep = timeStep(find(timeStep <= expiry));
           
           % leverageFunction Lookup Know Points
           timeStepLVF = timeStep(2:end);
           
           totalTimeStepSize = length(timeStep);
           eventTimeIdx = zeros(1,1);
           lastAliveExeriseIdx = 1;
           eventTimeIdx(1) = 1;
           scheduleSize = 1; 
           
         %% initialize leverageFunction
           leverageFunctionRaw = ones(totalTimeStepSize-1,numOfBins);
           vStrikes = zeros(totalTimeStepSize-1,numOfBins);
           
           eqCOMDupireSpotGF.levFuncInfo.levFunc = leverageFunctionRaw;
           eqCOMDupireSpotGF.levFuncInfo.timeStepLVF = timeStepLVF;
           eqCOMDupireSpotGF.levFuncInfo.vStrikes = vStrikes;
           
           v0 = eqCOMDupireSpotGF.v0;
           smallTime = 0.01;
           lv0 = eqCOMDupireSpotGF.GetLocalVol(smallTime,1.0);
           lvf0 = lv0/sqrt(v0);
            
          %% leverageFunction initialize
            
            % initial strikes
            dt = timeStep(2) - timeStep(1);
            dx = 1.0 * 5.0*lv0*sqrt(dt/365)/numOfBins*2.0;
            
            for j=1:numOfBins
                eqCOMDupireSpotGF.levFuncInfo.vStrikes(1,j) = 1.0 + (j-numOfBins/2.0)*dx;
                eqCOMDupireSpotGF.levFuncInfo.levFunc(1,j) = lvf0;
            end
            
           
          %% MC init 
           useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
           if strcmp(useQuasiRandomYN,'true')
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC');
                NMC = numOfPath;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC');
                NMC = numOfPath;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
           
         %% Path Generation Forward
          startTimeIdx = 1;
          endTimeIdx = length(timeStep)-1;
         %% induction Forward start
         %% below for the timeStep(idx) to timeStep(idx+1) 
         %% generated leverageFunction applied to timeStep(idx+1) to timeStep(idx+2)
         %% therefore we only need to generate from startTimeIdx to endTimeIdx-1
          
          for idx= startTimeIdx:1:endTimeIdx-1
              if idx == endTimeIdx
                abcd = 1.0;
              end
              
                if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                    mcmcOut   = inductMCForwardQESLV(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                    mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                end
                nextX = mcmcOut.nextX;
                nextV = mcmcOut.nextV;

             %% we sort simul path, asceding w.r.t nextX
               [sortedX, sortIdx] = sort(nextX,'ascend');
               sortedV = nextV(sortIdx);

               startXIdx = 1;
               endXIdx = 0;
               for idxS=1:numOfBins
                   if idxS == numOfBins-1
                        edfg = 1.0;
                   end
                   
                   inc = numOfPathPerBins;
                   endXIdx = startXIdx + inc;

                   sum = 0.0;
                   for idxW=startXIdx:endXIdx-1
                       sum = sum + sortedV(idxW);
                   end

                   %average of variance
                   sum = sum/inc;
                   nodeStrike = 0.5*(sortedX(startXIdx)+sortedX(endXIdx-1));
                   eqCOMDupireSpotGF.levFuncInfo.vStrikes(idx+1,idxS) = nodeStrike;
                   
                   % we look up the localVol from the endTime ??
%                    localVol = eqCOMDupireSpotGF.GetLocalVol(timeStep(idx+1)-0.01,nodeStrike);
                   localVol = eqCOMDupireSpotGF.GetLocalVol(timeStep(idx+1)+ 0.01,nodeStrike);
                   eqCOMDupireSpotGF.levFuncInfo.levFunc(idx+1,idxS) =  sqrt(localVol*localVol/sum);
                   startXIdx = endXIdx;
                   
               end
                
               aaa = 1.0;

            end
            
            
           out =eqCOMDupireSpotGF.levFuncInfo;  
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = computeStepdown1SMCHeston(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                
                afvSim = AFVSimulation;
                
                params0 = struct;
                params0.al = eqCOMDupireSpotGF.H + 0.5;
                params0.lam = 0.0;
                params0.nu = eqCOMDupireSpotGF.sigma;
                params0.eta = params0.nu/(sqrt(2.0.*params0.al-1).*gamma(params0.al));
                params0.rho = eqCOMDupireSpotGF.rho;
                params0.H = eqCOMDupireSpotGF.H;
                v0 = eqCOMDupireSpotGF.v0;
                xi = @(s) (0.071923356959668 + 0.*s);
                expiryRHeston = timeStep(end)/365.0;
                
                mcScheme = eqCOMDupireSpotGF.modelParams('MCScheme');
                if strcmp(mcScheme,'RSQE')
                    rsqeSim = afvSim.RSQESimHist(params0, xi,NMC, MCTimeStep, expiryRHeston);
                    xhist = rsqeSim.xhist;
                elseif strcmp(mcScheme,'HQE')
                    hqeSim = afvSim.HQESimHist(params0, xi,NMC, MCTimeStep, expiryRHeston);
                    xhist = hqeSim.xhist;
                end
                aaa = 1.0;
                
%                 numOfFactors = 2;
%                 Z= randn(NMC/2,MCTimeStep*numOfFactors);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
%            initX = zeros(NMC,1);
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
%                         mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
%                         mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     end
%                     nextX = mcmcOut.nextX;
%                     nextV = mcmcOut.nextV;
                    % one step forward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    dummyX = xhist(lastTimeIdx-1,:)';
                    nextX  = exp(dummyX);

                    comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function  output  = TargetFunctionHestonGlobalGA(eqCOMDupireSpotGF,tvar,optParams)
    
            model_otmPrice = 0.0;
            market_otmPrice = 0.0;

            % assigning model parameters
            eqCOMDupireSpotGF.H = tvar(1);
            eqCOMDupireSpotGF.rho = tvar(2);
            eqCOMDupireSpotGF.sigma = tvar(3);
            
            % cost function for levenberg marquardt algorithm
            out= zeros(optParams.numOfEquation,1);
            idx =1;

            expiry = optParams.params.expiry;
            strike = optParams.params.strike;
            spot =  eqCOMDupireSpotGF.spot.params('rawData');

           if  strcmp(optParams.params.calibrationErrorType,'impVolError')
               for i=1:optParams.params.expirySize
                   fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                    for j=1:optParams.params.strikeSize
                        if optParams.params.calibrationYN(i,j) == 1
                            market_vol = optParams.params.blackVol(i,j);
                            marketStrike = optParams.params.marketStrikes(i,j);
                            model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                            model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                            out(idx) = model_vol - market_vol;
                            idx = idx + 1;
                        end
                    end
               end
           elseif strcmp(optParams.params.calibrationErrorType,'priceError')
               for i=1:optParams.params.expirySize
                   fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                    for j=1:optParams.params.strikeSize
                        if optParams.params.calibrationYN(i,j) == 1
                            market_otmPrice = optParams.params.blackOTMPrice(i,j);
                            marketStrike = optParams.params.marketStrikes(i,j);
                            model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                            out(idx) = model_otmPrice - market_otmPrice;
                            idx = idx + 1;
                        end
                    end
               end
           elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
               for i=1:optParams.params.expirySize
                   fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                    for j=1:optParams.params.strikeSize
                        if optParams.params.calibrationYN(i,j) == 1
                            market_otmPrice = optParams.params.blackOTMPrice(i,j);
                            marketStrike = optParams.params.marketStrikes(i,j);
                            model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                            out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                            idx = idx + 1;
                        end
                    end
                end
           elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
               for i=1:optParams.params.expirySize
                   fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                    for j=1:optParams.params.strikeSize
                        if optParams.params.calibrationYN(i,j) == 1
                            market_otmPrice = optParams.params.blackOTMPrice(i,j);
                            marketStrike = optParams.params.marketStrikes(i,j);
                            model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                            out(idx) = model_otmPrice/market_otmPrice - 1.0;
                            idx = idx + 1;
                        end
                    end
               end
           else
               error('unknown calibrationErrorType!')
           end

           output = 0.0;
           dummyN = 0;
           for i=1:length(out)
               output = output + out(i)*out(i);
               dummyN = dummyN + 1;
           end

           output = output/dummyN;
        end
        
        function output = BoundaryConditionCheck(eqCOMDupireSpotGF,popt,lb,ub)
            output = true;
            
            if length(popt) ~= length(lb) || length(popt) ~= length(ub)
                output = false;
            end
            
            for i=1:length(popt)
                if popt(i) < lb(i) || popt(i) > ub(i)
                    output = false;
                    break;
                end
            end
            
        end
        
        function out = CalibrateToVolSurfaceHeston(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrice = zeros(expirySize,strikeSize);
            blackOTMPrice2 = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            marketStrikes = zeros(expirySize,strikeSize);
            
            calibrationYN = zeros(expirySize,strikeSize);
            
            strikeBoundExpiry = eqCOMDupireSpotGF.strikeUseBound(:,1);
%             lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(:,2);
%             upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(:,3);
            
            % undiscounted OTM(call for k>1
            if strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                for k=1:expirySize
                    fwd = black.Fwd(spot,expiry(k));
                    idxDummy = find(expiry(k)<=strikeBoundExpiry);
                    if ~isempty(idxDummy)
                        idxDummy2 = min(idxDummy);
                        lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(idxDummy2,2);
                        upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(idxDummy2,3);
            
                    else
                        lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(end,2);
                        upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(end,3);
                    end
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        marketStrikes(k,j) = fwd*strike(j);
                        % multiplied by fwd
                        if fwdMoneyness(k,j) >= lowerStrikeBound && fwdMoneyness(k,j) <= upperStrikeBound
                            calibrationYN(k,j) = 1;
                        end
                        
%                         calibrationYN(k,j) = 1;
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = fwd*black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = fwd*black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwd = black.Fwd(spot,expiry(k));
                    idxDummy = find(expiry(k)<=strikeBoundExpiry);
                    if ~isempty(idxDummy)
                        idxDummy2 = min(idxDummy);
                        lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(idxDummy2,2);
                        upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(idxDummy2,3);
            
                    else
                        lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(end,2);
                        upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(end,3);
                    end
                    
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        marketStrikes(k,j) = fwd*fwdMoneyness(k,j);
                        % multiplied by fwd
                        % test
                        calibrationYN(k,j) = 1;
%                         if fwdMoneyness(k,j) >= lowerStrikeBound && fwdMoneyness(k,j) <= upperStrikeBound
%                             calibrationYN(k,j) = 1;
%                         end
                        
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = fwd*black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = fwd*black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    fwd = black.Fwd(spot,expiry(k));
                    idxDummy = find(expiry(k)<=strikeBoundExpiry);
                    if ~isempty(idxDummy)
                        idxDummy2 = min(idxDummy);
                        lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(idxDummy2,2);
                        upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(idxDummy2,3);
            
                    else
                        lowerStrikeBound = eqCOMDupireSpotGF.strikeUseBound(end,2);
                        upperStrikeBound = eqCOMDupireSpotGF.strikeUseBound(end,3);
                    end
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        marketStrikes(k,j) = fwd*strike(j);
                        % multiplied by fwd
%                         if fwdMoneyness(k,j) >= lowerStrikeBound && fwdMoneyness(k,j) <= upperStrikeBound
%                             calibrationYN(k,j) = 1;
%                         end
                        
                        calibrationYN(k,j) = 1;
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = fwd*black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = fwd*black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                    end
                end
            end
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            params.blackVol = blackVol;
            params.fwdMoneyNess = fwdMoneyness;
            params.marketStrikes = marketStrikes;
            params.calibrationYN = calibrationYN;
            
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            params.blackOTMPrice = blackOTMPrices;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            params.kappa = eqCOMDupireSpotGF.kappa;
            params.theta = eqCOMDupireSpotGF.theta;
            params.rho = eqCOMDupireSpotGF.rho;
            params.sigma = eqCOMDupireSpotGF.sigma;
            params.v0 = eqCOMDupireSpotGF.v0;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.calibrationErrorType;
            %output vol proxy
            out.marketStrikes = params.fwdMoneyNess;
            out.fwdMoneyNess = params.fwdMoneyNess;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.hestonOTMPrice = zeros(expirySize,strikeSize);
            out.hestonBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'globalSigurdroemer'
                    
                    optParams.params = params;
                    psiSize = length(eqCOMDupireSpotGF.psi);
                    optParams.modelParamsSize = 4 + psiSize;  % H, rho, sigma, v0, {psi}
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
%                     tvar(1) = eqCOMDupireSpotGF.kappa;
%                     tvar(1) = eqCOMDupireSpotGF.theta;
%                     tvar(2) = eqCOMDupireSpotGF.rho;
%                     tvar(3) = eqCOMDupireSpotGF.sigma;
%                     tvar(4) = eqCOMDupireSpotGF.v0;
%                     tvar(5) = eqCOMDupireSpotGF.H;
                    
                    tvar(1) = eqCOMDupireSpotGF.rho;
                    tvar(2) = eqCOMDupireSpotGF.sigma;
                    tvar(3) = eqCOMDupireSpotGF.H;
                    
                    tvar(4) = eqCOMDupireSpotGF.v0;
                    tvar(5:5+psiSize-1,1) = eqCOMDupireSpotGF.psi;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [0.0001,-0.9999,0.0001,0.0001,0.0001*ones(1,psiSize)];
%                     ub = [0.5,0.0,100.0,100.0,100.0*ones(1,psiSize)];
                    
%                     lb = [-0.9999,0.0001,0.0001,0.0001,0.0001*ones(1,psiSize)];
                    lb = [-0.9999,0.0001,0.1,0.0001,0.0001*ones(1,psiSize)];
                    ub = [0.0,100.0,0.5,100.0,100.0*ones(1,psiSize)];
                    
                    options = optimoptions('lsqnonlin','Algorithm','trust-region-reflective',...
                    'MaxFunctionEvaluations',10^4,'MaxIterations',10^4,'FunctionTolerance',10^(-12),...
                    'StepTolerance',10^(-4),'FiniteDifferenceStepSize',10^(-4),'Display','off');

                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionRoughHestonRGlobalSigurdroemer',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
%                     [popt, resnorm,residual,exitflag,output1]=lsqnonlin('TargetFunctionRoughHestonRGlobalSigurdroemerVal',tvar,lb,ub,options,eqCOMDupireSpotGF,optParams);
                    [popt, resnorm,residual,exitflag,output1]=lsqnonlin('TargetFunctionRoughHestonRGlobalSigurdroemer',tvar,lb,ub,options,eqCOMDupireSpotGF,optParams);
                    
                    toc

%                     eqCOMDupireSpotGF.H = popt(1);
%                     eqCOMDupireSpotGF.rho = popt(2);
%                     eqCOMDupireSpotGF.sigma = popt(3);
%                     eqCOMDupireSpotGF.v0 = popt(4);
                    
                    
                    
                    eqCOMDupireSpotGF.rho = popt(1);
                    eqCOMDupireSpotGF.sigma = popt(2);
                    eqCOMDupireSpotGF.H = popt(3);
                    eqCOMDupireSpotGF.v0 = popt(4);
                    
                    eqCOMDupireSpotGF.psi = popt(5:5 + psiSize -1,1);
    
%                     
%                     out.H = popt(1);
%                     out.rho = popt(2);
%                     out.sigma = popt(3);
%                     out.v0 = popt(4);
                    
                    out.rho = popt(1);
                    out.sigma = popt(2);
                    out.H = popt(3);
                    
                    out.v0 = popt(4);
                    out.psi = popt(5:5 + psiSize -1,1);
    
                    
                    
                    out.paramsOut = zeros(4 + psiSize,1);
                    
%                     out.paramsOut(1) = out.H;
%                     out.paramsOut(2) = out.rho;
%                     out.paramsOut(3) = out.sigma;
                    
                    out.paramsOut(1) = out.rho;
                    out.paramsOut(2) = out.sigma;
                    out.paramsOut(3) = out.H;
                    
                    out.paramsOut(4) = out.v0;
                    out.paramsOut(5:5 + psiSize -1,1) = out.psi;
                    addParams = struct;
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           n = expiry(i)*eqCOMDupireSpotGF.psiDailyTimeStep;
                           
                           ti  = (0:n) / n * expiry(i)/365.0;
                           dt  = ti(end) - ti(end-1);
                           % initial forward variance curve
                           xi = eqCOMDupireSpotGF.Xicurve(ti);
                           addParams.n = n;
                           addParams.ti = ti;
                           addParams.dt = dt;
                           addParams.xi = xi;
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        aaa = 1.0;
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           n = expiry(i);
                           ti  = (0:n) / n * expiry(i)/365.0;
                           dt  = ti(end) - ti(end-1);
                           % initial forward variance curve
                           xi = eqCOMDupireSpotGF.Xicurve(ti);
                           addParams.n = n;
                           addParams.ti = ti;
                           addParams.dt = dt;
                           addParams.xi = xi;
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           n = expiry(i)*eqCOMDupireSpotGF.psiDailyTimeStep;
                           ti  = (0:n) / n * expiry(i)/365.0;
                           dt  = ti(end) - ti(end-1);
                           % initial forward variance curve
                           xi = eqCOMDupireSpotGF.Xicurve(ti);
                           addParams.n = n;
                           addParams.ti = ti;
                           addParams.dt = dt;
                           addParams.xi = xi;
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        aaa = 1.0;
                    else
                        disp('unImplemented')
                    end
                case 'globalSigurdroemerKappa0'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % rho, sigma, v0, H,  
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.rho;
                    tvar(2) = eqCOMDupireSpotGF.sigma;
                    tvar(3) = eqCOMDupireSpotGF.H;
                    tvar(4) = eqCOMDupireSpotGF.v0;
                    
                    

%                     x=  zeros(optParams.numOfEquation,1);
%                     lb = [0.0001,0.0001,-0.9999,0.0001,0.0001];
%                     ub = [100.0,100.0,0.9999,100.0,100.0];
%                     tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionRoughHestonGlobalSigurdroemerHFixed',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
%                     toc
%                     
%                     lb = [-0.9999,0.0001,0.0001,0.0001];
                    lb = [-0.9999,0.0001,0.1,0.0001];
                    ub = [0.0,100.0,0.5,100.0,];
                    
                    options = optimoptions('lsqnonlin','Algorithm','trust-region-reflective',...
                    'MaxFunctionEvaluations',10^4,'MaxIterations',10^4,'FunctionTolerance',10^(-12),...
                    'StepTolerance',10^(-4),'FiniteDifferenceStepSize',10^(-4),'Display','off');

                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionRoughHestonRGlobalSigurdroemer',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
%                     [popt, resnorm,residual,exitflag,output1]=lsqnonlin('TargetFunctionRoughHestonRGlobalSigurdroemerVal',tvar,lb,ub,options,eqCOMDupireSpotGF,optParams);
                    [popt, resnorm,residual,exitflag,output1]=lsqnonlin('TargetFunctionRoughHestonGlobalSigurdroemerKappa0',tvar,lb,ub,options,eqCOMDupireSpotGF,optParams);
                    
                    toc
                    
                                     
                    eqCOMDupireSpotGF.rho = popt(1);
                    eqCOMDupireSpotGF.sigma = popt(2);
                    eqCOMDupireSpotGF.H = popt(3);
                    eqCOMDupireSpotGF.v0 = popt(4);
                    
                    
%                     eqCOMDupireSpotGF.H = popt(1);
%                     eqCOMDupireSpotGF.rho = popt(2);
%                     eqCOMDupireSpotGF.sigma   = popt(3);
                    
                    out.rho = popt(1);
                    out.sigma = popt(2);
                    out.H = popt(3);
                    out.v0 = popt(4);

                    
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.rho;
                    out.paramsOut(2) = out.sigma;
                    out.paramsOut(3) = out.H;
                    
                    out.paramsOut(4) = out.v0;
                    addParams = struct;
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           n = expiry(i);
                           ti  = (0:n) / n * expiry(i)/365.0;
                           dt  = ti(end) - ti(end-1);
                           % initial forward variance curve
                           xi = eqCOMDupireSpotGF.XicurveKappa0(ti);
                           addParams.n = n;
                           addParams.ti = ti;
                           addParams.dt = dt;
                           addParams.xi = xi;
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        aaa = 1.0;
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 5;  % H, rho, sigma, 
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.kappa;
                    tvar(2) = eqCOMDupireSpotGF.theta;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.sigma;
                    tvar(5) = eqCOMDupireSpotGF.v0;
%                     tvar(6) = eqCOMDupireSpotGF.H;
                    
%                     tvar(1) = eqCOMDupireSpotGF.H;
%                     tvar(2) = eqCOMDupireSpotGF.rho;
%                     tvar(3) = eqCOMDupireSpotGF.sigma;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001,0.0001];
                    ub = [100.0,100.0,0.9999,100.0,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    toc
                    if ret == -1
                        tvar(1) = eqCOMDupireSpotGF.modelParams('kappa');
                        tvar(2) = eqCOMDupireSpotGF.modelParams('theta');
                        tvar(3) = eqCOMDupireSpotGF.modelParams('rho');
                        tvar(4) = eqCOMDupireSpotGF.modelParams('sigma');
                        tvar(5) = eqCOMDupireSpotGF.modelParams('v0');
%                         tvar(6) = eqCOMDupireSpotGF.modelParams('H');
                        
%                         tvar(1) = eqCOMDupireSpotGF.modelParams('H');
%                         tvar(2) = eqCOMDupireSpotGF.modelParams('rho');
%                         tvar(3) = eqCOMDupireSpotGF.modelParams('sigma');
                    
                        x=  zeros(optParams.numOfEquation,1);
                        
                        [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    elseif ~BoundaryConditionCheck(eqCOMDupireSpotGF,popt,lb,ub)
                        tvar(1) = eqCOMDupireSpotGF.modelParams('kappa');
                        tvar(2) = eqCOMDupireSpotGF.modelParams('theta');
                        tvar(3) = eqCOMDupireSpotGF.modelParams('rho');
                        tvar(4) = eqCOMDupireSpotGF.modelParams('sigma');
                        tvar(5) = eqCOMDupireSpotGF.modelParams('v0');
%                         tvar(6) = eqCOMDupireSpotGF.modelParams('H');
                        
%                          tvar(1) = eqCOMDupireSpotGF.modelParams('H');
%                         tvar(2) = eqCOMDupireSpotGF.modelParams('rho');
%                         tvar(3) = eqCOMDupireSpotGF.modelParams('sigma');
                        
                        x=  zeros(optParams.numOfEquation,1);
                        
                        [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    
                    
                        
                    eqCOMDupireSpotGF.kappa = popt(1);
                    eqCOMDupireSpotGF.theta = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.sigma = popt(4);
                    eqCOMDupireSpotGF.v0 = popt(5);
                    
                    
                    out.kappa = popt(1);
                    out.theta = popt(2);
                    out.rho = popt(3);
                    out.sigma = popt(4);
                    out.v0 = popt(5);
                    
                    out.paramsOut = zeros(5,1);
                    out.paramsOut(1) = out.kappa;
                    out.paramsOut(2) = out.theta;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.sigma;
                    out.paramsOut(5) = out.v0;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                case 'globalHFixed'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2;  % rho, sigma, 
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
%                     tvar(1) = eqCOMDupireSpotGF.kappa;
%                     tvar(2) = eqCOMDupireSpotGF.theta;
%                     tvar(3) = eqCOMDupireSpotGF.rho;
%                     tvar(4) = eqCOMDupireSpotGF.sigma;
%                     tvar(5) = eqCOMDupireSpotGF.v0;
%                     tvar(6) = eqCOMDupireSpotGF.H;
                    
%                     tvar(1) = eqCOMDupireSpotGF.H;
                    tvar(1) = eqCOMDupireSpotGF.rho;
                    tvar(2) = eqCOMDupireSpotGF.sigma;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [-0.9999,0.0001];
                    ub = [0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionRoughHestonGlobalHFixed',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    toc
                    if ret == -1
%                         tvar(1) = eqCOMDupireSpotGF.modelParams('kappa');
%                         tvar(2) = eqCOMDupireSpotGF.modelParams('theta');
%                         tvar(3) = eqCOMDupireSpotGF.modelParams('rho');
%                         tvar(4) = eqCOMDupireSpotGF.modelParams('sigma');
%                         tvar(5) = eqCOMDupireSpotGF.modelParams('v0');
%                         tvar(6) = eqCOMDupireSpotGF.modelParams('H');
                        
%                         tvar(1) = eqCOMDupireSpotGF.modelParams('H');
                        tvar(1) = eqCOMDupireSpotGF.modelParams('rho');
                        tvar(2) = eqCOMDupireSpotGF.modelParams('sigma');
                    
                        x=  zeros(optParams.numOfEquation,1);
                        
                        [ret, popt, info, covar]=levmar('TargetFunctionRoughHestonGlobalHFixed',tvar,x, 10, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    elseif ~BoundaryConditionCheck(eqCOMDupireSpotGF,popt,lb,ub)
%                         tvar(1) = eqCOMDupireSpotGF.modelParams('kappa');
%                         tvar(2) = eqCOMDupireSpotGF.modelParams('theta');
%                         tvar(3) = eqCOMDupireSpotGF.modelParams('rho');
%                         tvar(4) = eqCOMDupireSpotGF.modelParams('sigma');
%                         tvar(5) = eqCOMDupireSpotGF.modelParams('v0');
%                         tvar(6) = eqCOMDupireSpotGF.modelParams('H');
                        
%                          tvar(1) = eqCOMDupireSpotGF.modelParams('H');
                        tvar(1) = eqCOMDupireSpotGF.modelParams('rho');
                        tvar(2) = eqCOMDupireSpotGF.modelParams('sigma');
                        
                        x=  zeros(optParams.numOfEquation,1);
                        
                        [ret, popt, info, covar]=levmar('TargetFunctionRoughHestonGlobalHFixed',tvar,x, 10, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    
                    
%                     eqCOMDupireSpotGF.H = popt(1);
                    eqCOMDupireSpotGF.rho = popt(1);
                    eqCOMDupireSpotGF.sigma   = popt(2);
                    
                    
%                     out.H = popt(1);
                    out.rho = popt(1);
                    out.sigma = popt(2);
                   
                    
                    out.paramsOut = zeros(2,1);
%                     out.paramsOut(1) = out.H;
                    out.paramsOut(1) = out.rho;
                    out.paramsOut(2) = out.sigma;
                    
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        ggg = 1.0;
                         
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                case 'globalGA'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 3;  % H,rho,sigma
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.H;
                    tvar(2) = eqCOMDupireSpotGF.rho;
                    tvar(3) = eqCOMDupireSpotGF.sigma;
                    
                    % initial parameters costval
                   
                    
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,-0.9999,0.0001];
                    ub = [100.0,0.9999,100.0];
                    
                    optionsLevmar=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    
%                     [ret, poptLevmar, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, optionsLevmar,'unc',eqCOMDupireSpotGF,optParams);
                    
                    targetfun = @(x) TargetFunctionHestonGlobalGA(eqCOMDupireSpotGF,x,optParams);
                    
%                     tic
%                     options = optimset('fmincon');
%                     options = optimset(options,'algorithm','interior-point');
%                     [parameters, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);


                    optionsGa = gaoptimset(@ga);
%                     optionsGa = gaoptimset(optionsGa,'InitialPopulation',[tvar';parameters']);
%                     optionsGa = gaoptimset(optionsGa,'InitialPopulation',[tvar';poptLevmar']);
                    
                    optionsGa = gaoptimset(optionsGa,'MutationFcn',@mutationadaptfeasible);
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    [popt,fval,exitflag,outut]=ga(targetfun,length(tvar),[],[],[],[],lb,ub,[],optionsGa);
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
%                     toc
                    
                    eqCOMDupireSpotGF.H = popt(1);
                    eqCOMDupireSpotGF.rho = popt(2);
                    eqCOMDupireSpotGF.sigma = popt(3);
                    
                    out.H = popt(1);
                    out.rho = popt(2);
                    out.sigma = popt(3);
                    
                    out.paramsOut = zeros(3,1);
                    out.paramsOut(1) = out.H;
                    out.paramsOut(2) = out.rho;
                    out.paramsOut(3) = out.sigma;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        aaa = 1.0;
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                case 'globalKappaFixed'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % kappa is fixed so  theta, rho, sigma, v0,
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
%                     tvar(1) = eqCOMDupireSpotGF.kappa;
                    tvar(1) = eqCOMDupireSpotGF.theta;
                    tvar(2) = eqCOMDupireSpotGF.rho;
                    tvar(3) = eqCOMDupireSpotGF.sigma;
                    tvar(4) = eqCOMDupireSpotGF.v0;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     x=  zeros(optParams.numOfEquation,1);
%                     lb = [0.0001,0.0001,-0.9999,0.0001,0.0001];
%                     ub = [100.0,100.0,0.9999,100.0,100.0];
                     
                    options=[1E-03,1E-17, 1E-17, 1E-17, 1E-11];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    % one constraint for theta added
%                     x=  zeros(optParams.numOfEquation + 1,1);
                    
                    lb = [0.0001,-0.9999,0.0001,0.0001];
                    ub = [100.0,0.9999,100.0,100.0];
                    
%                     tic
                    [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobalKappaFixed',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        tvar(1) = eqCOMDupireSpotGF.theta;
                        tvar(2) = eqCOMDupireSpotGF.rho;
                        tvar(3) = eqCOMDupireSpotGF.sigma;
                        tvar(4) = eqCOMDupireSpotGF.v0;
                        x=  zeros(optParams.numOfEquation,1);
                        [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobalKappaFixed',tvar,x, 10, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                     elseif ~BoundaryConditionCheck(eqCOMDupireSpotGF,popt,lb,ub)
                        tvar(1) = eqCOMDupireSpotGF.modelParams('theta');
                        tvar(2) = eqCOMDupireSpotGF.modelParams('rho');
                        tvar(3) = eqCOMDupireSpotGF.modelParams('sigma');
                        tvar(4) = eqCOMDupireSpotGF.modelParams('v0');
                        x=  zeros(optParams.numOfEquation,1);
                        [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobalKappaFixed',tvar,x, 10, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                     
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
%                     toc
                    
%                     eqCOMDupireSpotGF.kappa = popt(1);
                    eqCOMDupireSpotGF.theta = popt(2-1);
                    eqCOMDupireSpotGF.rho   = popt(3-1);
                    eqCOMDupireSpotGF.sigma = popt(4-1);
                    eqCOMDupireSpotGF.v0    = popt(5-1);
                    
                    out.kappa = eqCOMDupireSpotGF.kappa;
                    
                    out.theta = popt(2-1);
                    out.rho = popt(3-1);
                    out.sigma = popt(4-1);
                    out.v0 = popt(5-1);
                    
                    out.paramsOut = zeros(5,1);
                    out.paramsOut(1) = out.kappa;
                    out.paramsOut(2) = out.theta;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.sigma;
                    out.paramsOut(5) = out.v0;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                case 'globalKappaConstrained'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 5;  % kappa , rho, sigma, v0,
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              1;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.kappa;
                    tvar(2) = eqCOMDupireSpotGF.theta;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.sigma;
                    tvar(5) = eqCOMDupireSpotGF.v0;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     x=  zeros(optParams.numOfEquation,1);
%                     lb = [0.0001,0.0001,-0.9999,0.0001,0.0001];
%                     ub = [100.0,100.0,0.9999,100.0,100.0];
                     
                    options=[1E-03,1E-17, 1E-17, 1E-17, 1E-11];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001,0.0001];
                    ub = [100.0,100.0,0.9999,100.0,100.0];
                    
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobalKappaConstrained',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);

%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobalKappaFixed',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.kappa = popt(1);
                    eqCOMDupireSpotGF.theta = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.sigma = popt(4);
                    eqCOMDupireSpotGF.v0    = popt(5);
                    
                    out.kappa = popt(1);
                    out.theta = popt(2);
                    out.rho = popt(3);
                    out.sigma = popt(4);
                    out.v0 = popt(5);
                    
                    out.paramsOut = zeros(5,1);
                    out.paramsOut(1) = out.kappa;
                    out.paramsOut(2) = out.theta;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.sigma;
                    out.paramsOut(5) = out.v0;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                    
                case 'JustCalculateError'
                    
%                     optParams.params = params;
%                     optParams.modelParamsSize = 5;  % kappa, theta, rho, sigma, v0,
%                     optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
%                                               0;   % constraint
%                     
%                     tvar = zeros(optParams.modelParamsSize,1);
%                     
%                     tvar(1) = eqCOMDupireSpotGF.kappa;
%                     tvar(2) = eqCOMDupireSpotGF.theta;
%                     tvar(3) = eqCOMDupireSpotGF.rho;
%                     tvar(4) = eqCOMDupireSpotGF.sigma;
%                     tvar(5) = eqCOMDupireSpotGF.v0;
                    
%                     % initial parameters costval
% %                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
%                     x=  zeros(optParams.numOfEquation,1);
%                     lb = [0.0001,0.0001,-0.9999,0.0001,0.0001];
%                     ub = [100.0,100.0,0.9999,100.0,100.0];
%                     tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
% %                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqCOMDupireSpotGF.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
%                     
%                     toc
%                     
%                     eqCOMDupireSpotGF.kappa = popt(1);
%                     eqCOMDupireSpotGF.theta = popt(2);
%                     eqCOMDupireSpotGF.rho   = popt(3);
%                     eqCOMDupireSpotGF.sigma = popt(4);
%                     eqCOMDupireSpotGF.v0    = popt(5);
                    
                    out.kappa = eqCOMDupireSpotGF.kappa;
                    out.theta = eqCOMDupireSpotGF.theta;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.sigma = eqCOMDupireSpotGF.sigma;
                    out.v0 = eqCOMDupireSpotGF.v0;
                    out.H  = eqCOMDupireSpotGF.H;
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.kappa;
                    out.paramsOut(2) = out.theta;
                    out.paramsOut(3) = out.H;
                    out.paramsOut(4) = out.rho;
                    out.paramsOut(5) = out.sigma;
                    out.paramsOut(6) = out.v0;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           n = expiry(i)*eqCOMDupireSpotGF.psiDailyTimeStep;
                           
                           ti  = (0:n) / n * expiry(i)/365.0;
                           dt  = ti(end) - ti(end-1);
                           % initial forward variance curve
                           xi = eqCOMDupireSpotGF.XicurveKappa0(ti);
                           addParams.n = n;
                           addParams.ti = ti;
                           addParams.dt = dt;
                           addParams.xi = xi;
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        aaa = 0.0;
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        
                    elseif strcmp(params.calibrationErrorType,'relativePriceErrorSS')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                            
                    elseif strcmp(params.calibrationErrorType,'relativePriceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice/market_otmPrice - 1.0;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    else
                        disp('unImplemented')
                    end
                    
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMCSSSABR(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
%             detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = spotPrice * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMCCLVBSATM(eqCOMDupireSpotGF,observeTime,modelStates)
         if observeTime == 0
             out = exp(modelStates);
         else
             out = eqCOMDupireSpotGF.MappingFunctionBSATM(observeTime/365.0,modelStates);
             
         end
         
        end
        
        function out = EQFwdMCHestonCLV(eqCOMDupireSpotGF,observeTime,modelStates)
         if observeTime == 0
             out = modelStates;
         else
             out = eqCOMDupireSpotGF.MappingFunctionHeston(observeTime/365.0,modelStates);
             
         end
         
        end
        
        function out = EQFwdMCCLV(eqCOMDupireSpotGF,observeTime,modelStates)
         if observeTime == 0
             out = modelStates;
         else
             out = eqCOMDupireSpotGF.MappingFunction(observeTime/365.0,modelStates);
             
         end
         
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)

         out = modelStates;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCReducedNMonths(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = computeLocalVolGridSSSABR(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
                    f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                    
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
%                     
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib_FwdMN(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                     if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                     end
                    
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;

                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup*detFwd_f(idxjj)/detFwd(idxjj),f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);

                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeCalendarSpreadOptionMCReducedInt(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = GenerateLocalVolSS(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSSReduced(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end

                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib_FwdMN(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = computeForwardStartingVanillaMCCLVBSATM(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % resetDate
            % expiryDate
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
%             
% %             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            timeStep = sort(union(timeStep,maturity),'ascend');
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
%            initX = ones(NMC,1);
           initX = zeros(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(NMC,1);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== 1
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewCLVBSATM(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                % at startDate, we reset strike
                if i==1
                    comFwd = eqCOMDupireSpotGF.EQFwdMCCLVBSATM(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCCLVBSATM(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeForwardStartingVanillaMCCLV(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % resetDate
            % expiryDate
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
%             
% %             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            timeStep = sort(union(timeStep,maturity),'ascend');
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            initX = zeros(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(NMC,1);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== 1
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewCLV(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                % at startDate, we reset strike
                if i==1
                    comFwd = eqCOMDupireSpotGF.EQFwdMCCLV(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCCLV(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeForwardStartingVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % resetDate
            % expiryDate
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(NMC,1);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== 1
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                % at startDate, we reset strike
                if i==1
                    comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeStepdown1SMCReducedInt(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReducedIntEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
%            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQSSSABR(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMCSSSABR(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMCSSSABR(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMCHestonSLV(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = vanillaParams.params('maturityDate');
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;

%             expiry = eqCOMDupireSpotGF.localVolSurface.maturities;
            expiry = eqCOMDupireSpotGF.levFuncInfo.timeStepLVF;
            
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            timeStep = sort(union(timeStep,maturity),'ascend');
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            
            
            
            
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
                        mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQESLV(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                        mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMCHeston(eqHeston,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            if isfield(eqCOMDupireSpotGF.localVolSurface,'expiry')
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            else
                expiry = eqCOMDupireSpotGF.localVolSurface.maturities;
            end
            
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
                        mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                        mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCHestonSLV(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             if isfield(eqCOMDupireSpotGF.localVolSurface,'expiry')
%                 expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             else
%                 expiry = eqCOMDupireSpotGF.localVolSurface.maturities;
%             end
%             

            expiry = eqCOMDupireSpotGF.levFuncInfo.timeStepLVF;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
%             if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;

            if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
                        mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQESLV(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                        mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
            end
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    if isKI(i)
                        payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                        payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    end
               end
           else
               for i=1:modelStatesSize
                    if isKI(i)
                        payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                        payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardStartingVanillaMCHestonSLV(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = vanillaParams.params('maturityDate');
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;

%             expiry = eqCOMDupireSpotGF.localVolSurface.maturities;
            expiry = eqCOMDupireSpotGF.levFuncInfo.timeStepLVF;
            
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            timeStep = sort(union(timeStep,maturity),'ascend');
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            
            
            
            
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
%             
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(NMC,1);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== lastAliveExerciseIdx
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
                        mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQESLV(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                        mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                
                % at startDate, we reset strike
                if i==lastAliveExerciseIdx
                    comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeBarrierMCHeston(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            if isfield(eqCOMDupireSpotGF.localVolSurface,'expiry')
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            else
                expiry = eqCOMDupireSpotGF.localVolSurface.maturities;
            end
            
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;

            if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
                        mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                        mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
            end
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    if isKI(i)
                        payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                        payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    end
               end
           else
               for i=1:modelStatesSize
                    if isKI(i)
                        payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                        payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardStartingVanillaMCHeston(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                
                afvSim = AFVSimulation;
                
                params0 = struct;
                params0.al = eqCOMDupireSpotGF.H + 0.5;
                params0.lam = 0.0;
                params0.nu = eqCOMDupireSpotGF.sigma;
                params0.eta = params0.nu/(sqrt(2.0.*params0.al-1).*gamma(params0.al));
                params0.rho = eqCOMDupireSpotGF.rho;
                params0.H = eqCOMDupireSpotGF.H;
                v0 = eqCOMDupireSpotGF.v0;
                xi = @(s) (0.071923356959668 + 0.*s);
                expiryRHeston = timeStep(end)/365.0;
                
                mcScheme = eqCOMDupireSpotGF.modelParams('MCScheme');
                if strcmp(mcScheme,'RSQE')
                    rsqeSim = afvSim.RSQESimHist(params0, xi,NMC, MCTimeStep, expiryRHeston);
                    xhist = rsqeSim.xhist;
                elseif strcmp(mcScheme,'HQE')
                    hqeSim = afvSim.HQESimHist(params0, xi,NMC, MCTimeStep, expiryRHeston);
                    xhist = hqeSim.xhist;
                end
                aaa = 1.0;
%                 params0 = struct('al',0.600044293714652,'lam',0,'eta',0.458266093945180,'rho',-0.764593637956381,'H',0.100044293714652);
%                 params0.nu = 0.458266093945180;
%                 params0.eta = params0.nu/(sqrt(2.0.*params0.al-1).*gamma(params0.al));
%                 xi = @(s) (0.071923356959668 + 0.*s);
%                 
%                 numOfFactors = 2;
%                 Z= randn(NMC/2,MCTimeStep*numOfFactors);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
          relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
           reSetS = zeros(NMC,1);
           
%            strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== lastAliveExerciseIdx
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
%                         mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
%                         mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
%                         mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
%                         mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
%                     else
%                         error('unImplemented MC Scheme');
%                     end
%                     
%                     
%                     nextX = mcmcOut.nextX;
%                     nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                
                % at startDate, we reset strike
                if i==lastAliveExerciseIdx
                    comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           dummyX = xhist(lastTimeIdx-1,:)';
           nextX  = exp(dummyX);
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHeston(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           
           out.fwdRatio = fwdRatio;
           
        end
        
        function out = computeForwardStartingVanillaMCHestonCLV(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
               
               
%                % correlate brownian motion
%                rho = eqCOMDupireSpotGF.rho;
%                UTemp = U;
%                
%                for i=1:totalTimeStepsSize-1
%                    U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                    U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                numOfFactors = 2;
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
                
%                 % correlate brownian motion
%                 % correlate brownian motion
%                 rho = eqCOMDupireSpotGF.rho;
%                 UTemp = U;
%                
%                 for i=1:totalTimeStepsSize-1
%                     U(:,2*(i-1)+1) = sqrt(1-rho^2)*UTemp(:,2*(i-1)+1) + rho*UTemp(:,2*(i-1)+2);
%                     U(:,2*(i-1)+2) =                                    1.0*UTemp(:,2*(i-1)+2); 
%                 end

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           %  X  = S(t)/F(0,t)  : spot process
           %  V  = instantaneous variance proces
           
           initX = ones(NMC,1);
           initV = ones(NMC,1)*eqCOMDupireSpotGF.v0;
           
           nextX = initX;
           nextV = initV;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
          relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
           reSetS = zeros(NMC,1);
           
%            strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== lastAliveExerciseIdx
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    if strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'Milstein')
                        mcmcOut   = inductMCForwardMilstein(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    elseif strcmp(eqCOMDupireSpotGF.modelParams('MCScheme'), 'NonCentralChiSquare')
                        mcmcOut   = inductMCForwardNCCS(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,2*(idx-1)+1),U(:,2*(idx-1)+2));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                
                % at startDate, we reset strike
                if i==lastAliveExerciseIdx
                    comFwd = eqCOMDupireSpotGF.EQFwdMCHestonCLV(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCHestonCLV(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           
           out.fwdRatio = fwdRatio;
           
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function mcmcOut = inductMCForwardEuler(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastV,Zs,Zv)
            
            % Zs , Zv are independent normal browninan motion
            
            predictorS = lastX;
            predictorV = lastV;
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.kappa;
            rho   = eqCOMDupireSpotGF.rho;
            
            
            for i=1:length(lastX)
                predictorV(i) = lastV(i) + kappa*(theta - lastV(i))*Dt + sigma*sqrt(lastV(i)*Dt)*Zv(i);
                % full truncation for variance process
                predictorV(i) = max(predictorV(i),0.0);
                
                Zsc = rho*Zv(i) + sqrt(1.0-rho^2)*Zs(i);
                predictorS(i) = lastX(i)*exp(-0.5*lastV(i)*Dt + sqrt(lastV(i)*Dt)*Zsc);
            end
            
            mcmcOut.nextX = predictorS;
            mcmcOut.nextV = predictorV;
        end
        
        function mcmcOut = inductMCForwardMilstein(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastV,Zs,Zv)
            
            % Zs , Zv are independent normal browninan motion
            
            predictorS = lastX;
            predictorV = lastV;
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.kappa;
            rho   = eqCOMDupireSpotGF.rho;
            
            
            for i=1:length(lastX)
                predictorV(i) = lastV(i) + kappa*(theta - lastV(i))*Dt + sigma*sqrt(lastV(i)*Dt)*Zv(i) ...
                                + 0.25*sigma^2*Dt*(Zv(i)^2-1);
                % full truncation for variance process
                predictorV(i) = max(predictorV(i),0.0);
                
                Zsc = rho*Zv(i) + sqrt(1.0-rho^2)*Zs(i);
                predictorS(i) = lastX(i)*exp(-0.5*lastV(i)*Dt + sqrt(lastV(i)*Dt)*Zsc);
            end
            
            mcmcOut.nextX = predictorS;
            mcmcOut.nextV = predictorV;
        end
        
        function mcmcOut = inductMCForwardNCCS(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastV,Zs,Zv)
            
            % for Variance Process we sample from the Noncentral Chi Square Distribution
            % Zs , Zv are independent normal browninan motion
            
            predictorS = lastX;
            predictorV = lastV;
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            
            % Andersen's Quadratic Exponential Settings
            gamma1 = 0.5;
            gamma2 = 0.5;
            phic = 1.5;
%             ApplyMCYN = 'YES';
            ApplyMCYN = 'NO';
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.kappa;
            rho   = eqCOMDupireSpotGF.rho;
            
            % Parameters for the QE scheme
            E = exp(-kappa*Dt);  % Simplify the coding
            K0 = -kappa*rho*theta*Dt/sigma;
            K1 = (kappa*rho/sigma - 1/2)*gamma1*Dt - rho/sigma;
            K2 = (kappa*rho/sigma - 1/2)*gamma2*Dt + rho/sigma;
            K3 = gamma1*Dt*(1-rho^2);
            K4 = gamma2*Dt*(1-rho^2);
            A = K2 + K4/2;
            
            C = sigma^2*(1-E)/(4*kappa);
            dim = 4*kappa*theta/(sigma^2);
            Dorg = 4*kappa*E/(sigma^2)/(1-E);
            D = Dorg*lastV;
            predictorV = C * ncx2rnd(dim,D,length(lastX),1);
            for i=1:length(lastX)
%                 Uv = normcdf(Zv(i));
%                 D = Dorg*lastV(i);
                
%                 predictorV(i) = C * ncx2inv(Uv,dim,D);
                % Martingale Correction
                if (strcmp(ApplyMCYN,'YES')) && (A < 1/(2*C))
                    M = exp(A*Dorg*C/(1-2*A*C))/(sqrt(1-2*A*C))^(dim);
                    K0 = -log(M) - (K1 + 0.5*K3)*lastV(i);
                end
                    
                predictorS(i) = lastX(i)*exp(K0+K1*lastV(i) + K2*predictorV(i) + ...
                                sqrt(K3*lastV(i)+K4*predictorV(i))*Zs(i));
                
            end
            
            mcmcOut.nextX = predictorS;
            mcmcOut.nextV = predictorV;
        end
        
        function mcmcOut = inductMCForwardQE(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastV,Zs,Zv)
            
            % U (NMC,numOfFactors)  independent brownian motion
            
            % Zs , Zv are independent normal browninan motion
            
            predictorS = lastX;
            predictorV = lastV;
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            
            % Andersen's Quadratic Exponential Settings
            gamma1 = 0.5;
            gamma2 = 0.5;
            phic = 1.5;
%             ApplyMCYN = 'YES';
            ApplyMCYN = 'NO';
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            rho   = eqCOMDupireSpotGF.rho;
            
            % Parameters for the QE scheme
            E = exp(-kappa*Dt);  % Simplify the coding
            K0 = -kappa*rho*theta*Dt/sigma;
            K1 = (kappa*rho/sigma - 1/2)*gamma1*Dt - rho/sigma;
            K2 = (kappa*rho/sigma - 1/2)*gamma2*Dt + rho/sigma;
            K3 = gamma1*Dt*(1-rho^2);
            K4 = gamma2*Dt*(1-rho^2);
            A = K2 + K4/2;
            
            for i=1:length(lastX)
                m  = theta + (lastV(i) - theta)*E;
                s2 = lastV(i)*sigma^2*E/kappa*(1-E) + theta*sigma^2/2/kappa*(1-E)^2;
                phi = s2/m^2;
                
                if phi <= phic
                    % moment matching for large variance nonCentral Chi
                    % Square Distribution
                    % large variance -> large noncentrality
                    b = sqrt(2/phi -1 + sqrt(2/phi*(2/phi-1)));
                    a = m/(1+b^2);
                    predictorV(i) = a*(b+Zv(i))^2;
                    % Martingale Correction
                    if (strcmp(ApplyMCYN,'YES')) && (A < 1/(2*a))
                        M = exp(A*b^2*a/(1-2*A*a))/sqrt(1-2*A*a);
                        K0 = -log(M) - (K1 + 0.5*K3)*lastV(i);
                    end

                    predictorS(i) = lastX(i)*exp(K0+K1*lastV(i) + K2*predictorV(i) + ...
                                    sqrt(K3*lastV(i)+K4*predictorV(i))*Zs(i));
                    
                else
                    % moment matching for small variance
                    p = (phi -1)/(phi +1);
                    beta = (1-p)/m;
                    Uv = normcdf(Zv(i));
                    if (0.0 <= Uv) && (Uv <= p)
                        phiinv = 0.0;
                    elseif (p < Uv) && (Uv <=1)
                        phiinv = 1/beta*log((1-p)/(1-Uv));
                    end
                    predictorV(i) = phiinv;
                    % Martingale Correction
                    if (strcmp(ApplyMCYN,'YES')) && (A < beta)
                        M = p + beta*(1-p)/(beta -A);
                        K0 = -log(M) -(K1 + 0.5*K3)*lastV(i);
                    end

                    predictorS(i) = lastX(i)*exp(K0+K1*lastV(i) + K2*predictorV(i) + ...
                                    sqrt(K3*lastV(i)+K4*predictorV(i))*Zs(i));
                end

            end
            
            mcmcOut.nextX = predictorS;
            mcmcOut.nextV = predictorV;
        end
        
        function mcmcOut = inductMCForwardQESLV(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastV,Zs,Zv)
            % SLV dynamics for HestonSLV model
            % we augment heston dynamics with leverageFunction L(t,S)
            % we do not apply martingale correction for simplicity
            
            % Zs , Zv are independent normal browninan motion
            
            predictorS = lastX;
            predictorV = lastV;
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            
            % Andersen's Quadratic Exponential Settings
            gamma1 = 0.5;
            gamma2 = 0.5;
            phic = 1.5;
%             ApplyMCYN = 'YES';
            ApplyMCYN = 'NO';
            
            kappa = eqCOMDupireSpotGF.kappa;
            theta = eqCOMDupireSpotGF.theta;
            sigma = eqCOMDupireSpotGF.sigma;
            v0    = eqCOMDupireSpotGF.v0;
            rho   = eqCOMDupireSpotGF.rho;
            
            % Parameters for the QE scheme
            E = exp(-kappa*Dt);  % Simplify the coding
%             K0 = -kappa*rho*theta*Dt/sigma;
%             K1 = (kappa*rho/sigma - 1/2)*gamma1*Dt - rho/sigma;
%             K2 = (kappa*rho/sigma - 1/2)*gamma2*Dt + rho/sigma;
%             K3 = gamma1*Dt*(1-rho^2);
%             K4 = gamma2*Dt*(1-rho^2);
%             A = K2 + K4/2;
            lf = eqCOMDupireSpotGF.GetLevFunc(fromTime + 0.01,lastX);
            for i=1:length(lastX)
%                 lf = eqCOMDupireSpotGF.GetLevFunc(fromTime + 0.01,lastX(i));
                K0 = -kappa*rho*theta*Dt/sigma*lf(i);
                K1 = (kappa*rho*lf(i)/sigma - 1/2.0*lf(i)*lf(i))*gamma1*Dt - rho*lf(i)/sigma;
                K2 = (kappa*rho*lf(i)/sigma - 1/2.0*lf(i)*lf(i))*gamma2*Dt + rho*lf(i)/sigma;
                K3 = gamma1*Dt*(1-rho^2)*lf(i)*lf(i);
                K4 = gamma2*Dt*(1-rho^2)*lf(i)*lf(i);
                
                m  = theta + (lastV(i) - theta)*E;
                s2 = lastV(i)*sigma^2*E/kappa*(1-E) + theta*sigma^2/2/kappa*(1-E)^2;
                phi = s2/m^2;
                
                if phi <= phic
                    % moment matching for large variance nonCentral Chi
                    % Square Distribution
                    % large variance -> large noncentrality
                    b = sqrt(2/phi -1 + sqrt(2/phi*(2/phi-1)));
                    a = m/(1+b^2);
                    predictorV(i) = a*(b+Zv(i))^2;
%                     % Martingale Correction
%                     if (strcmp(ApplyMCYN,'YES')) && (A < 1/(2*a))
%                         M = exp(A*b^2*a/(1-2*A*a))/sqrt(1-2*A*a);
%                         K0 = -log(M) - (K1 + 0.5*K3)*lastV(i);
%                     end
                    
                    predictorS(i) = lastX(i)*exp(K0+K1*lastV(i) + K2*predictorV(i) + ...
                                    sqrt(K3*lastV(i)+K4*predictorV(i))*Zs(i));
                    
                else
                    % moment matching for small variance
                    p = (phi -1)/(phi +1);
                    beta = (1-p)/m;
                    Uv = normcdf(Zv(i));
                    if (0.0 <= Uv) && (Uv <= p)
                        phiinv = 0.0;
                    elseif (p < Uv) && (Uv <=1)
                        phiinv = 1/beta*log((1-p)/(1-Uv));
                    end
                    predictorV(i) = phiinv;
%                     % Martingale Correction
%                     if (strcmp(ApplyMCYN,'YES')) && (A < beta)
%                         M = p + beta*(1-p)/(beta -A);
%                         K0 = -log(M) -(K1 + 0.5*K3)*lastV(i);
%                     end
                    
                    predictorS(i) = lastX(i)*exp(K0+K1*lastV(i) + K2*predictorV(i) + ...
                                    sqrt(K3*lastV(i)+K4*predictorV(i))*Zs(i));
                end
                
            end
            
            mcmcOut.nextX = predictorS;
            mcmcOut.nextV = predictorV;
        end
        
        function out = EQFwdMCHeston(eqCOMDupireSpotGF,observeTime,modelStates)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);

             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end

             out = stochasticFwd;
        end
        
        function out = EQFwdMCHestonR(eqCOMDupireSpotGF,observeTime,modelStates)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 

             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);

             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end

             out = stochasticFwd;
        end
        
        function out = CDFReghai(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = input_k;
            dk = 1e-3*k;
            kMoneynessUp = (k+dk)/fwd;
            dummyUp = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(term*365.0,kMoneynessUp,eqCOMDupireSpotGF.paramsIn);
            blackVolUp = dummyUp.impVol;
            
            kMoneynessDn = (k-dk)/fwd;
            dummyDn = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(term*365.0,kMoneynessDn,eqCOMDupireSpotGF.paramsIn);
            blackVolDn = dummyDn.impVol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            kMoneyness = k/fwd;
            dummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(term*365.0,kMoneyness,eqCOMDupireSpotGF.paramsIn);
            blackVol = dummy.impVol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
%                 alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
%                 alpha = -1.0 + H_ncdf(d1);
%                 alpha = -H_ncdf(-d1);
                DalphaDd1 = H_ndf(d1);
%                 beta = 1.0 - H_ncdf(d2);
                beta = H_ncdf(-d2);
                
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        
        function out = CDFHeston(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = input_k;
            dk = 1e-3*k;
%             dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
%             blackVolUp = dummyUp.vol;
%             
%             dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
%             blackVolDn = dummyDn.vol;
            
            if fwd <= input_k
                % put option
                price_up = eqCOMDupireSpotGF.modelOTMPrice(fwd,k+dk,term*365.0,'Put');
                price_dn = eqCOMDupireSpotGF.modelOTMPrice(fwd,k-dk,term*365.0,'Put');
                dPut_dk = (price_up - price_dn)/(2*dk);
                out = dPut_dk;
            else
                % call option
                price_up = eqCOMDupireSpotGF.modelOTMPrice(fwd,k+dk,term*365.0,'Call');
                price_dn = eqCOMDupireSpotGF.modelOTMPrice(fwd,k-dk,term*365.0,'Call');
                dCall_dk = (price_up - price_dn)/(2*dk);
                out = 1.0 + dCall_dk;
            end
            
        end
        
        function out = CallPutDecision(eqCOMDupireSpotGF,x,fwd)
                 out = ones(size(x));
                 idx1 = x <= fwd;
                 out(idx1) = 0;
        end
        
        function out = CDFHestonVector(eqCOMDupireSpotGF,input_kt,term)
            input_k = input_kt';
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = input_k;
            dk = 1e-3*k;
%             dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
%             blackVolUp = dummyUp.vol;
%             
%             dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
%             blackVolDn = dummyDn.vol;

            % put:0 call :1
            callPutFlag = CallPutDecision(eqCOMDupireSpotGF,k,fwd);
            
            put_price_up = eqCOMDupireSpotGF.modelOTMPriceVector(fwd,k+dk,term*365.0,'Put');
            put_price_dn = eqCOMDupireSpotGF.modelOTMPriceVector(fwd,k-dk,term*365.0,'Put');
            dPut_dk = (put_price_up - put_price_dn)./(2*dk);
            outP = dPut_dk;
            
            call_price_up = eqCOMDupireSpotGF.modelOTMPriceVector(fwd,k+dk,term*365.0,'Call');
            call_price_dn = eqCOMDupireSpotGF.modelOTMPriceVector(fwd,k-dk,term*365.0,'Call');
            dCall_dk = (call_price_up - call_price_dn)./(2*dk);
            outC = 1.0 + dCall_dk;
            
            outT = (1 - callPutFlag).*outP + callPutFlag.*outC;
            out = outT';
                
%             if fwd <= input_k
%                 % put option
%                 price_up = eqCOMDupireSpotGF.modelOTMPrice(fwd,k+dk,term*365.0,'Put');
%                 price_dn = eqCOMDupireSpotGF.modelOTMPrice(fwd,k-dk,term*365.0,'Put');
%                 dPut_dk = (price_up - price_dn)/(2*dk);
%                 out = dPut_dk;
%             else
%                 % call option
%                 price_up = eqCOMDupireSpotGF.modelOTMPrice(fwd,k+dk,term*365.0,'Call');
%                 price_dn = eqCOMDupireSpotGF.modelOTMPrice(fwd,k-dk,term*365.0,'Call');
%                 dCall_dk = (price_up - price_dn)/(2*dk);
%                 out = 1.0 + dCall_dk;
%             end
            
        end
       
        
        function out = CDF2(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = input_k;
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            if fwd <= input_k
                % put option
                price_up = blackPrice(fwd,k+dk,blackVolUp,term,-1);
                price_dn = blackPrice(fwd,k-dk,blackVolDn,term,-1);
                dPut_dk = (price_up - price_dn)/(2*dk);
                out = dPut_dk;
            else
                % call option
                price_up = blackPrice(fwd,k+dk,blackVolUp,term,1);
                price_dn = blackPrice(fwd,k-dk,blackVolDn,term,1);
                dCall_dk = (price_up - price_dn)/(2*dk);
                out = 1.0 + dCall_dk;
            end
            
        end
        
        function out = CDF(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = input_k;
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k,term);
            blackVol = dummy.vol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
%                 alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
%                 alpha = -1.0 + H_ncdf(d1);
%                 alpha = -H_ncdf(-d1);
                DalphaDd1 = H_ndf(d1);
%                 beta = 1.0 - H_ncdf(d2);
                beta = H_ncdf(-d2);
                
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        
        function out = CDF_Log(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = exp(input_k);
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k,term);
            blackVol = dummy.vol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
%                 alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
%                 alpha = -1.0 + H_ncdf(d1);
%                 alpha = -H_ncdf(-d1);
                DalphaDd1 = H_ndf(d1);
%                 beta = 1.0 - H_ncdf(d2);
                beta = H_ncdf(-d2);
                
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        
        function out =  InvCDFReghai(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            
%             dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
%             blackVol = dummy.vol;
            
            dummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(term*365.0,1.0,eqCOMDupireSpotGF.paramsIn);
            blackVol = dummy.impVol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
%             guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess/lower < 1e12 && CDFReghai(eqCOMDupireSpotGF,lower,term) > y
                lower = lower/2.0;
            end
            
            upper = guess;
            while upper/guess < 1e12 && CDFReghai(eqCOMDupireSpotGF,upper,term) < y
                upper = upper*2.0;
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDFReghai(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch ME
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out =  InvCDFHeston(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
%             guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess/lower < 1e12 && CDFHeston(eqCOMDupireSpotGF,lower,term) > y
                lower = lower/2.0;
            end
            
            upper = guess;
            while upper/guess < 1e12 && CDFHeston(eqCOMDupireSpotGF,upper,term) < y
                upper = upper*2.0;
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDFHeston(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out =  InvCDF2(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
%             guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess/lower < 1e12 && CDF2(eqCOMDupireSpotGF,lower,term) > y
                lower = lower/2.0;
            end
            
            upper = guess;
            while upper/guess < 1e12 && CDF2(eqCOMDupireSpotGF,upper,term) < y
                upper = upper*2.0;
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDF2(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out =  InvCDF(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
%             guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess/lower < 1e12 && CDF(eqCOMDupireSpotGF,lower,term) > y
                lower = lower/2.0;
            end
            
            upper = guess;
            while upper/guess < 1e12 && CDF(eqCOMDupireSpotGF,upper,term) < y
                upper = upper*2.0;
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDF(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out =  InvCDF_Log(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
            guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess - lower < 10000 && CDF(eqCOMDupireSpotGF,lower,term) > y
                lower = lower  - log(2.0);
            end
            
            upper = guess;
            while upper - guess < 10000 && CDF(eqCOMDupireSpotGF,upper,term) < y
                upper = upper + log(2.0);
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDF(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out = CollocationPointsY(eqCOMDupireSpotGF,term)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            out = zeros(length(xBar),1);
            for idxDum=1:length(xBar)
                out(idxDum) = eqCOMDupireSpotGF.InvCDF(H_ncdf(xBar(idxDum)),term);
            end
        end
        
        function out = BSATM_Expectation(eqCOMDupireSpotGF,t,x0,dt)
            fromDay = t*365.0;
            toDay = fromDay + dt*365.0;
            
            fwdFactorFrom = eqCOMDupireSpotGF.FwdFactor(fromDay);
            fwdFactorTo = eqCOMDupireSpotGF.FwdFactor(toDay);
            
            out = log(fwdFactorTo/fwdFactorFrom);
        end
        
        function out = BSATM_StdDeviation(eqCOMDupireSpotGF,t,x0,dt)
            from = t*365.0;
            to = from + dt*365.0;
            
            sig = eqCOMDupireSpotGF.BSATMVol;
            sigSize = length(sig);
            tenor = eqCOMDupireSpotGF.maturityTimes*365.0;
            
            for i=1:length(tenor)
                if from<= tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(tenor)
                if to<= tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = 0.0;
           vol = 0.0;
           for i=startIdx:endIdx
                if (i< endIdx) U = tenor(i);else U = to;end
                if (i< sigSize) vol = sig(i);else vol = sig(sigSize);end
                scale = U - lastU;
                variances = variances + vol*vol * scale;
                lastU = U;
           end
            
            variances = variances/365.0;
            out = sqrt(variances);
        end
        
        function out = OU_Expectation(eqCOMDupireSpotGF,t,x0,dt)
            level_OU = eqCOMDupireSpotGF.level_OU;
            speed_OU = eqCOMDupireSpotGF.speed_OU;
            
            out = level_OU + (x0 - level_OU)*exp(-speed_OU*dt);
        end
        
        function out = OU_StdDeviation(eqCOMDupireSpotGF,t,x0,dt)
            vol_OU   = eqCOMDupireSpotGF.vol_OU;
            speed_OU = eqCOMDupireSpotGF.speed_OU;
            
            variance = 0.5*vol_OU*vol_OU/speed_OU*(1-exp(-2.0*speed_OU*dt));
            out = sqrt(variance);
        end
        
        function out = CollocationPointsX(eqCOMDupireSpotGF,term)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            x0_OU = eqCOMDupireSpotGF.x0_OU;
            expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
            stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            out = expectation + stdDeviation*xBar;
            
        end
        
        function out  = PiecewiseCubicHermiteInterpExtrapLinear(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            % Left & Right Linear Extrapolation !!
            if r < xBar(1)
                pp = pchip(xBar,yBar);
                lhPol = pp.coefs(1,:);
                out = (r-xBar(1))*lhPol(3) + yBar(1);
            elseif r > xBar(end)
                pp = pchip(xBar,yBar);
                rhPol = pp.coefs(end,:);
                rhPolD = [rhPol(1)*3,rhPol(2)*2,rhPol(3)];
                slopeR = polyval(rhPolD,xBar(end)-xBar(end-1));
                out = (r- xBar(end))*slopeR + yBar(end);
            else
                out = pchip(xBar,yBar,r);
            end
        end
        
        function out  = PiecewiseCubicHermiteInterpExtrapFlat(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            % Left & Right Linear Extrapolation !!
            out = zeros(length(r),1);
            for idxDD=1:length(r)
                if r(idxDD) < xBar(1)
                    out(idxDD) = yBar(1);
                elseif r(idxDD) > xBar(end)
                    out(idxDD) = yBar(end);
                else
                    out(idxDD) = pchip(xBar,yBar,r(idxDD));
                end
            end
        end
        
        function out  = PiecewiseCubicHermiteInterp(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            out = pchip(xBar,yBar,r);
            
        end
        
        function out  = LagrangeInterp(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            value = 0.0;
            for i=1:length(yBar)
                % if r is close to the knot points then no need interpol
                if r == xBar(i)
                    value = yBar(i);
                    break;
                end
                
                weight_i = 1.0;
                for j=1:length(xBar)
                    if j ~= i
                    weight_i = weight_i * (r-xBar(j))/(xBar(i) - xBar(j));
                    end
                end
                value = value + weight_i * yBar(i);
            end
            out = value;
        end
        
        function out = MappingFunctionBSATM(eqCOMDupireSpotGF,term,x_OU)
            mappingTable  = eqCOMDupireSpotGF.mappingTable;
            maturityTimes = eqCOMDupireSpotGF.maturityTimes;
            
            yBar = zeros(size(mappingTable,1),1);
            for i=1:length(yBar)
                sBar = mappingTable(i,:);
%                 yBar(i) = interp1(maturityTimes',sBar,term,'linear','extrap');
                yBar(i) = interp1ExtrapLinearHong(maturityTimes',sBar,term);
            end
            
            x0_OU_log = log(eqCOMDupireSpotGF.x0_OU);
            expectation  = eqCOMDupireSpotGF.BSATM_Expectation(0.0,x0_OU_log,term);
            stdDeviation = eqCOMDupireSpotGF.BSATM_StdDeviation(0.0,x0_OU_log,term);
            
            r = (x_OU - (expectation-0.5*stdDeviation*stdDeviation))/stdDeviation;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            
            r = gridStreching_sigma * r;
            
%             out = eqCOMDupireSpotGF.LagrangeInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
%             yBarNew = log(yBar);
            yBarNew = yBar;
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            if strcmp( eqCOMDupireSpotGF.modelParams('collocationExtrapolation'),'Flat')
%                 out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapFlat(yBarNew,r);
                out1 = pchipFlatExtrap(xBar,yBarNew,r);
            else
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
            end
%             out = exp(out1);
            out = out1;
            
        end
        
        function out = MappingFunction(eqCOMDupireSpotGF,term,x_OU)
            mappingTable  = eqCOMDupireSpotGF.mappingTable;
            maturityTimes = eqCOMDupireSpotGF.maturityTimes;
            
            yBar = zeros(size(mappingTable,1),1);
            for i=1:length(yBar)
                sBar = mappingTable(i,:);
%                 yBar(i) = interp1(maturityTimes',sBar,term,'linear','extrap');
                yBar(i) = interp1ExtrapLinearHong(maturityTimes',sBar,term);
            end
            
            x0_OU = eqCOMDupireSpotGF.x0_OU;
            expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
            stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            
            r = (x_OU - expectation)/stdDeviation;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            
            r = gridStreching_sigma * r;
            
%             out = eqCOMDupireSpotGF.LagrangeInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
%             yBarNew = log(yBar);
            yBarNew = yBar;
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            if strcmp( eqCOMDupireSpotGF.modelParams('collocationExtrapolation'),'Flat')
%                 out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapFlat(yBarNew,r);
                out1 = pchipFlatExtrap(xBar,yBarNew,r);
            else
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
            end
%             out = exp(out1);
            out = out1;
            
        end
        
        function out = MappingFunctionHeston(eqCOMDupireSpotGF,term,x)
            mappingTable  = eqCOMDupireSpotGF.mappingTable;
            maturityTimes = eqCOMDupireSpotGF.maturityTimes;
            
            yBar = zeros(size(mappingTable,1),1);
            for i=1:length(yBar)
                sBar = mappingTable(i,:);
%                 yBar(i) = interp1(maturityTimes',sBar,term,'linear','extrap');
                yBar(i) = interp1ExtrapLinearHong(maturityTimes',sBar,term);
            end
            
            % from heston x to normal r
            % first find the cdf of x
            outCDF = CDFHestonVector(eqCOMDupireSpotGF,x,term);
            
%             outCDF = zeros(length(x),1);
%             for idxDum = 1:length(x)
%                 outCDF(idxDum) = CDFHeston(eqCOMDupireSpotGF,x(idxDum),term);
%             end
            r = norminv(outCDF);    
            
%             x0_OU = eqCOMDupireSpotGF.x0_OU;
%             expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
%             stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            
%             r = (x_OU - expectation)/stdDeviation;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            
            r = gridStreching_sigma * r;
            
%             out = eqCOMDupireSpotGF.LagrangeInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
%             yBarNew = log(yBar);
            yBarNew = yBar;
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            if strcmp( eqCOMDupireSpotGF.modelParams('collocationExtrapolation'),'Flat')
%                 out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapFlat(yBarNew,r);
                out1 = pchipFlatExtrap(xBar,yBarNew,r);
            else
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
            end
%             out = exp(out1);
            out = out1;
            
        end
        
        function out = MappingFunction_Log(eqCOMDupireSpotGF,term,x_OU)
            mappingTable  = eqCOMDupireSpotGF.mappingTable;
            maturityTimes = eqCOMDupireSpotGF.maturityTimes;
            
            yBar = zeros(size(mappingTable,1),1);
            for i=1:length(yBar)
                sBar = mappingTable(i,:);
%                 yBar(i) = interp1(maturityTimes',sBar,term,'linear','extrap');
                yBar(i) = interp1ExtrapLinearHong(maturityTimes',sBar,term);
            end
            
            x0_OU = eqCOMDupireSpotGF.x0_OU;
            expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
            stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            
            r = (x_OU - expectation)/stdDeviation;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            
            r = gridStreching_sigma * r;
%             out = eqCOMDupireSpotGF.LagrangeInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
%             yBarNew = log(yBar);
            yBarNew = yBar;
            
            if strcmp( eqCOMDupireSpotGF.modelParams('collocationExtrapolation'),'Flat')
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapFlat(yBarNew,r);
            else
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
            end
            out = exp(out1);
            
        end
        
        function out = InitializeMappingTableHeston(eqCOMDupireSpotGF,terms)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            mappingTableHeston = zeros(length(xBar),length(terms));
            for idxDum=1:length(xBar)
%                 dummyCum = H_ncdf(xBar(idxDum));
                dummyCum = H_ncdf(xBar(idxDum)/gridStreching_sigma);
                for idxDum2 = 1:length(terms)
                    mappingTableHeston(idxDum,idxDum2) = eqCOMDupireSpotGF.InvCDFHeston(dummyCum,terms(idxDum2));
                end
            end
            
            eqCOMDupireSpotGF.mappingTableHeston = mappingTableHeston;
            out = mappingTableHeston;
            
        end
        
        function out = InitializeMappingTable2(eqCOMDupireSpotGF,terms)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            mappingTable = zeros(length(xBar),length(terms));
            for idxDum=1:length(xBar)
%                 dummyCum = H_ncdf(xBar(idxDum));
                dummyCum = H_ncdf(xBar(idxDum)/gridStreching_sigma);
                for idxDum2 = 1:length(terms)
                    mappingTable(idxDum,idxDum2) = eqCOMDupireSpotGF.InvCDF2(dummyCum,terms(idxDum2));
                end
            end
            
            eqCOMDupireSpotGF.mappingTable = mappingTable;
            out = mappingTable;
            
        end
        
        function out = InitializeMappingTable(eqCOMDupireSpotGF,terms)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            mappingTable = zeros(length(xBar),length(terms));
            for idxDum=1:length(xBar)
%                 dummyCum = H_ncdf(xBar(idxDum));
                dummyCum = H_ncdf(xBar(idxDum)/gridStreching_sigma);
                for idxDum2 = 1:length(terms)
                    mappingTable(idxDum,idxDum2) = eqCOMDupireSpotGF.InvCDF(dummyCum,terms(idxDum2));
                end
            end
            
            eqCOMDupireSpotGF.mappingTable = mappingTable;
            out = mappingTable;
            
        end
        
        function out = InitializeMappingTableReghai(eqCOMDupireSpotGF,terms)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            mappingTable = zeros(length(xBar),length(terms));
            for idxDum=1:length(xBar)
%                 dummyCum = H_ncdf(xBar(idxDum));
                dummyCum = H_ncdf(xBar(idxDum)/gridStreching_sigma);
                for idxDum2 = 1:length(terms)
                    mappingTable(idxDum,idxDum2) = eqCOMDupireSpotGF.InvCDFReghai(dummyCum,terms(idxDum2));
                end
            end
            
            eqCOMDupireSpotGF.mappingTable = mappingTable;
            out = mappingTable;
            
        end
        
        function out = SABR_ImpVol(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho_SS;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            inrangeYN = 0;
            multiplier = 0.0;
            try
                if Xz < 0.000001 && Xz > -0.000001
                    multiplier = 1.0;
                    inrangeYN = 1;
                else
                    multiplier = z/Xz;
                end
            catch ME
                rethrow(ME)
                
            end
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
            
        end
        
        function out = SABR_ImpVol2(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho_SS;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = SABR_ImpVol_Qlib(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho_SS;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            
            dummyQLEpsilon = 2.22045e-16;
            thresHold = dummyQLEpsilon*10.0;
            dummyz2 = abs(z*z);
            
             inrangeYN = 0;
            if dummyz2 > thresHold
                multiplier = z/Xz;
                 inrangeYN = 1;
            else
                multiplier = 1.0 -0.5*rho*z -(3.0*rho*rho-2.0)*z*z/12.0;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
%             out = model_vol;
%             out.multiplier = multiplier;
        end
        
        function out = CalibrateToVolSurfaceSSSABR(eqCOMDupireSpotGF,black,calibrationFlag)
%             out = 0.0;
            
            % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
          %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
            
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            % calibration Params
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            params.blackVol = blackVol;
           
            params.fwdMoneyness = fwdMoneyness;
            params.marketStrike = params.fwdMoneyness;
            params.calibrationYN = eqCOMDupireSpotGF.useYN;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            params.lb = eqCOMDupireSpotGF.modelParams('lb');
            params.ub = eqCOMDupireSpotGF.modelParams('ub');
            
            % model params
            params.alpha_0 = eqCOMDupireSpotGF.alpha_0;
            params.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            params.rho = eqCOMDupireSpotGF.rho_SS;
            params.nu = eqCOMDupireSpotGF.nu;
            
            params.beta = eqCOMDupireSpotGF.beta;
            params.lambda = eqCOMDupireSpotGF.lambda;
            params.omega =eqCOMDupireSpotGF.omega;
            params.time_shift =eqCOMDupireSpotGF.time_shift;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.modelParams('calibrationErrorType');
            
            %output 
            out.marketStrikes = params.fwdMoneyness;
            out.fwdMoneyness = params.fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.SSSABROTMPrice = zeros(expirySize,strikeSize);
            out.SSSABRBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global4'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'global4'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho_SS;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4RhoSS',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho_SS   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho_SS = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                model_vol = dummyOut.vol;
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                        aaa = 1.0;
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global4bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho_SS, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho_SS;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho_SS   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho_SS = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho_SS, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho_SS;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [1e-8,1e-8,-0.9999,1e-8,1e-8,1e-8];
%                     ub = [100.0,100.0,0.9999,100.0,100.0,100.0];
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');

                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho_SS   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho_SS = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho_SS, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho_SS;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho_SS   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho_SS = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho_SS, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho_SS;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho_SS');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho_SS');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho_SS   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho_SS = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho_SS, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho_SS;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho_SS');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho_SS');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho_SS   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho_SS = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'justCalculateError'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho_SS = eqCOMDupireSpotGF.rho_SS;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho_SS;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                dummyVol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                model_vol = dummyVol.vol;
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                otherwise
                        disp('unimplemented')
            end
            
            eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
            eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
            
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    paramsIn.expiry = params.expiry;
                    paramsIn.Ks = params.Ks;
                    paramsIn.Pricingidx = params.Pricingidx;
                      
                    out.paramsIn = paramsIn;
                    eqCOMDupireSpotGF.paramsIn = paramsIn;
                    
                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSSSABRRawNormalCLV < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% SS SABR parameters
        %dynamic
        alpha_0
        alpha_inf
        rho
        nu
        
        %static
        beta
        lambda
        omega
        time_shift
       
       %% Ornstein-Uhlenbeck
       speed_OU
       level_OU
       vol_OU
       x0_OU
       
       lagrangeOrder
       HermiteNormalXBar
       gridStreching_sigma
       
       maturityTimes
       mappingTable
       
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        settleFuturesRef
        atmVols
        
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSSSABRRawNormalCLV(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                eqCOMDupireSpotGF.alpha_0 = EQModel.modelParams('alpha_0');
                eqCOMDupireSpotGF.alpha_inf = EQModel.modelParams('alpha_inf');
                eqCOMDupireSpotGF.rho = EQModel.modelParams('rho');
                eqCOMDupireSpotGF.nu = EQModel.modelParams('nu');
                
                
                eqCOMDupireSpotGF.beta = EQModel.modelParams('beta');
                eqCOMDupireSpotGF.lambda = EQModel.modelParams('lambda');
                eqCOMDupireSpotGF.omega = EQModel.modelParams('omega');
                eqCOMDupireSpotGF.time_shift = EQModel.modelParams('time_shift');
                
              %% Ornstein Uhlenbeck
                eqCOMDupireSpotGF.speed_OU = EQModel.modelParams('speed_OU');
                eqCOMDupireSpotGF.level_OU = EQModel.modelParams('level_OU');
                eqCOMDupireSpotGF.vol_OU = EQModel.modelParams('vol_OU');
                eqCOMDupireSpotGF.x0_OU = EQModel.modelParams('x0_OU');
                
                eqCOMDupireSpotGF.lagrangeOrder = EQModel.modelParams('lagrangeOrder');
                eqCOMDupireSpotGF.HermiteNormalXBar = EQModel.modelParams('HermiteNormalXBar');
                if length(eqCOMDupireSpotGF.HermiteNormalXBar) ~= eqCOMDupireSpotGF.lagrangeOrder
                    err('size of HermiteNormalXBar should match lagrangeOrder !')
                end
                
                if ~isKey(EQModel.modelParams,'gridStreching_sigma')
                    eqCOMDupireSpotGF.gridStreching_sigma = 1.0;
                else
                    eqCOMDupireSpotGF.gridStreching_sigma = EQModel.modelParams('gridStreching_sigma');
                end
                
                eqCOMDupireSpotGF.maturityTimes = EQModel.modelParams('maturityTimes');
                eqCOMDupireSpotGF.mappingTable = [];
              %% SS SABR model params start
                
              %% SS SABR model params end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                if ~isKey(EQModel.modelParams,'settleFutures')
                    eqCOMDupireSpotGF.settleFutures= [];
                else
                    eqCOMDupireSpotGF.settleFutures= EQModel.modelParams('settleFutures');
                end
                
                if ~isKey(EQModel.modelParams,'settleFuturesRef')
                    eqCOMDupireSpotGF.settleFuturesRef= [];
                else
                    eqCOMDupireSpotGF.settleFuturesRef = EQModel.modelParams('settleFuturesRef');
                end
                
                if ~isKey(EQModel.modelParams,'atmVols')
                    eqCOMDupireSpotGF.atmVols= [];
                else
                    eqCOMDupireSpotGF.atmVols = EQModel.modelParams('atmVols');
                end
                
                if isKey(EQModel.modelParams,'localVolSurface')
                    eqCOMDupireSpotGF.localVolSurface = EQModel.modelParams('localVolSurface');
                end
                
                if ~isKey(EQModel.modelParams,'calendarSpreadDt')
                    eqCOMDupireSpotGF.modelParams('calendarSpreadDt')= 0.5*0.000001/365.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = initFuturesStates(eqCOMDupireSpotGF,NMC)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            out =  spotPrice*ones(NMC,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            out = nextFuturesStates;
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end

        function localVol = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry;
            
            %find which strip fromTime belongs to 
            idxS = find(expiry > fromTime);
            if isempty(idxS)
                idxStrip = length(expiry);
            else
                idxStrip = min(idxS);
            end
            
            % find exact timeSlice in the given strip
            localVolTimeStep = eqCOMDupireSpotGF.localVolSurface.localVolTimeStep{idxS};
            localVolSurfaceSABR = eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR{idxS};
            forwardMoneyness = eqCOMDupireSpotGF.localVolSurface.forwardMoneyness{idxS};

            idx = find(localVolTimeStep > fromTime);
            if isempty(idx)
                idxNow = length(localVolTimeStep);
            else
                idxNow = min(idx);
            end
            
            localVolLine = localVolSurfaceSABR(idxNow,:);
%             forwardMoneynessLine = forwardMoneyness(idxNow,:);
            
            localVol =  zeros(length(x),1);
            
            dummyOut = interp1FlatExtrapNew(forwardMoneyness,localVolLine,x','linear');
            dummyOut2 = sqrt(dummyOut);
            localVol = dummyOut2';
            
%             for ii =1:length(x)
%                 dummyVariance = H_interpolation(forwardMoneyness,localVolLine,x(ii),1);
%                 localVol(ii) = sqrt(dummyVariance);
%             end

        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
         function mcmcOut = inductMCForwardNewReducedQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
         end
        
         function mcmcOut = inductMCForwardNewReducedNMonthsQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut1 = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            mcmcOut.nextFuturesStates = mcmcOut1.nextX;
            mcmcOut.idxNM = 1;
        end
        
        function mcmcOut = inductMCForwardNewCLV(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;

            Dt = (toTime - fromTime)/365.0;
            drift = eqCOMDupireSpotGF.OU_Expectation(fromTime,predictor,Dt);
            stdDev = eqCOMDupireSpotGF.OU_StdDeviation(fromTime,predictor,Dt);
            
            predictor = drift + stdDev*dZ;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(spotRawData*predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end

        function mcmcOut = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMCCLV(eqCOMDupireSpotGF,observeTime,modelStates)
         if observeTime == 0
             out = modelStates;
         else
             out = eqCOMDupireSpotGF.MappingFunction(observeTime/365.0,modelStates);
             
         end
         
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)

         out = modelStates;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCReducedNMonths(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = computeLocalVolGridSSSABR(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
                    f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                    
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
%                     
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib_FwdMN(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                     if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                     end
                    
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;

                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup*detFwd_f(idxjj)/detFwd(idxjj),f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);

                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeCalendarSpreadOptionMCReducedInt(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = GenerateLocalVolSS(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSSReduced(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end

                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib_FwdMN(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = computeForwardStartingVanillaMCCLV(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % resetDate
            % expiryDate
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
%             
% %             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            timeStep = sort(union(timeStep,maturity),'ascend');
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            initX = zeros(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(NMC,1);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== 1
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewCLV(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                % at startDate, we reset strike
                if i==1
                    comFwd = eqCOMDupireSpotGF.EQFwdMCCLV(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMCCLV(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeForwardStartingVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % resetDate
            % expiryDate
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,daysToReset),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqCOMDupireSpotGF.FwdFactor(daysToReset);
           detFwdMaturity = eqCOMDupireSpotGF.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(NMC,1);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== 1
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABRReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                % at startDate, we reset strike
                if i==1
                    comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
                    reSetS = comFwd;
                    
                end
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           df_ep = 1.0;
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeStepdown1SMCReducedInt(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReducedIntEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = CDF(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = input_k;
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k,term);
            blackVol = dummy.vol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
%                 alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
%                 alpha = -1.0 + H_ncdf(d1);
%                 alpha = -H_ncdf(-d1);
                DalphaDd1 = H_ndf(d1);
%                 beta = 1.0 - H_ncdf(d2);
                beta = H_ncdf(-d2);
                
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        
        function out = CDF_Log(eqCOMDupireSpotGF,input_k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            k = exp(input_k);
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k,term);
            blackVol = dummy.vol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
%                 alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
%                 alpha = -1.0 + H_ncdf(d1);
%                 alpha = -H_ncdf(-d1);
                DalphaDd1 = H_ndf(d1);
%                 beta = 1.0 - H_ncdf(d2);
                beta = H_ncdf(-d2);
                
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        
        function out =  InvCDF(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
%             guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess/lower < 1e12 && CDF(eqCOMDupireSpotGF,lower,term) > y
                lower = lower/2.0;
            end
            
            upper = guess;
            while upper/guess < 1e12 && CDF(eqCOMDupireSpotGF,upper,term) < y
                upper = upper*2.0;
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDF(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out =  InvCDF_Log(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            %change Knot Points from S(T) space to log S(T) 
            guess = log(fwd*exp(blackVol * sqrt(term) *atmX));
            lower = guess;
            while guess - lower < 10000 && CDF(eqCOMDupireSpotGF,lower,term) > y
                lower = lower  - log(2.0);
            end
            
            upper = guess;
            while upper - guess < 10000 && CDF(eqCOMDupireSpotGF,upper,term) < y
                upper = upper + log(2.0);
            end
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);

            try
                targetF = @(x) CDF(eqCOMDupireSpotGF,x,term) - y;
                [B,fval,exitflag,outputCalib] = fzero(targetF,[lower upper]);
            catch
                aaa = 1.0;
                rethrow(ME)
            end
            
            if exitflag ~= 1
                aaa = 1.0;
                erro('failed to find the solution!')
            end
            out= B;
            
        end
        
        function out = CollocationPointsY(eqCOMDupireSpotGF,term)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            out = zeros(length(xBar),1);
            for idxDum=1:length(xBar)
                out(idxDum) = eqCOMDupireSpotGF.InvCDF(H_ncdf(xBar(idxDum)),term);
            end
        end
        
        function out = OU_Expectation(eqCOMDupireSpotGF,t,x0,dt)
            level_OU = eqCOMDupireSpotGF.level_OU;
            speed_OU = eqCOMDupireSpotGF.speed_OU;
            
            out = level_OU + (x0 - level_OU)*exp(-speed_OU*dt);
        end
        
        function out = OU_StdDeviation(eqCOMDupireSpotGF,t,x0,dt)
            vol_OU   = eqCOMDupireSpotGF.vol_OU;
            speed_OU = eqCOMDupireSpotGF.speed_OU;
            
            variance = 0.5*vol_OU*vol_OU/speed_OU*(1-exp(-2.0*speed_OU*dt));
            out = sqrt(variance);
        end
        
        function out = CollocationPointsX(eqCOMDupireSpotGF,term)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            x0_OU = eqCOMDupireSpotGF.x0_OU;
            expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
            stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            out = expectation + stdDeviation*xBar;
            
        end
        
        function out  = PiecewiseCubicHermiteInterpExtrapLinear(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            % Left & Right Linear Extrapolation !!
            if r < xBar(1)
                pp = pchip(xBar,yBar);
                lhPol = pp.coefs(1,:);
                out = (r-xBar(1))*lhPol(3) + yBar(1);
            elseif r > xBar(end)
                pp = pchip(xBar,yBar);
                rhPol = pp.coefs(end,:);
                rhPolD = [rhPol(1)*3,rhPol(2)*2,rhPol(3)];
                slopeR = polyval(rhPolD,xBar(end)-xBar(end-1));
                out = (r- xBar(end))*slopeR + yBar(end);
            else
                out = pchip(xBar,yBar,r);
            end
        end
        
        function out  = PiecewiseCubicHermiteInterpExtrapFlat(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            % Left & Right Linear Extrapolation !!
            if r < xBar(1)
                out = yBar(1);
            elseif r > xBar(end)
                out = yBar(end);
            else
                out = pchip(xBar,yBar,r);
            end
        end
        
        function out  = PiecewiseCubicHermiteInterp(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            out = pchip(xBar,yBar,r);
            
        end
        
        function out  = LagrangeInterp(eqCOMDupireSpotGF,yBar,r)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            
            if length(yBar) ~= length(xBar)
                error('size of yBar should be equal to the size of xBar')
            end
            
            value = 0.0;
            for i=1:length(yBar)
                % if r is close to the knot points then no need interpol
                if r == xBar(i)
                    value = yBar(i);
                    break;
                end
                
                weight_i = 1.0;
                for j=1:length(xBar)
                    if j ~= i
                    weight_i = weight_i * (r-xBar(j))/(xBar(i) - xBar(j));
                    end
                end
                value = value + weight_i * yBar(i);
            end
            out = value;
        end
        
        function out = MappingFunction(eqCOMDupireSpotGF,term,x_OU)
            mappingTable  = eqCOMDupireSpotGF.mappingTable;
            maturityTimes = eqCOMDupireSpotGF.maturityTimes;
            
            yBar = zeros(size(mappingTable,1),1);
            for i=1:length(yBar)
                sBar = mappingTable(i,:);
%                 yBar(i) = interp1(maturityTimes',sBar,term,'linear','extrap');
                yBar(i) = interp1ExtrapLinearHong(maturityTimes',sBar,term);
            end
            
            x0_OU = eqCOMDupireSpotGF.x0_OU;
            expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
            stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            
            r = (x_OU - expectation)/stdDeviation;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            
            r = gridStreching_sigma * r;
            
%             out = eqCOMDupireSpotGF.LagrangeInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
%             yBarNew = log(yBar);
            yBarNew = yBar;
            
            if strcmp( eqCOMDupireSpotGF.modelParams('collocationExtrapolation'),'Flat')
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapFlat(yBarNew,r);
            else
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
            end
%             out = exp(out1);
            out = out1;
            
        end
        
        function out = MappingFunction_Log(eqCOMDupireSpotGF,term,x_OU)
            mappingTable  = eqCOMDupireSpotGF.mappingTable;
            maturityTimes = eqCOMDupireSpotGF.maturityTimes;
            
            yBar = zeros(size(mappingTable,1),1);
            for i=1:length(yBar)
                sBar = mappingTable(i,:);
%                 yBar(i) = interp1(maturityTimes',sBar,term,'linear','extrap');
                yBar(i) = interp1ExtrapLinearHong(maturityTimes',sBar,term);
            end
            
            x0_OU = eqCOMDupireSpotGF.x0_OU;
            expectation  = eqCOMDupireSpotGF.OU_Expectation(0.0,x0_OU,term);
            stdDeviation = eqCOMDupireSpotGF.OU_StdDeviation(0.0,x0_OU,term);
            
            r = (x_OU - expectation)/stdDeviation;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            
            r = gridStreching_sigma * r;
%             out = eqCOMDupireSpotGF.LagrangeInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterp(yBar,r);
%             out = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
%             yBarNew = log(yBar);
            yBarNew = yBar;
            
            if strcmp( eqCOMDupireSpotGF.modelParams('collocationExtrapolation'),'Flat')
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapFlat(yBarNew,r);
            else
                out1 = eqCOMDupireSpotGF.PiecewiseCubicHermiteInterpExtrapLinear(yBarNew,r);
            end
            out = exp(out1);
            
        end
        
        function out = InitializeMappingTable(eqCOMDupireSpotGF,terms)
            xBar = eqCOMDupireSpotGF.HermiteNormalXBar;
            gridStreching_sigma = eqCOMDupireSpotGF.gridStreching_sigma;
            mappingTable = zeros(length(xBar),length(terms));
            for idxDum=1:length(xBar)
%                 dummyCum = H_ncdf(xBar(idxDum));
                dummyCum = H_ncdf(xBar(idxDum)/gridStreching_sigma);
                for idxDum2 = 1:length(terms)
                    mappingTable(idxDum,idxDum2) = eqCOMDupireSpotGF.InvCDF(dummyCum,terms(idxDum2));
                end
            end
            
            eqCOMDupireSpotGF.mappingTable = mappingTable;
            out = mappingTable;
            
        end
        
        function out = SABR_ImpVol(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            inrangeYN = 0;
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
                inrangeYN = 1;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
            
        end
        
        function out = SABR_ImpVol2(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = SABR_ImpVol_Qlib(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            
            dummyQLEpsilon = 2.22045e-16;
            thresHold = dummyQLEpsilon*10.0;
            dummyz2 = abs(z*z);
            
             inrangeYN = 0;
            if dummyz2 > thresHold
                multiplier = z/Xz;
                 inrangeYN = 1;
            else
                multiplier = 1.0 -0.5*rho*z -(3.0*rho*rho-2.0)*z*z/12.0;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
%             out = model_vol;
%             out.multiplier = multiplier;
        end
        
        function out = CalibrateToVolSurfaceSSSABR(eqCOMDupireSpotGF,black,calibrationFlag)
%             out = 0.0;
            
            % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
          %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
            
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            % calibration Params
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            params.blackVol = blackVol;
           
            params.fwdMoneyness = fwdMoneyness;
            params.marketStrike = params.fwdMoneyness;
            params.calibrationYN = eqCOMDupireSpotGF.useYN;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            params.lb = eqCOMDupireSpotGF.modelParams('lb');
            params.ub = eqCOMDupireSpotGF.modelParams('ub');
            
            % model params
            params.alpha_0 = eqCOMDupireSpotGF.alpha_0;
            params.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            params.rho = eqCOMDupireSpotGF.rho;
            params.nu = eqCOMDupireSpotGF.nu;
            
            params.beta = eqCOMDupireSpotGF.beta;
            params.lambda = eqCOMDupireSpotGF.lambda;
            params.omega =eqCOMDupireSpotGF.omega;
            params.time_shift =eqCOMDupireSpotGF.time_shift;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.modelParams('calibrationErrorType');
            
            %output 
            out.marketStrikes = params.fwdMoneyness;
            out.fwdMoneyness = params.fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.SSSABROTMPrice = zeros(expirySize,strikeSize);
            out.SSSABRBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global4'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'global4'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global4bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [1e-8,1e-8,-0.9999,1e-8,1e-8,1e-8];
%                     ub = [100.0,100.0,0.9999,100.0,100.0,100.0];
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');

                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'justCalculateError'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                otherwise
                        disp('unimplemented')
            end
            
            eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
            eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
            
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSSSABRRawNewLV < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% SS SABR parameters
        %dynamic
        alpha_0
        alpha_inf
        rho
        nu
        
        %static
        beta
        lambda
        omega
        time_shift
       
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        settleFuturesRef
        atmVols
        
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSSSABRRawNewLV(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                eqCOMDupireSpotGF.alpha_0 = EQModel.modelParams('alpha_0');
                eqCOMDupireSpotGF.alpha_inf = EQModel.modelParams('alpha_inf');
                eqCOMDupireSpotGF.rho = EQModel.modelParams('rho');
                eqCOMDupireSpotGF.nu = EQModel.modelParams('nu');
                
                
                eqCOMDupireSpotGF.beta = EQModel.modelParams('beta');
                eqCOMDupireSpotGF.lambda = EQModel.modelParams('lambda');
                eqCOMDupireSpotGF.omega = EQModel.modelParams('omega');
                eqCOMDupireSpotGF.time_shift = EQModel.modelParams('time_shift');
                
              %% SS SABR model params start
                
              %% SS SABR model params end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                if ~isKey(EQModel.modelParams,'settleFutures')
                    eqCOMDupireSpotGF.settleFutures= [];
                else
                    eqCOMDupireSpotGF.settleFutures= EQModel.modelParams('settleFutures');
                end
                
                if ~isKey(EQModel.modelParams,'settleFuturesRef')
                    eqCOMDupireSpotGF.settleFuturesRef= [];
                else
                    eqCOMDupireSpotGF.settleFuturesRef = EQModel.modelParams('settleFuturesRef');
                end
                
                if ~isKey(EQModel.modelParams,'atmVols')
                    eqCOMDupireSpotGF.atmVols= [];
                else
                    eqCOMDupireSpotGF.atmVols = EQModel.modelParams('atmVols');
                end
                
                if isKey(EQModel.modelParams,'localVolSurface')
                    eqCOMDupireSpotGF.localVolSurface = EQModel.modelParams('localVolSurface');
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = initFuturesStates(eqCOMDupireSpotGF,NMC)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            out =  spotPrice*ones(NMC,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            out = nextFuturesStates;
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end

        function localVol = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry;
            
            %find which strip fromTime belongs to 
            idxS = find(expiry > fromTime);
            if isempty(idxS)
                idxStrip = length(expiry);
            else
                idxStrip = min(idxS);
            end
            
            % find exact timeSlice in the given strip
            localVolTimeStep = eqCOMDupireSpotGF.localVolSurface.localVolTimeStep{idxS};
            localVolSurfaceSABR = eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR{idxS};
            forwardMoneyness = eqCOMDupireSpotGF.localVolSurface.forwardMoneyness{idxS};

            idx = find(localVolTimeStep > fromTime);
            if isempty(idx)
                idxNow = length(localVolTimeStep);
            else
                idxNow = min(idx);
            end
            
            localVolLine = localVolSurfaceSABR(idxNow,:);
%             forwardMoneynessLine = forwardMoneyness(idxNow,:);
            
            localVol =  zeros(length(x),1);
            
            dummyOut = interp1FlatExtrapNew(forwardMoneyness,localVolLine,x','linear');
            dummyOut2 = sqrt(dummyOut);
            localVol = dummyOut2';
            
%             for ii =1:length(x)
%                 dummyVariance = H_interpolation(forwardMoneyness,localVolLine,x(ii),1);
%                 localVol(ii) = sqrt(dummyVariance);
%             end

        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
         function mcmcOut = inductMCForwardNewReducedQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut1 = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            mcmcOut.nextFuturesStates = mcmcOut1.nextX;
            mcmcOut.idxNM = 1;
        end

        function mcmcOut = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
%         spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
%         detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
% 
%          %stochasticFwd(0,T)
%          stateSize = length(modelStates);
%          stochasticFwd = zeros(stateSize,1);
% 
%          for i=1:stateSize
%             stochasticFwd(i) = detFwd * modelStates(i); 
%          end

%          out = stochasticFwd;
         
         out = modelStates;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
%             % piecewise constant interpol using reducedFutures
%             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
%              
%             forwardCurveSize = size(reducedFutures,1);
%             forwardMaturity = reducedFutures(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              fwd_NM = reducedFutures(idxNM,2);
%              
%              
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = computeLocalVolGridSSSABR(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
            dt = 0.5*0.000001/365.0;
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup*detFwd_f(idxjj)/detFwd(idxjj),f_matT);
                    b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeCalendarSpreadOptionMCReducedInt(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = GenerateLocalVolSS(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        function out = computeStepdown1SMCReducedInt(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReducedIntEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        
        function out = SABR_ImpVol(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out = model_vol;
        end
        
        function out = SABR_ImpVol2(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = SABR_ImpVol_Qlib(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            
            dummyQLEpsilon = 2.22045e-16;
            thresHold = dummyQLEpsilon*10.0;
            dummyz2 = abs(z*z);
            if dummyz2 > thresHold
                multiplier = z/Xz;
            else
                multiplier = 1.0 -0.5*rho*z -(3.0*rho*rho-2.0)*z*z/12.0;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = CalibrateToVolSurfaceSSSABR(eqCOMDupireSpotGF,black,calibrationFlag)
%             out = 0.0;
            
            % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
          %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
            
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            % calibration Params
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            params.blackVol = blackVol;
           
            params.fwdMoneyness = fwdMoneyness;
            params.marketStrike = params.fwdMoneyness;
            params.calibrationYN = eqCOMDupireSpotGF.useYN;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            params.lb = eqCOMDupireSpotGF.modelParams('lb');
            params.ub = eqCOMDupireSpotGF.modelParams('ub');
            
            % model params
            params.alpha_0 = eqCOMDupireSpotGF.alpha_0;
            params.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            params.rho = eqCOMDupireSpotGF.rho;
            params.nu = eqCOMDupireSpotGF.nu;
            
            params.beta = eqCOMDupireSpotGF.beta;
            params.lambda = eqCOMDupireSpotGF.lambda;
            params.omega =eqCOMDupireSpotGF.omega;
            params.time_shift =eqCOMDupireSpotGF.time_shift;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.modelParams('calibrationErrorType');
            
            %output 
            out.marketStrikes = params.fwdMoneyness;
            out.fwdMoneyness = params.fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.SSSABROTMPrice = zeros(expirySize,strikeSize);
            out.SSSABRBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global4'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'global4'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global4bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [1e-8,1e-8,-0.9999,1e-8,1e-8,1e-8];
%                     ub = [100.0,100.0,0.9999,100.0,100.0,100.0];
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');

                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'justCalculateError'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                otherwise
                        disp('unimplemented')
            end
            
            eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
            eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
            
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSSSABRRaw < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% SS SABR parameters
        %dynamic
        alpha_0
        alpha_inf
        rho
        nu
        
        %static
        beta
        lambda
        omega
        time_shift
       
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        settleFuturesRef
        atmVols
        
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSSSABRRaw(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                eqCOMDupireSpotGF.alpha_0 = EQModel.modelParams('alpha_0');
                eqCOMDupireSpotGF.alpha_inf = EQModel.modelParams('alpha_inf');
                eqCOMDupireSpotGF.rho = EQModel.modelParams('rho');
                eqCOMDupireSpotGF.nu = EQModel.modelParams('nu');
                
                
                eqCOMDupireSpotGF.beta = EQModel.modelParams('beta');
                eqCOMDupireSpotGF.lambda = EQModel.modelParams('lambda');
                eqCOMDupireSpotGF.omega = EQModel.modelParams('omega');
                eqCOMDupireSpotGF.time_shift = EQModel.modelParams('time_shift');
                
              %% SS SABR model params start
                
              %% SS SABR model params end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                if ~isKey(EQModel.modelParams,'settleFutures')
                    eqCOMDupireSpotGF.settleFutures= [];
                else
                    eqCOMDupireSpotGF.settleFutures= EQModel.modelParams('settleFutures');
                end
                
                if ~isKey(EQModel.modelParams,'settleFuturesRef')
                    eqCOMDupireSpotGF.settleFuturesRef= [];
                else
                    eqCOMDupireSpotGF.settleFuturesRef = EQModel.modelParams('settleFuturesRef');
                end
                
                if ~isKey(EQModel.modelParams,'atmVols')
                    eqCOMDupireSpotGF.atmVols= [];
                else
                    eqCOMDupireSpotGF.atmVols = EQModel.modelParams('atmVols');
                end
                
                if isKey(EQModel.modelParams,'localVolSurface')
                    eqCOMDupireSpotGF.localVolSurface = EQModel.modelParams('localVolSurface');
                end
                
                if ~isKey(EQModel.modelParams,'calendarSpreadDt')
                    eqCOMDupireSpotGF.modelParams('calendarSpreadDt')= 0.5*0.000001/365.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = initFuturesStates(eqCOMDupireSpotGF,NMC)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            out =  spotPrice*ones(NMC,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            out = nextFuturesStates;
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end

        function localVol = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry;
            
            %find which strip fromTime belongs to 
            idxS = find(expiry > fromTime);
            if isempty(idxS)
                idxStrip = length(expiry);
            else
                idxStrip = min(idxS);
            end
            
            % find exact timeSlice in the given strip
            localVolTimeStep = eqCOMDupireSpotGF.localVolSurface.localVolTimeStep{idxS};
            localVolSurfaceSABR = eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR{idxS};
            forwardMoneyness = eqCOMDupireSpotGF.localVolSurface.forwardMoneyness{idxS};

            idx = find(localVolTimeStep > fromTime);
            if isempty(idx)
                idxNow = length(localVolTimeStep);
            else
                idxNow = min(idx);
            end
            
            localVolLine = localVolSurfaceSABR(idxNow,:);
%             forwardMoneynessLine = forwardMoneyness(idxNow,:);
            
            localVol =  zeros(length(x),1);
            
            dummyOut = interp1FlatExtrapNew(forwardMoneyness,localVolLine,x','linear');
            dummyOut2 = sqrt(dummyOut);
            localVol = dummyOut2';
            
%             for ii =1:length(x)
%                 dummyVariance = H_interpolation(forwardMoneyness,localVolLine,x(ii),1);
%                 localVol(ii) = sqrt(dummyVariance);
%             end

        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
         function mcmcOut = inductMCForwardNewReducedQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
         end
        
         function mcmcOut = inductMCForwardNewReducedNMonthsQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut1 = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            mcmcOut.nextFuturesStates = mcmcOut1.nextX;
            mcmcOut.idxNM = 1;
        end

        function mcmcOut = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
%         spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
%         detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
% 
%          %stochasticFwd(0,T)
%          stateSize = length(modelStates);
%          stochasticFwd = zeros(stateSize,1);
% 
%          for i=1:stateSize
%             stochasticFwd(i) = detFwd * modelStates(i); 
%          end

%          out = stochasticFwd;
         
         out = modelStates;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCReducedNMonths(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        
        function out = computeLocalVolGridSSSABR_KYJ_FwdFactor(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
%                 detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
%                 detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
%                 detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
%                 detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                detFwd(idxjj)   = S_ghost     * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactorOrg(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
%                 detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
%                 detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
%                 detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
%                 detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                detFwd(idxjj)   = S_ghost     * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactor(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
%                 detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
%                 detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
%                 detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
%                 detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                detFwd(idxjj)   = S_ghost     * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end

        function out = computeLocalVolGridSSSABR_KYJ_FwdMN(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end

        function out = computeLocalVolGridSSSABR_KYJ(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end

        function out = computeLocalVolGridSSSABR(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
                    f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                    
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
%                     
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib_FwdMN(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                     if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                     end
                    
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;

                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup*detFwd_f(idxjj)/detFwd(idxjj),f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);

                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeCalendarSpreadOptionMCReducedInt(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end

        function out = GenerateLocalVolSS(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_KYJ_FwdFactor(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactor(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdFactor(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactorOrg(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactor(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
function out = GenerateLocalVolSS_KYJ_FwdMN(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end

        function out = GenerateLocalVolSS_KYJ(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                

%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(1)/settleFuturesRef(1);
                
                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end

                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib_FwdMN(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = computeStepdown1SMCReducedInt(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReducedIntEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        function out = CDF(eqCOMDupireSpotGF,k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k,term);
            blackVol = dummy.vol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
                alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
                alpha = -1.0 + H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = 1.0 - H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        function out =  InvCDF(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
            targetF = @(x) CDF(eqCOMDupireSpotGF,x,term) - y;
            
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);
            [B,fval,exitflag,outputCalib] = fzero(targetF,guess);
            
            out= B;
            
        end
        
        function out = SABR_ImpVol(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            inrangeYN = 0;
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
                inrangeYN = 1;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
            
        end
        
        function out = SABR_ImpVol2(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = SABR_ImpVol_Qlib(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            
            dummyQLEpsilon = 2.22045e-16;
            thresHold = dummyQLEpsilon*10.0;
            dummyz2 = abs(z*z);
            
             inrangeYN = 0;
            if dummyz2 > thresHold
                multiplier = z/Xz;
                 inrangeYN = 1;
            else
                multiplier = 1.0 -0.5*rho*z -(3.0*rho*rho-2.0)*z*z/12.0;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
%             out = model_vol;
%             out.multiplier = multiplier;
        end
        
        function out = CalibrateToVolSurfaceSSSABR(eqCOMDupireSpotGF,black,calibrationFlag)
%             out = 0.0;
            
            % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
          %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
            
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            % calibration Params
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            params.blackVol = blackVol;
           
            params.fwdMoneyness = fwdMoneyness;
            params.marketStrike = params.fwdMoneyness;
            params.calibrationYN = eqCOMDupireSpotGF.useYN;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            params.lb = eqCOMDupireSpotGF.modelParams('lb');
            params.ub = eqCOMDupireSpotGF.modelParams('ub');
            
            % model params
            params.alpha_0 = eqCOMDupireSpotGF.alpha_0;
            params.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            params.rho = eqCOMDupireSpotGF.rho;
            params.nu = eqCOMDupireSpotGF.nu;
            
            params.beta = eqCOMDupireSpotGF.beta;
            params.lambda = eqCOMDupireSpotGF.lambda;
            params.omega =eqCOMDupireSpotGF.omega;
            params.time_shift =eqCOMDupireSpotGF.time_shift;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.modelParams('calibrationErrorType');
            
            %output 
            out.marketStrikes = params.fwdMoneyness;
            out.fwdMoneyness = params.fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.SSSABROTMPrice = zeros(expirySize,strikeSize);
            out.SSSABRBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            
            % computeSmileReferenceRefelctedVol
            
            out.SRRVol = zeros(expirySize,strikeSize);
            out.SRRStrike = zeros(expirySize,strikeSize);
            
            out.SRRVolRef = zeros(expirySize,strikeSize);
            out.SRRStrikeRef = zeros(expirySize,strikeSize);
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global4'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'global4'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                outDummy = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                model_vol = outDummy.vol;
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global4bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [1e-8,1e-8,-0.9999,1e-8,1e-8,1e-8];
%                     ub = [100.0,100.0,0.9999,100.0,100.0,100.0];
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');

                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'justCalculateError'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                outDummyVol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                model_vol = outDummyVol.vol;
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                    
                case 'computeSlopeReferenceReflectedVol'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    sRef = eqCOMDupireSpotGF.modelParams('reference');
                    slope = eqCOMDupireSpotGF.modelParams('slope');
                    slopeAdjustFactor = 1.0/(1.0 + slope*(spot/sRef - 1.0));
                    
                    for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           % slope adjusted vol
                           fwdAdj = fwd * slopeAdjustFactor;
                        for j=1:params.strikeSize
                            market_vol = params.blackVol(i,j);
                            market_otmPrice = params.blackOTMPrices(i,j);
                            marketStrike = params.marketStrike(i,j);
                            out.SRRStrike(i,j)  = spot * strike(j);
                            
                            outDummyVol = eqCOMDupireSpotGF.SABR_ImpVol(fwdAdj, out.SRRStrike(i,j), expiry(i)/365.0);
                            model_vol = outDummyVol.vol;
                            out.SRRVol(i,j) =  model_vol;
                            
                            out.SRRStrikeRef(i,j)  = sRef * strike(j);
                            
                            outDummyVol2 = eqCOMDupireSpotGF.SABR_ImpVol(fwdAdj, out.SRRStrikeRef(i,j), expiry(i)/365.0);
                            model_vol2 = outDummyVol2.vol;
                            out.SRRVolRef(i,j) = model_vol2;
                            
                            
                        end
                    end

%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             if params.calibrationYN(j,k) == 1
%                                 out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
%                                 out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
%                                 N=N+1;
%                             end
%                         end
%                     end
% 
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                otherwise
                        disp('unimplemented')
            end
            
            eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
            eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
            
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
classdef EQCOMDupireSSSABRRaw < EQModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        localVolSurface
        modelSchedule
        fwdMoneyness
        numOfFactors
        
        slope
        
       %% SS SABR parameters
        %dynamic
        alpha_0
        alpha_inf
        rho
        nu
        
        %static
        beta
        lambda
        omega
        time_shift
       
        
       %% pricingVolGrid data
        pricingForwardGrid
        pricingReducedForwardGrid
        pricingVolGrid
        pricingVolGridSize
        volNodeFwdCurve
        modelStatesFixedGrid
        basePrice

        targetStrikes
        
        %fixedStrike    
        settleFutures
        settleFuturesRef
        atmVols
        
        deltaH
        useYN
        deltaCutOff
        
       %reducedFutures
        reducedFutures
    end
    
    methods
        function eqCOMDupireSpotGF = EQCOMDupireSSSABRRaw(EQModel)
            if nargin > 0
                eqCOMDupireSpotGF.zeroCurve = EQModel.zeroCurve;
                eqCOMDupireSpotGF.dividendCurve = EQModel.dividendCurve;
                eqCOMDupireSpotGF.repoRateCurve = EQModel.repoRateCurve;
                eqCOMDupireSpotGF.forwardCurve = EQModel.forwardCurve;
                
                %% SSR
                eqCOMDupireSpotGF.referenceForwardCurve = EQModel.referenceForwardCurve;
                
                eqCOMDupireSpotGF.spot = EQModel.spot;
                eqCOMDupireSpotGF.mktData =  EQModel.mktData;
                eqCOMDupireSpotGF.modelParams = EQModel.modelParams;
                
              %% SSR modelparams start
                
                if isKey(EQModel.modelParams,'slope')
                    eqCOMDupireSpotGF.slope = EQModel.modelParams('slope');
                else
                    eqCOMDupireSpotGF.slope = 0.0;
                end
                
              %% SSR modelparams end
                eqCOMDupireSpotGF.alpha_0 = EQModel.modelParams('alpha_0');
                eqCOMDupireSpotGF.alpha_inf = EQModel.modelParams('alpha_inf');
                eqCOMDupireSpotGF.rho = EQModel.modelParams('rho');
                eqCOMDupireSpotGF.nu = EQModel.modelParams('nu');
                
                
                eqCOMDupireSpotGF.beta = EQModel.modelParams('beta');
                eqCOMDupireSpotGF.lambda = EQModel.modelParams('lambda');
                eqCOMDupireSpotGF.omega = EQModel.modelParams('omega');
                eqCOMDupireSpotGF.time_shift = EQModel.modelParams('time_shift');
                
              %% SS SABR model params start
                
              %% SS SABR model params end
                
                if ~isKey(EQModel.modelParams,'LocalVolInterpol')
                    eqCOMDupireSpotGF.modelParams('LocalVolInterpol')='matlabLinearMarketFwdMoneyness';
                end
                
                if ~isKey(EQModel.modelParams,'VolProxyOnMarketStrikeYN')
                    eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN')='No';
                end
                
                if ~isKey(EQModel.modelParams,'UseLetsBeRational')
                   eqCOMDupireSpotGF.modelParams('UseLetsBeRational')='YES';
                end
                
                if ~isKey(EQModel.modelParams,'localVolFloor')
                   eqCOMDupireSpotGF.modelParams('localVolFloor')= 0.0001;
                end
                
                if ~isKey(EQModel.modelParams,'localVolCap')
                   eqCOMDupireSpotGF.modelParams('localVolCap')= 5.0;
                end
                
                if ~isKey(EQModel.modelParams,'settleFutures')
                    eqCOMDupireSpotGF.settleFutures= [];
                else
                    eqCOMDupireSpotGF.settleFutures= EQModel.modelParams('settleFutures');
                end
                
                if ~isKey(EQModel.modelParams,'settleFuturesRef')
                    eqCOMDupireSpotGF.settleFuturesRef= [];
                else
                    eqCOMDupireSpotGF.settleFuturesRef = EQModel.modelParams('settleFuturesRef');
                end
                
                if ~isKey(EQModel.modelParams,'atmVols')
                    eqCOMDupireSpotGF.atmVols= [];
                else
                    eqCOMDupireSpotGF.atmVols = EQModel.modelParams('atmVols');
                end
                
                if isKey(EQModel.modelParams,'localVolSurface')
                    eqCOMDupireSpotGF.localVolSurface = EQModel.modelParams('localVolSurface');
                end
                
                if ~isKey(EQModel.modelParams,'calendarSpreadDt')
                    eqCOMDupireSpotGF.modelParams('calendarSpreadDt')= 0.5*0.000001/365.0;
                end
                
                eqCOMDupireSpotGF.modelSchedule = [];
                eqCOMDupireSpotGF.fwdMoneyness = [];
                
              %% SSR
                eqCOMDupireSpotGF.targetStrikes = [];
                
                % 1 factor local vol model
                eqCOMDupireSpotGF.numOfFactors = 1;

                %fixedStrike
                eqCOMDupireSpotGF.deltaCutOff = 0;
                eqCOMDupireSpotGF.useYN = [];
            end
        end
        
        function out= expmReghai(eqCOMDupireSpotGF,A,t)
            % Scale A by power of 2 so that its norm is < 1/2 .
            A = A*t;
            [f,e] = log2(norm(A,'inf'));
            s = max(0,e+1);
            A = A/2^s;
            Aexp = eye(size(A,1)) + A;
            testflag = 0;
%             tic
            if ~testflag
                for i=1:s
                    Aexp = Aexp*Aexp;
                end
            else
                D =Aexp;
                B= Aexp;
                E = zeros(size(A,1),size(A,1));
                for i=1:s
                    E = D*B;
                    D = E;
                    B = E;
                end
                Aexp = E;
            end
%             toc
            out = Aexp;
        end
        
        function path= bbgenerator(eqCOMDupireSpotGF,rn)
            %generate brownian bridge 
            
            [Npaths,Nsteps]=size(rn);
            path=zeros(Npaths,Nsteps);
            map= zeros(1,Nsteps);
            bridgeindex = zeros(1,Nsteps);
            leftindex = zeros(1,Nsteps);
            rightindex = zeros(1,Nsteps);
            sigma = zeros(1,Nsteps);
            leftweight = zeros(1,Nsteps);
            rightweight = zeros(1,Nsteps);
            
            map(Nsteps)=1;
            bridgeindex(1)=Nsteps;
            sigma(1)=sqrt(Nsteps);
            leftweight(1)=1; 
            rightweight(1)=1; 
            j=1;
            for i=2:Nsteps    
                while map(j) 
                    j=j+1;
                end
                k=j;
                while ~map(k) 
                    k=k+1;
                end
                l=j+fix((k-1-j)/2);
                map(l)=i;
                bridgeindex(i)=l;
                leftindex(i)=j;
                rightindex(i)=k;
                leftweight(i)  = (k-l)/(k+1-j);
                rightweight(i) = (l+1-j)/(k+1-j);
                sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
                j=k+1;
                if j>=Nsteps+1
                    j=1;
                end
            end

            path(:,Nsteps)=sigma(1)*rn(:,1);
            for i=2:Nsteps
                j=leftindex(i);
                k=rightindex(i);
                l=bridgeindex(i);
                if j~=1
                    path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                else
                    path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
                end
            end
        end
        
        function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
            size = tdma.size;
            out = vector;
            
            for i=2:size
                out(i) = out(i) - tdma.ll(i)*out(i-1);
            end
            
            out(size) = out(size)/tdma.dd(size);
            
            for i=size-1:-1:1
                out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
            end
            
        end
        
        function out = initFuturesStates(eqCOMDupireSpotGF,NMC)
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            out =  spotPrice*ones(NMC,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            out = nextFuturesStates;
        end
        
        function out = LUDecompose(eqCOMDupireSpotGF,tdma)
            out = tdma;
            if out.bupdated == 1
                for i=1:out.size
                    out.uu(i) = out.c(i);
                end
                
                out.dd(1) = out.b(1);
                for i=2:out.size
                    out.ll(i) = out.a(i)/out.dd(i-1);
                    out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
                end
                
                out.bupdated = 0;
            end
        
        end
        
        function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
    
            x = zeros(n,1);
            y = zeros(n,1);
            d = zeros(n,1);
            %fwd

            x(1) = 1.0;
            x(2) = 1.0;

            for i=3:n
                x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
            end

            %bwd
            y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
            y(n-1) = y(n);

            for i=n-2:-1:1
                y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
            end

            %set d

            d(1) = x(1)*y(1)/b(1);
            for i=2:n
                d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
            end


        end

        
        function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)

            localVol = zeros(length(Ks),1);
            volProxyKs = zeros(1,length(volProxy));
            try
                fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            catch ME
                aaa = 0;
            end
            
            if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
                if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
                    aaa =1.0;
                else
                    for i=1:length(volProxy)
                        volProxyKs(i) = Ks(ipos(i));
                    end

                    localVol = pchip(volProxyKs,volProxy,Ks);
                    %flat extrapolation at the ends
                    for i=1:length(Ks)
                        if Ks(i) <= Ks(ipos(1))
                            localVol(i) = localVol(ipos(1));
                        elseif Ks(i) >= Ks(ipos(length(ipos)))
                            localVol(i) = localVol(ipos(length(ipos)));
                        end
                    end
                end
            elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
                for i=1:length(volProxy)
                    volProxyKs(i) = fwdMoneynessPerExpiry(i);
                end
                
                for i=1:length(Ks)
                    localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
                end   
                
            else
                for i=1: length(volProxy)
    %                 Extrapolation at the left and right
                    if i == 1
                        for j=1:ipos(i)-1
                            localVol(j) = volProxy(i);
                        end
                    elseif i == length(volProxy)
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                        for j=ipos(i):length(Ks)
                            localVol(j) = volProxy(i);
                        end
    %                  linear interpolation in the spot grid in-between    
                    else
                        for j=ipos(i-1):ipos(i)-1
                            localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
                        end
                    end
                end
            end
            
        end
                        
        
        function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastC),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshC = lastC;
            meshP = lastP;
            
            tdma.size = length(meshC);
            tdma.bupdated = 0;
            tdma.a = zeros(tdma.size,1);
            tdma.b = zeros(tdma.size,1);
            tdma.c = zeros(tdma.size,1);
            tdma.uu = zeros(tdma.size,1);
            tdma.ll = zeros(tdma.size,1);
            tdma.dd = zeros(tdma.size,1);
            
            dK2 = dK*dK;
            for i=2:tdma.size-1
                tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            tdma.a(1) = 0;
            tdma.a(tdma.size) = 0;
            tdma.c(1) = 0;
            tdma.c(tdma.size) = 0;
            tdma.b(1) = 1;
            tdma.b(tdma.size) = 1;
            tdma.bupdated = 1;
            
            tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
            pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
            newC = pmeshC;
            
            pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
            newP = pmeshP;
        end
        
        function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
            
            %filling proxy grid vol
            proxy = zeros(length(lastProb),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);

            %filling end
            
            meshProb = lastProb;
            
            size = length(meshProb);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            
            condProb = zeros(size,size);
            if optParams.params.GFType == -1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 tic
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 toc
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 0
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            
            elseif optParams.params.GFType == 1
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newProb = lastProb*A_exp1;
                condProb =A_exp1;
            elseif optParams.params.GFType == 2
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newProb = lastProb;
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*A_exp1;
                    condProb = A_exp1*A_exp1;
                end
                
            elseif optParams.params.GFType == 3
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                newProb = newProb*oneStep;
                
                condProb = oneStep;
                
            else
                oneStepDt = optParams.params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
            
                % Test for HugeDecomposition for inverse matrix
                [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
                A = gallery('tridiag',a1,b,c1);
                oneStep = OneStepGF(size,x,y,d,A);
                newProb = lastProb;
                %numberOfTimeStep;
                %oneStepDt = 1.0/365;
                
                condProb = eye(size,size);
                for t=1:NT
                    newProb = newProb*oneStep;
                    condProb = oneStep*oneStep;
                end 
                
            end
            
            out.newProb = newProb;
            out.condProb = condProb;
        end
        
        function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = params.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = params.Ks(2) - params.Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            if params.backGFType == -1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
                newP = A_exp1*lastP;
                
            elseif params.backGFType == 0
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expmdemo1(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 1
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg*NT);
                newP = A_exp1*lastP;
            elseif params.backGFType == 2
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == -21
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
                A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
%                 A_exp1 = expm(A_rg);
                
                newP = lastP;
                
                for t=1:NT
                    newP = A_exp1*newP;
                end
                
            elseif params.backGFType == 3

                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);

                
            elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
                oneStepDt = params.oneStepDt;
                NT = round(dT/oneStepDt);
                
                % rescale matrix element to smaller time step
                a= a*oneStepDt/dT;
                b = (b-1)*oneStepDt/dT + 1.0;
                c = c*oneStepDt/dT;
                
                a1 = a(2:size);
                c1 = c(1:size-1);
                
                tdma.size = length(lastP);
                tdma.a = a;
                tdma.b = b;
                tdma.c = c;
                tdma.uu = zeros(tdma.size,1);
                tdma.ll = zeros(tdma.size,1);
                tdma.dd = zeros(tdma.size,1);

                tdma.bupdated = 1;

                tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
                
                newP = lastP;
                for t=1:NT
                    newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
                end

            else 
                disp('unImplemented')
            end
        end
        
        function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end

        function newP = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,fromTime,toTime,lastP)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            Ks = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            
            proxy = eqCOMDupireSpotGF.pricingVolGrid(idxNow,:);
            
            %filling end
            
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (fromTime - toTime)/365.0;
            
%             dK2 = dK*dK;
%             for i=1:size
%                 temp = 0.5*dT*proxy(i)*proxy(i)/dK;
%                 a(i) = -temp*(1.0/dK);
%                 b(i) = 1 + 2.0/dK*temp;
%                 b_rg(i) = b(i) - 1.0;
%                 c(i) = -temp*(1.0/dK);
%             end
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = A_exp1*lastP;
            
        end
        
        function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            idx = find(expiry > fromTime & expiry < toTime);
            if isempty(idx)
                out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime);
            else
                
                timeStep = expiry(idx);
                defaultTimeStep = [fromTime;toTime];
                timeStep = sort(union(timeStep,defaultTimeStep),'ascend');
                condProbOneStep = zeros(length(Ks),length(Ks));
                condProb = eye(length(Ks),length(Ks));
                
                for i=1:length(timeStep)-1
                    condProbOneStep = generateCondProbOneStep(eqCOMDupireSpotGF,timeStep(i),timeStep(i+1));
                    condProb = condProb*condProbOneStep;
                end
                
                out =  condProb;
            end
            
            
        end
        
        function out = generateCondProbOneStep(eqCOMDupireSpotGF,fromTime,toTime)
            
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(Ks),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
            %filling end
            
            size = length(proxy);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);
            
            dK = Ks(2) - Ks(1);
            dT = (toTime - fromTime)/365.0;
            
            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end
            
            a(1) = 0;
            c(size) = 0;
            
            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;
            
            b(1) = 1;
            b(size) = 1;
            
            % to create tridiagonal matrix
            
            a1 = a(2:size);
            c1 = c(1:size-1);
            
            b_rg(1) = 0;
            b_rg(size) = 0;
            
            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            out = A_exp1;
            
        end
        
        function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
        end

        function localVol = InterpolateLocalVolSSSABR(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry;
            
            %find which strip fromTime belongs to 
            idxS = find(expiry > fromTime);
            if isempty(idxS)
                idxStrip = length(expiry);
            else
                idxStrip = min(idxS);
            end
            
            % find exact timeSlice in the given strip
            localVolTimeStep = eqCOMDupireSpotGF.localVolSurface.localVolTimeStep{idxS};
            localVolSurfaceSABR = eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR{idxS};
            forwardMoneyness = eqCOMDupireSpotGF.localVolSurface.forwardMoneyness{idxS};

            idx = find(localVolTimeStep > fromTime);
            if isempty(idx)
                idxNow = length(localVolTimeStep);
            else
                idxNow = min(idx);
            end
            
            localVolLine = localVolSurfaceSABR(idxNow,:);
%             forwardMoneynessLine = forwardMoneyness(idxNow,:);
            
            localVol =  zeros(length(x),1);
            
            dummyOut = interp1FlatExtrapNew(forwardMoneyness,localVolLine,x','linear');
            dummyOut2 = sqrt(dummyOut);
            localVol = dummyOut2';
            
%             for ii =1:length(x)
%                 dummyVariance = H_interpolation(forwardMoneyness,localVolLine,x(ii),1);
%                 localVol(ii) = sqrt(dummyVariance);
%             end

        end
        
        function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)

            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %             Ks = params.Ks;
            localVol =  zeros(length(x),1);
            fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            % we use the same linear interpolation scheme as calibration
            useVolProxyInterpolationYN = 'YES';
            if strcmp(useVolProxyInterpolationYN,'YES')
                
                for ii =1:length(x)
                    localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
                end
            else
                volKnotPoints = zeros(length(ipos),1);
                for i=1: length(ipos)
                    volKnotPoints(i) = Ks(ipos(i));
                end


                for i=1:length(x)
                    xIdx = find(volKnotPoints <=x(i));
                    if isempty(xIdx)
                        localVol(i) = localVolLine(1);
                    elseif max(xIdx) == ipos(length(ipos))
                        localVol(i) = localVolLine(length(ipos));
                    else
                        localVolIdx = max(xIdx);
                        weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
                        localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
                    end
                end
            end
            
        end
        
        function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
            
            NMC = length(lastX);
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        function out = inductMCMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U)
            
            NMC = length(lastX);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            % we generate conditional probability matrix first
            condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
            
%             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
            uSample = U;
            
            nextX = zeros(NMC,1);
            nextXIdx = zeros(NMC,1);
%             Ks = params.Ks;
            nPoints = length(Ks);
            
            for i=1:NMC
                IndexMc = lastXIdx(i);
                j= IndexMc;
                cumProb = Qa(IndexMc,:);
                idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
                nextX(i) = Ks(idxH);
                nextXIdx(i) = idxH;

            end
            
            out.nextX = nextX;
            out.nextXIdx = nextXIdx;

            
        end
        
        
        
        function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
            
            timeSchedule = timeScheduleInfo.timeSchedule;
            dTSchedule365 = timeScheduleInfo.dTSchedule365;
            volKnotIdx = timeScheduleInfo.volKnowIdx;
            
            predictor = lastX;
            for j=startIdx:endIdx-1
                
                localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
                Dt = dTSchedule365(j);
                sqrtDt = sqrt(dTSchedule365(j));
                for i=1:length(lastX)
                    predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
                end
            end
            newX = predictor;
            
        end
        
         function mcmcOut = inductMCForwardNewReducedQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
         end
        
         function mcmcOut = inductMCForwardNewReducedNMonthsQuanto(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)

            mcmcOut1 = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ);
            mcmcOut.nextFuturesStates = mcmcOut1.nextX;
            mcmcOut.idxNM = 1;
        end

        function mcmcOut = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            lookupTime = max(fromTime-0.01,0);
            localVol = eqCOMDupireSpotGF.InterpolateLocalVolSSSABR(predictor,lookupTime);
            
            detFwdFrom = eqCOMDupireSpotGF.FwdFactor(fromTime); 
            detFwdTo =  eqCOMDupireSpotGF.FwdFactor(toTime); 
            detFwdFactor = detFwdTo/detFwdFrom;
            
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            predictor = predictor*detFwdFactor;
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
            %lookup issue add small epsilon to fromTime
            localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            for i=1:length(lastX)
%                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
                predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
                
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCForwardNewPC(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
            
            previousState = lastX;
            %lookup issue add small epsilon to fromTime
            volBegin = eqCOMDupireSpotGF.InterpolateLocalVol(previousState,fromTime + 0.01);
            Dt = (toTime - fromTime)/365.0;
            sqrtDt = sqrt(Dt);
            predictor = previousState.*exp(-0.5.*volBegin.*volBegin*Dt + 1.0.*volBegin.*sqrtDt.*dZ); 
            
%             for i=1:length(lastX)
%                 predictor(i) = predictor(i)*exp(-0.5*volBegin(i)*volBegin(i)*Dt + volBegin(i)*sqrtDt*dZ(i)); 
%             end
            
            volEnd = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,toTime - 0.01);
            corrector = previousState.*exp(-0.5.*0.5.*volBegin.*volBegin*Dt-0.5.*0.5.*volEnd.*volEnd*Dt ...
                        + 1.0.*volBegin.*sqrtDt.*dZ); 
            
            mcmcOut.nextX = corrector;
            
        end
        
        function out = ComputePutOTMPricesMCMC(eqCOMDupireSpotGF,StrikeSim,nextXIdx,fromTime,toTime)
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            if fromTime == toTime-1
                bbb = 1.0;
            end
            
            if fromTime == toTime
                condProb = eye(length(Ks));
            else
                condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
            end
            
            SpotMCMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,Ks);
            putOTM = zeros(length(nextXIdx),1);
            
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);
            
            df_ft = df_t/df_f;
            putOTMPrices = zeros(length(Ks),1);
            
            for i=1:length(nextXIdx)
                transProb = condProb(nextXIdx(i),:);
                % vectorization
                putOTMPrices = max(StrikeSim(i)*ones(length(transProb),1) - SpotMCMC,0);
                putOTM(i) = transProb*putOTMPrices;
                
            end
            
            putOTM = putOTM*df_ft;
            out = putOTM;
            
        end
        
        function out = ComputePutOTMPricesMC(eqCOMDupireSpotGF,StrikeSim,nextX,fromTime,toTime,nestedU,nestedNMC,nestedMCOneTimeStep)

            putOTM = zeros(length(nextX),1);
            putOTMPrices = zeros(nestedNMC,1);
            
            if fromTime == toTime
                for i=1:length(nextX)
                    SpotMC = eqCOMDupireSpotGF.EQSpotMC(toTime,nextX(i));
                    % vectorization
                    putOTM(i) = max(StrikeSim(i) - SpotMC,0);
                   
                end
            
            else

                df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
                df_t = eqCOMDupireSpotGF.zeroCurve.DF(toTime/365.0);

                df_ft = df_t/df_f;
                

                %daily timeSteps
                nestedMCTimeStepsSize = toTime - fromTime;
                nestedTimeStep = [toTime:-1*nestedMCOneTimeStep:fromTime]';
                defaultTimeStep = [fromTime; toTime];
                
                nestedTimeStep = sort(union(defaultTimeStep,nestedTimeStep),'ascend');
                
                expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
                idx = find(expiry > fromTime & expiry < toTime);
                if ~isempty(idx)
                    volTimeStep = expiry(idx);
                    nestedTimeStep = sort(union(volTimeStep,nestedTimeStep),'ascend');
                end
                
                nestedMCTimeStepsSize = length(nestedTimeStep);
                
                for i=1:length(nextX)
                    nestedNextX = nextX(i)*ones(nestedNMC,1);
                    for j=1:nestedMCTimeStepsSize-1
                        mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,nestedTimeStep(j),nestedTimeStep(j+1),nestedNextX,nestedU(:,j));
                        nestedNextX = mcmcOut.nextX;
                    end

                    SpotMC = eqCOMDupireSpotGF.EQSpotMCMC(toTime,nestedNextX);
                    % vectorization
                    putOTMPrices = max(StrikeSim(i)*ones(length(nestedNMC),1) - SpotMC,0);
                    putOTM(i) = mean(putOTMPrices);

                end
                
                putOTM = putOTM*df_ft;
            end
            
            
            out = putOTM;
            
        end
        
        function out = UpdatePutShortIndex(eqCOMDupireSpotGF,indexSim,fixingS,prevOTM,currOTM,fromTime,toTime)
%             cd = 
            df_f = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0);
            df_fterm = eqCOMDupireSpotGF.zeroCurve.DF(fromTime/365.0 + 0.25);
            fwdCD = (df_f/df_fterm -1.0)/0.25; 
            dt = (toTime-fromTime)/365.0;
            
            indexUpDate = zeros(length(indexSim),1);
            for i=1:length(indexSim)
                indexUpDate(i) = indexSim(i)*(1+ (-currOTM(i)+prevOTM(i))/fixingS(i) + (1-0.075)*fwdCD*dt);
            end
            
            out = indexUpDate;
        end
        
        function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
                    out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
            % Calculate Object Function
            % settings for bisection
            a = 0.001;
            b = 10;
            Tol = 1e-10;
            MaxIter = 1000;

            strikes = params.marketStrikes;
            strikeSize = length(params.marketStrikes);
            out.vols    = zeros(strikeSize,1);
            out.interpOTMPrices = zeros(strikeSize,1);
            out.interpBlackPrices = zeros(strikeSize,1);
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            cValue = meshProb*meshC;
            pValue = meshProb*meshP;
            
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                if K <= 1.0
%                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
                     out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
                     out.interpOTMPrices(i) = pValue(i);
                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
                     
                else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
                    out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
                    out.interpOTMPrices(i) = cValue(i);
                    out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
                end
            end
            
            
        end
        
        function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
            expiry = params.expiry;
            
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            
            mesh = zeros(modelXSize,1);
            
            if strike <= 1.0
                for i=1:modelXSize
                    mesh(i,1) = max(strike- modelX(i),0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            else
                for i=1:modelXSize
                    mesh(i,1) = max(modelX(i)-strike,0);
                end

                volExpiry = expiry(find(expiry <= maturity));
                volExpiry = [volExpiry; 0];
                timeStep = sort(union(volExpiry,maturity),'ascend');
                for i=length(timeStep):-1:2
                    mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
                end
                
                out = mesh(params.Pricingidx);
                
            end
            
        end
        
        function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwd);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwd(idx11) >= basePrice*strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwd(idx11) < basePrice*callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
                            elseif comFwd(idx11) < basePrice*strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwd(idx11) > basePrice*lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function columnOut = MCPayoffWorst(eqCOMDupireSpotGF,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function out = EQSpotMCMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQSpotMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
            spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
            detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
             
             %stochasticFwd(0,T)
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = detFwd * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = EQFwdMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = fxDupireSpotGF.localVolSurface.Ks;
%         spotPrice = eqCOMDupireSpotGF.spot.params('rawData');
%         detFwd = eqCOMDupireSpotGF.Fwd(spotPrice,observeTime); 
% 
%          %stochasticFwd(0,T)
%          stateSize = length(modelStates);
%          stochasticFwd = zeros(stateSize,1);
% 
%          for i=1:stateSize
%             stochasticFwd(i) = detFwd * modelStates(i); 
%          end

%          out = stochasticFwd;
         
         out = modelStates;
        end
        
        % NM : nearest month forward
        function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end

        function out = CommoFwdNMFinePDEGrid(eqCOMDupireSpotGF,observeTime)
            modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
           % NM : nearest month forward
        function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
            if observeTime > maturityTime
                disp('observeTime is passed maturityTime!')
                out = 0;
                return;
            end
            
            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(maturityTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             %fwd(0,T_NM)
             fwd_NM = forwardCurve(idxNM,2);
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        function out = CommoFwdContractMMCReduced(eqCOMDupireSpotGF,observeTime,maturityTime,modelStates)
            
            reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
            forwardCurveSize = size(reducedFutures,1);
            forwardMaturity = reducedFutures(:,1); 
            idx = find(maturityTime == forwardMaturity);
             if isempty(idx)
                error('contractM is not in the reducedFutures!!')
%                 idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             fwd_NM = reducedFutures(idxNM,2);
             
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
            
           
        end
        
        function out = CommoFwdNMMCReduced(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCReducedNMonths(eqCOMDupireSpotGF,observeTime,modelStates)
            % for eqCOMDupireSSSABR
            out = eqCOMDupireSpotGF.EQFwdMC(observeTime,modelStates);
        end
        
        function out = CommoFwdNMMCInterp(eqCOMDupireSpotGF,observeTime,modelStates)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
            forwardCurveSize = size(forwardCurve,1);
            forwardMaturity = forwardCurve(:,1); 
            idx = find(observeTime <= forwardMaturity);
             if isempty(idx)
                idxNM = forwardCurveSize;
             else
                idxNM = min(idx);
             end
             
             % interpol using reducedFutures
             reducedFutures = eqCOMDupireSpotGF.modelParams('reducedFutures');
             
             %fwd(0,T_NM)
             targetExpiry = forwardCurve(idxNM,1);
             % interpolated Fwd
             interpolFwd = H_interpolation(reducedFutures(:,1),reducedFutures(:,2),targetExpiry,1);
             
%              fwd_NM = forwardCurve(idxNM,2);
             
             fwd_NM = interpolFwd;
             
             stateSize = length(modelStates);
             stochasticFwd = zeros(stateSize,1);
             
             for i=1:stateSize
                stochasticFwd(i) = fwd_NM * modelStates(i); 
             end
             
             out = stochasticFwd;
        end
        
        
        function out = computeLocalVolGridSSSABR_KYJ_FwdFactor(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
%                 detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
%                 detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
%                 detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
%                 detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                detFwd(idxjj)   = S_ghost     * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactorOrg(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
%                 detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
%                 detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
%                 detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
%                 detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                detFwd(idxjj)   = S_ghost     * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactor(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
%                 detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
%                 detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
%                 detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
%                 detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                detFwd(idxjj)   = S_ghost     * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * eqCOMDupireSpotGF.FwdFactor(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * eqCOMDupireSpotGF.FwdFactor(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end

        function out = computeLocalVolGridSSSABR_KYJ_FwdMN(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
%                     
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;

                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end

        function out = computeLocalVolGridSSSABR_KYJ(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            
            timeGridNew = zeros(length(timeGrid),1);
            
            for idxaa=1:length(timeGrid)-1
                timeGridNew(idxaa) = timeGrid(idxaa+1);
            end
            dTimeGrid = timeGrid(end) - timeGrid(end-1);
            timeGridNew(length(timeGrid)) = timeGridNew(length(timeGrid)-1) + dTimeGrid;
            
            timeGrid = timeGridNew;

            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
%                     d1 = (f_total_var - b_total_var)*0.5*ddt;
                    d1 = (f_total_var - b_total_var)*ddt;    

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end

        function out = computeLocalVolGridSSSABR(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,strike_lookup,b_matT);
                     
                    f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;

%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol(1.0,b_strike_lookup,matT);
                    
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;
                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                    
                    if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                    end
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;
                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                     
%                     f_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
%                     b_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
%                     
                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeLocalVolGridSSSABR_Qlib_FwdMN(eqCOMDupireSpotGF,timeGrid,strikeMoneynessGrid,settleFuturesRef,rfz1D)
            % we look up implied volatility using smileDynamicsAdjFactors
            % applied  strikeMoneynessGrid

            % and when we calculate local vol using dupire formula we use
            % original strikeMoneynessGrid
            spotRawData = eqCOMDupireSpotGF.spot.params('rawData');
            S_ghost = spotRawData/(1.0 + eqCOMDupireSpotGF.slope*(spotRawData/settleFuturesRef - 1.0));
%             strikeMoneynessGridNew = strikeMoneynessGrid/spotRawData;
%             lookupMoneynessGrid = strikeMoneynessGridNew * (1.0 + eqCOMDupireSpotGF.slope*(smileDynamicsAdjFactors - 1.0));
            localVolStrip = zeros(length(timeGrid),length(strikeMoneynessGrid));
            
%             dt = 0.5*0.000001/365.0;
            
            dt = eqCOMDupireSpotGF.modelParams('calendarSpreadDt');
            
            ddt = 1.0/dt;
            dk = 0.01;
            ddk = 100.0;
            ddk2 = 10000.0;
            
            if timeGrid(1) == 0
                timeGrid(1) = 0.5*timeGrid(2);
            end
            timeGridR = timeGrid/365.0;
            
            detFwd = zeros(length(timeGrid),1);
            detFwd_f = zeros(length(timeGrid),1);
            detFwd_b = zeros(length(timeGrid),1);
            detFwd_lv = zeros(length(timeGrid),1);
            for idxjj = 1:length(timeGrid)
                matT   = timeGridR(idxjj);
                f_matT = timeGridR(idxjj) + dt;
                b_matT = timeGridR(idxjj) - dt;
                
                detFwd(idxjj)   = S_ghost     * exp(rfz1D*matT)   * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                detFwd_f(idxjj) = S_ghost     * exp(rfz1D*f_matT) * eqCOMDupireSpotGF.GetFwdDividends(f_matT*365.0);
                detFwd_b(idxjj) = S_ghost     * exp(rfz1D*b_matT) * eqCOMDupireSpotGF.GetFwdDividends(b_matT*365.0);
                detFwd_lv(idxjj)= spotRawData * exp(rfz1D*matT) * eqCOMDupireSpotGF.GetFwdDividends(matT*365.0);
                
                for idxkk = 1:length(strikeMoneynessGrid)
                    
                     if idxjj == 46 && idxkk == 40
                        debugPoint2 = 0.0;
                     end
                    
                    f_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(dk);
                    b_strike_lookup =  strikeMoneynessGrid(idxkk) * exp(-1.0*dk);
                    strike_lookup = strikeMoneynessGrid(idxkk);
                    logMoneyness = log(strikeMoneynessGrid(idxkk)) - log(detFwd_lv(idxjj));
                    
%                     sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),strike_lookup,matT);
                    sabr_imv = dummyOut.vol;

                    
                    total_var = sabr_imv * sabr_imv * timeGridR(idxjj);
                    
                    dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup*detFwd_f(idxjj)/detFwd(idxjj),f_matT);
                    dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup*detFwd_b(idxjj)/detFwd(idxjj),b_matT);
                    f_sabr_imv = dummy_f.vol;
                    b_sabr_imv = dummy_b.vol;
                    
%                     dummy_f = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_f(idxjj),strike_lookup,f_matT);
%                     dummy_b = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd_b(idxjj),strike_lookup,b_matT);
%                     f_sabr_imv = dummy_f.vol;
%                     b_sabr_imv = dummy_b.vol;
                    
                    f_total_var = f_sabr_imv * f_sabr_imv * f_matT;
                    b_total_var = b_sabr_imv * b_sabr_imv * b_matT;
                    d1 = (f_total_var - b_total_var)*0.5*ddt;
                    
%                     f_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
%                     b_k_sabr_imv = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);

                    dummy_f_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),f_strike_lookup,matT);
                    dummy_b_k = eqCOMDupireSpotGF.SABR_ImpVol_Qlib(detFwd(idxjj),b_strike_lookup,matT);
                    
                    f_k_sabr_imv = dummy_f_k.vol;
                    b_k_sabr_imv = dummy_b_k.vol;

                    
 
                    f_k_total_var = f_k_sabr_imv * f_k_sabr_imv * matT;
                    b_k_total_var = b_k_sabr_imv * b_k_sabr_imv * matT;

                    d2 = (f_k_total_var - b_k_total_var)*0.5*ddk;
                    d3 = (f_k_total_var - 2.0.*total_var + b_k_total_var)*ddk2;

                    localVolStrip(idxjj,idxkk) = d1/(1 - logMoneyness/total_var*d2 + 0.25*(-0.25 - 1/total_var + logMoneyness*logMoneyness/(total_var*total_var))*d2*d2 + 0.5*d3);
      
                end
            end
            
            out = localVolStrip;
        end
        
        function out = computeCalendarSpreadOptionMCReducedInt(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            numOfFactors = 1;
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
           initX = ones(NMC,1);
           nextX = initX;
%            
           % initial model States (futuresState)
%             nextFuturesStates = eqCOMDupireSpotGF.initFuturesStates(NMC);
            
%            forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%            forwardCurveSize = size(forwardCurve,1);
%            forwardMaturity = forwardCurve(:,1); 
% %            idx = find(0.0 <= forwardMaturity);
% %            if isempty(idx)
% % %               idxNM = forwardCurveSize;
% %                 error('no alive futures!')
% %            else
% %               idxNM = min(idx);
% %            end
%            
%            initFuturesStatesDefault = zeros(forwardCurveSize,NMC);
%            
%            for idx=1:forwardCurveSize
%                 initFuturesStatesDefault(idx,:) = forwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            nextFuturesStates = initFuturesStates;
           
%            nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     mcmcOut   = inductMCForwardNewReduced(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,2*(idx-1)+1:2*(idx-1)+numOfFactors));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           longTenorMaturityDate = H_Date(vanillaParams.params('longTenorMaturityDate'));
           shortTenorMaturityDate = H_Date(vanillaParams.params('shortTenorMaturityDate'));
           
           longTenorMaturityTime = DateDiff(longTenorMaturityDate,valueDate);
           shortTenorMaturityTime = DateDiff(shortTenorMaturityDate,valueDate);
           
           comFwdL = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,longTenorMaturityTime,nextX);
           comFwdS = eqCOMDupireSpotGF.CommoFwdContractMMCReduced(currentTime,shortTenorMaturityTime,nextX);
           
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwdL(i) - comFwdS(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwdL(i) + comFwdS(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwdL;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end

        function out = GenerateLocalVolSS(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_KYJ_FwdFactor(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactor(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdFactor(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactorOrg(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
%                 localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN_FwdFactor(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
function out = GenerateLocalVolSS_KYJ_FwdMN(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(1) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end

        function out = GenerateLocalVolSS_KYJ(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            % if option is expired then we neglect
            expiry = expiry(find(expiry> 0));
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_KYJ(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(idxhh),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                

%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(1)/settleFuturesRef(1);
                
                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end

                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = GenerateLocalVolSS_Qlib_FwdMN(eqCOMDupireSpotGF,valueDate,stepdownParams,useSSSABRAutoCallScheduleFlag)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if useSSSABRAutoCallScheduleFlag
                autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
                if autoCallSchedule.days(end) <= maturity
                    localVolStripSize = length(autoCallAliveExpiry);
                    localVolStripExpiry = autoCallAliveExpiry;
                else
                    localVolStripSize = length(autoCallAliveExpiry) + 1;
                    localVolStripExpiry = [autoCallAliveExpiry;maturity];
                end
                
            else
                if expiry(end) <= maturity
                    localVolStripSize = length(volExpiryAlive);
                    localVolStripExpiry = expiry;
                else
                    localVolStripSize = length(volExpiryAlive) + 1;
                    localVolStripExpiry = [volExpiryAlive;maturity];
                end
            end
            
            

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
          %% local vol surface construct from parmetric implied vol surface start
            
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            
            atmVols1D = zeros(length(localVolStripExpiry),1);
            rfz1D = zeros(length(localVolStripExpiry),1);
            for idxhh=1:length(localVolStripExpiry)
                atmVols1D(idxhh) = H_interpolation(atmVols(:,1),atmVols(:,2),localVolStripExpiry(idxhh),1);
                rfz1D(idxhh) = eqCOMDupireSpotGF.zeroCurve.Zero(localVolStripExpiry(idxhh)/365.0);
            end
            
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
% 
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
%                 x_max = settleFutures(idxhh)*max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 x_min = settleFutures(idxhh)*min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
%                 smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);
                
                x_max = settleFutures(1)*max(1.3,settleFuturesRef(1)/settleFutures(1)*exp( 5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = settleFutures(1)*min(0.7,settleFuturesRef(1)/settleFutures(1)*exp(-5.0*atmVols1D(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(1)/settleFuturesRef(1);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                 if idxhh == localVolStripSize
                    debugPoint = 0.0;
                end
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR_Qlib_FwdMN(timeStepStrip,forwardMoneynessStrip,settleFuturesRef(1),rfz1D(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
            out = eqCOMDupireSpotGF.localVolSurface;
            
        end
        
        function out = computeStepdown1SMCReducedInt(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReducedIntEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            
            volExpiryAlive = expiry(find(expiry <= maturity));
            
            if expiry(end) <= maturity
                localVolStripSize = length(volExpiryAlive);
                localVolStripExpiry = expiry;
            else
                localVolStripSize = length(volExpiryAlive) + 1;
                localVolStripExpiry = [volExpiryAlive;maturity];
            end

%             volExpiryAliveSize = length(volExpiryAlive);
            % we include valueDate in the scheduel
            volExpiry =[volExpiryAlive;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% local vol surface construct from parmetric implied vol surface start
            localVolSurfaceSABR = cell(localVolStripSize,1);
            forwardMoneyness = cell(localVolStripSize,1);
            localVolTimeStep = cell(localVolStripSize,1);
            
            settleFutures    = eqCOMDupireSpotGF.settleFutures;
            settleFuturesRef = eqCOMDupireSpotGF.settleFuturesRef;
            atmVols          = eqCOMDupireSpotGF.atmVols;
            

            smileDynamicsAdjFactors = zeros(localVolStripSize,1);
            dy = 0.05;
            for idxhh=1:localVolStripSize
                
                timeStepStrip1 = timeStep(find(timeStep <= localVolStripExpiry(idxhh)));
                if idxhh > 1
                    timeStepStrip = timeStepStrip1(find(timeStepStrip1 > localVolStripExpiry(idxhh-1)));
                else
                    timeStepStrip = timeStepStrip1;
                end
                
                timeStepStrip = sort(timeStepStrip,'ascend');
                localVolTimeStep{idxhh} = timeStepStrip;
                
                x_max = max(1.3,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp( 5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));
                x_min = min(0.7,settleFuturesRef(idxhh)/settleFutures(idxhh)*exp(-5.0*atmVols(idxhh)*sqrt(localVolStripExpiry(idxhh)/365.0)));

                smileDynamicsAdjFactors(idxhh) = settleFutures(idxhh)/settleFuturesRef(idxhh);

                gridSize = floor(log(x_max/x_min)/dy) + 1;
                localVolStrip= zeros(length(timeStepStrip),gridSize);
                forwardMoneynessStrip = zeros(1,gridSize);
                
                for idxjj=1:gridSize
                    forwardMoneynessStrip(idxjj) = x_min*exp((idxjj-1)*dy);
                end
                forwardMoneyness{idxhh} = forwardMoneynessStrip;
                
                localVolStrip = eqCOMDupireSpotGF.computeLocalVolGridSSSABR(timeStepStrip,forwardMoneynessStrip,smileDynamicsAdjFactors(idxhh));
                localVolSurfaceSABR{idxhh} = localVolStrip;
            end

            eqCOMDupireSpotGF.localVolSurface.localVolSurfaceSABR = localVolSurfaceSABR;
            eqCOMDupireSpotGF.localVolSurface.forwardMoneyness = forwardMoneyness;
            eqCOMDupireSpotGF.localVolSurface.localVolTimeStep = localVolTimeStep;
            eqCOMDupireSpotGF.localVolSurface.localVolStripExpiry = localVolStripExpiry;

          %% local vol surface construct from parmetric implied vol surface end
          
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = inductMCForwardNewSSSABR(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
              
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCReduced(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCReduced(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCInterp(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMCInterp(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCEQ(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCPC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewPC(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown1SMCFlat(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % flat vol 0.35
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            eqCOMDupireSpotGF.localVolSurface.volProxy = ones(size(fwdMoneyness))*0.35;
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'YES')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
                columnOut = eqCOMDupireSpotGF.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        
        function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 NMC = 40000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
                columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
             
        function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                    comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = unhitValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end

        function out = computeStepdown1SGFFinePDEGrid(eqCOMDupireSpotGF,valueDate,stepdownParams)
            %% generate local vol pricing grid start
            basePrice = stepdownParams.params('basePrice');
            eqCOMDupireSpotGF.basePrice = basePrice;
            
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy;
            fwdMoneyness = eqCOMDupireSpotGF.fwdMoneyness;
            impVolSurf = eqCOMDupireSpotGF.mktData('impliedVolSurface').volSurfaceR;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            fwdNM = zeros(size(expiry));
            for i=1:length(fwdNM)
                fwdNM(i) = eqCOMDupireSpotGF.FwdNM(expiry(i));
            end
            
            % grid size is log equi distant with 0.5% size
            dy = 0.0025;
              
            X_max = 2.5;
            X_min = 0.1;
            
            K = round((X_max-X_min)/dy);
            temp_pricingReducedForwardGrid = zeros(size(volProxy,1),K+1);
            temp_pricingVolGrid = zeros(size(volProxy,1),K+1);

            pricingVolGridSize = K+1;
            
            for i=1:size(volProxy,1)
                fwdMoneynessPerExpiry = fwdMoneyness(i,:);
                volProxyPerExpiry = volProxy(i,:);
                
                for j=1:K+1
                      temp_pricingReducedForwardGrid(i,j) = X_min + (j-1)*dy;
                      temp_pricingVolGrid(i,j) = H_interpolation(fwdMoneynessPerExpiry,volProxyPerExpiry,temp_pricingReducedForwardGrid(i,j),1);
                end
            end
            
            volNodeFwdCurve = zeros(length(expiry),2);
            
            for i=1:length(expiry)
                volNodeFwdCurve(i,1) = expiry(i);
                volNodeFwdCurve(i,2) = fwdNM(i);
            end
            
            pricingForwardGrid = temp_pricingReducedForwardGrid(end,:);
            pricingReducedForwardGrid = pricingForwardGrid;
            pricingVolGrid = temp_pricingVolGrid;
                
            eqCOMDupireSpotGF.pricingForwardGrid = pricingForwardGrid;
            eqCOMDupireSpotGF.pricingReducedForwardGrid = pricingReducedForwardGrid;
            eqCOMDupireSpotGF.pricingVolGrid = pricingVolGrid;
            eqCOMDupireSpotGF.volNodeFwdCurve = volNodeFwdCurve;
            
            %% pricing PDE Grid
            
            modelStatesFixedGrid = eqCOMDupireSpotGF.pricingReducedForwardGrid;
            modelStates = modelStatesFixedGrid;
            eqCOMDupireSpotGF.modelStatesFixedGrid = modelStatesFixedGrid;
            modelStatesSize = length(modelStatesFixedGrid);
          
            %% generate pricingVolGrid for localVol look-up & also for pricing fixed grid end
            
            
          %% PricingIdx & WeightNextPriceIndex update start
           
           spotLevel = 1.0;
           
           isFoundSpotLevel = 'false'; 
           for i=1:modelStatesSize-1
               
               if spotLevel >= modelStates(i) && spotLevel < modelStates(i+1)
                   PricingIdx = i;
                   WeightNextPriceIndex = (spotLevel - modelStates(i))/(modelStates(i+1)-modelStates(i));
                   isFoundSpotLevel = 'true';
                   break;
               end
           end
           
           if strcmp(isFoundSpotLevel,'false')
               error('spot price is out of grid range!');
           end
           
          %% PricingIdx & WeightNextPriceIndex update end
            
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           
           modelStates = eqCOMDupireSpotGF.modelStatesFixedGrid;
           
           modelStatesSize = length(modelStates);
           
           %down barrier touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
          
          %% fixedGrid fwd
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
           comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
           
           strikeMat = autoCallSchedule.strike(scheduleSize);
           couponMat = autoCallSchedule.coupon(scheduleSize);
           
%            strikeOH = strikeMat + overHedgeShift;
           % samsung overhedge payoff
           if strcmp(SSpayoffYN,'YES')
               for i=1:modelStatesSize
                   if comFwd(i) < basePrice*callStrike1
                        payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
                   elseif comFwd(i) < basePrice*strikeMat 
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
               
           else
               for i=1:modelStatesSize
                   if comFwd(i) >= basePrice*strikeMat
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
                   else
                       payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
                       payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
                   end
               end

               payoffStateB.cashflow = payoffStateA.cashflow;

               for i=1:modelStatesSize
                    if comFwd(i) > basePrice*lowerBarrier
                        payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
                    end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            hitValue.payoff = payoffStateA.cashflow_npv;
            
            unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            unhitValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = hitValue.payoff;
                meshB = unhitValue.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     mesh  = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh,basePrice);
%                     meshB = inductGFBackwardFixedGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB,basePrice);
                    
%                     mesh  = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackwardPricingVolGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    
                    mesh  = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackwardFinePDEGrid(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                  %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                   
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           meshB(j) = mesh(j); 
                        end
                    end
                end
                
                hitValue.payoff = mesh;
                unhitValue.payoff = meshB;
                nMFuture.payoff = comFwd;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
              %% AtDate operation on Early ExerciseDate
                
                if i > lastAliveExeriseIdx
                    currentTime = timeStep(lastTimeIdx);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValue  = hitValue.payoff/onePayoffD;
                    contiValueB = unhitValue.payoff/onePayoffD;
                    
                    %AutoCall applied
                 %% fixedGrid fwd
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMFixedGrid(currentTime);
                    comFwd = eqCOMDupireSpotGF.CommoFwdNMFinePDEGrid(currentTime);
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                     % intermediate variables
                    payoffStateA.cashflow = zeros(modelStatesSize,1);
                    payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    
                    payoffStateC.cashflow = zeros(modelStatesSize,1);
                    payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
                    
                    % samsung overhedge payoff
                    if strcmp(SSpayoffYN,'YES')
                        for j=1:modelStatesSize
                            if comFwd(j)/basePrice >= strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                            else
%                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% 
%                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                                
                                payoffStateA.cashflow(j) = contiValue(j);
                                payoffStateB.cashflow(j) = contiValueB(j);
                           end
                        end
                    else
                        for j=1:modelStatesSize
                            if comFwd(j) >= basePrice*strikeEarly
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
                            else
                               payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
                               payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
                               payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
                           end
                        end
                        
                    end
                    % barrier
                    for j=1:modelStatesSize
                        if comFwd(j) <= basePrice*lowerBarrier
                           payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
                        end
                    end
                    
                    payoffStateC.cashflow = comFwd;
                    
                    for j=1:modelStatesSize
                        payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
                        payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
                        payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
                    end
                    
                    %process payoff at early exercise date
                    hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
                    hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
                    hitValue.payoff = payoffStateA.cashflow_npv;
                    
                    unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
                    unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
                    unhitValue.payoff = payoffStateB.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
                    nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
                    nMFuture.payoff = payoffStateC.cashflow_npv;
                    
                end
                   
           end
           
           
            %% interpolate on the grid
            hitValue.npv = (1-WeightNextPriceIndex)*hitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*hitValue.payoff(PricingIdx+1);
                             
            unhitValue.npv = (1-WeightNextPriceIndex)*unhitValue.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*unhitValue.payoff(PricingIdx+1);
                             
            nMFuture.npv = (1-WeightNextPriceIndex)*nMFuture.payoff(PricingIdx) ...
                                 + WeightNextPriceIndex*nMFuture.payoff(PricingIdx+1);                 
            
           
           out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
           modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = length(modelStates);
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           currentTime = timeStep(totalTimeStepSize);
           currentTimeIdx = totalTimeStepSize;
           currentNodeIdx = scheduleSize;
           
           comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
           dummyVar = 0;
           if isKey(vanillaParams.params,'digitalYN')
               if strcmp(vanillaParams.params('digitalYN'),'YES')
                   digitalCoupon = vanillaParams.params('digitalCoupon'); 
                   if strcmp(vanillaParams.params('callPutFlag'),'C')
                       for i=1:modelStatesSize
                           if (comFwd(i) >= strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   else
                       for i=1:modelStatesSize
                           if (comFwd(i) <  strike) dummyVar = 1.0;else dummyVar = 0.0;end;
                            payoffStateA.cashflow(i) = digitalCoupon*dummyVar;
                            payoffStateB.cashflow(i) = digitalCoupon*dummyVar;
                       end
                   end
               end
                   
               
           else
               if strcmp(vanillaParams.params('callPutFlag'),'C')
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                        payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
                   end
               else
                   for i=1:modelStatesSize
                        payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                        payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
                   end
               end
           end
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           dummyMeshB = zeros(length(timeStep),2);
           dummyIdx = 1;
           dummyMeshB(dummyIdx,1) = eventTimeIdx(scheduleSize);
           dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
           dummyIdx = dummyIdx + 1;
           for i=scheduleSize:-1:lastAliveExerciseIdx
                % past schedule neglect
                startTimeIdx = eventTimeIdx(i);
                endTimeIdx = 1;  % valueDate
                
                endTimeIdx = endTimeIdx + 1;
            %% induction start
                mesh = europeanValue.payoff;
                meshB = americanValue.payoff;
                meshC = nMFuture.payoff;
                for idx= startTimeIdx:-1:endTimeIdx
                    if idx== 292
                        bbb =1.0;
                    end
                    mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
                    meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
                    meshC = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshC);
                    % one step backward induction end
                    lastTimeIdx = lastTimeIdx -1;
                    currentTime = timeStep(lastTimeIdx);
                    
                    %apply american option
                    comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
                    onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    contiValueB  = meshB/onePayoffD;
                    
                    payoffStateB.cashflow = zeros(modelStatesSize,1);
                    payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
                    if strcmp(vanillaParams.params('callPutFlag'),'C')
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
                            if payoffStateB.cashflow(j) > contiValueB(j)
                                ccc = 1.0;
                            end
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                        
                    else
                        for j=1:modelStatesSize
                            payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
                            payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
                        end
                    end
                    payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
                    meshB = payoffStateB.cashflow_npv;
                    dummyMeshB(dummyIdx,1) = timeStep(lastTimeIdx);
                    dummyMeshB(dummyIdx,2) = payoffStateB.cashflow_npv(Pricingidx);
                    dummyIdx = dummyIdx + 1;
                end
                
                europeanValue.payoff = mesh;
                americanValue.payoff = meshB;
                nMFuture.payoff = meshC;
              %% induction end
                
                % backward induction in the payoff script
                currentNodeIdx = currentNodeIdx -1;
           end
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = europeanValue.payoff(Pricingidx);
           europeanValue.npvUnd = europeanValue.payoff(Pricingidx)/df_ep;
           
           americanValue.npv = americanValue.payoff(Pricingidx);
           nMFuture.npv = nMFuture.payoff(Pricingidx);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeVanillaMC(eqCOMDupireSpotGF,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
           comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeBarrierMCEQ(eqCOMDupireSpotGF,valueDate,barrierParams)
            % model schedule generation
            expiryDate = H_Date(barrierParams.endDate);
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(barrierParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
            %% MC init 
            useQuasiRandomYN = eqCOMDupireSpotGF.modelParams('useQuasiRandomYN');
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
            else
                rng('default');
                rng(0);
                
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           
           nextX = initX;
           
          %% grid generation based on basePrice
            
           
          %% payoff at maturity
           nominal = barrierParams.params('nominal');
           basePrice = barrierParams.params('basePrice');
           strike =  barrierParams.params('strike');
           lowerBarrier = barrierParams.params('lowerBarrier');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.strike = strike;
           
           KIYN = barrierParams.params('KIYN');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
            % check whether current future price breach the barrier
           comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
           
           if strcmp(barrierParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                    payoffStateB.cashflow(i) = nominal*max(comFwd(i)/basePrice - strike,0);
                   end
               end
           else
               for i=1:modelStatesSize
                   if isKI(i)
                    payoffStateA.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                    payoffStateB.cashflow(i) = nominal*max(-comFwd(i)/basePrice + strike,0);
                   end
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
           
           %process payoff at maturity
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMCMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MCMC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                % first random numbers goes to the enddate..
                for i=1:size(U,2)
                    U(:,i) = U0(:,size(U,2)-(i-1));
                end
            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
           Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMCMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMCMC(StrikeSim,nextXIdx, ...
                                                              timeStep(idx+1),timeStep(endTimeIdx+1));
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computePutShortStrategy1SMC(eqCOMDupireSpotGF,valueDate,putShortStrategyParams)
%             out =0;
            % model schedule generation
            scheduleInt = putShortStrategyParams.params('denominatorResetScheduleInt');
            scheduleSize = size(scheduleInt,1);
            resetSchedule=[];
            for i=1:scheduleSize
                resetSchedule.resetDate(i) = H_Date(scheduleInt(i,1));
                resetSchedule.days(i) = DateDiff(resetSchedule.resetDate(i),valueDate);
                resetSchedule.strike(i) = scheduleInt(i,2);
            end
            resetSchedule.scheduleSize = scheduleSize;
            
            expiryDate = putShortStrategyParams.endDate;
%             expiryDate = resetSchedule.resetEndDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredReset = 0;
            aliveReset = 0;
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(putShortStrategyParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include resetDates
            for i= 1:scheduleSize
                daysToExercise = resetSchedule.days(i);
                if daysToExercise <= 0
                    expiredReset = expiredReset +1;
                    continue;
                else
                    aliveReset = aliveReset + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveResetIdx = 1 + expiredReset;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            
            % we add endDate(maturity) to eventTimes
            
%           eventTimeIdx = zeros(scheduleSize,1);
            eventTimeIdx = zeros(scheduleSize+1,1);
            
            for i= scheduleSize:-1:lastAliveResetIdx
                idx = find(timeStep == resetSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
            %endDate
            idx = find(timeStep == maturity);
            eventTimeIdx(end) = idx;
            
          %% MC init 
            useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end

            else
                rng('default');
                rng(0);
                
                NMC = 400;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';

            end
           
           %nested MC init
           nestedNMC = eqCOMDupireSpotGF.modelParams('nestedNMC'); 
           rng('default');
           rng(0);
           nestedMaxTimeStepsSize = eqCOMDupireSpotGF.modelParams('nestedMaxTimeStepsSize'); 
           nestedMCOneTimeStep = eqCOMDupireSpotGF.modelParams('nestedMCOneTimeStep');
           
           MCTimeStep = nestedMaxTimeStepsSize -1; % nowDate (0) should be skipped
           Z= randn(nestedNMC/2,MCTimeStep);
           Z=[Z;-Z];
           Z=Z';

           %TimeInversion
           sizeZ = size(Z,1);
           A = zeros(sizeZ,sizeZ);
           for i=1:sizeZ
               A(i,sizeZ + 1 -i) = 1;
           end
           Z = A*Z;

           nestedU = Z';
           %NestedU contains random number for nested monte carlo
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = lastAliveResetIdx ; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           
           nominal = putShortStrategyParams.params('nominal');
           basePrice = putShortStrategyParams.params('basePrice');
           lowerBarrier = putShortStrategyParams.params('lowerBarrier');
           
           callStrike = putShortStrategyParams.params('callStrike');
           gearing = putShortStrategyParams.params('gearing');
           
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           
           payoffInfo.callStrike = callStrike;
           payoffInfo.resetSchedule = resetSchedule;
           
           modelStatesSize = NMC;

           indexValue.npv = 0;
           indexValue.payoff = zeros(modelStatesSize,1);
           indexValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           indexValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           optionValue.npv = 0;
           optionValue.payoff = zeros(modelStatesSize,1);
           optionValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize+1);
           optionValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize+1);
           
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           indexSim = zeros(modelStatesSize,1);
           prevOpt = zeros(modelStatesSize,1);
           currentOpt = zeros(modelStatesSize,1);
           
           % get the valueDate index Fixing
           allFixings = putShortStrategyParams.params('allFixings');
           putShortStrategyIndexFixing = allFixings('PutShortStrategyIndex');
           pastIndex = putShortStrategyIndexFixing(StrDate(valueDate,'str'));
           
           % we find the last denominatorfixing
           KOSPI200Fixing = allFixings('KOSPI200');
           pastSFixing = KOSPI200Fixing(StrDate(resetSchedule.resetDate(lastAliveResetIdx-1),'str'));
           
           
           indexSim = pastIndex*ones(modelStatesSize,1);
           eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
          %% Path Generation Forward
           % scheduleSize -> scheduleSize +1 because of endDate
           for i=lastAliveResetIdx:1:scheduleSize + 1
                if i > lastAliveResetIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                if i == lastAliveResetIdx
                    fixingS  = pastSFixing*ones(modelStatesSize,1);
                    StrikeSim = fixingS*resetSchedule.strike(i-1);
                end
                
%                 prevOTM = eqCOMDupireSpotGF.ComputePutOTMPrices(StrikeSim,nextXIdx, ...
%                                                               timeStep(startTimeIdx),timeStep(endTimeIdx+1));
                
                prevOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                              timeStep(startTimeIdx),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                                                          
                Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
                    mcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    nextX = mcOut.nextX;
                    
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    eqFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    % after induction we evaluate option price at
                    % timeSteps(idx+1)
                    currOTM = eqCOMDupireSpotGF.ComputePutOTMPricesMC(StrikeSim,nextX, ...
                                                             timeStep(idx+1),timeStep(endTimeIdx+1),...
                                                              nestedU,nestedNMC,nestedMCOneTimeStep);
                
                    indexSim = eqCOMDupireSpotGF.UpdatePutShortIndex(indexSim,fixingS,prevOTM,currOTM,timeStep(idx),timeStep(idx+1));
                    prevOTM = currOTM;
                    % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
                end
                
                fixingS = eqFwd;
                
                % reset update is only for resetSchedule
                if i <= scheduleSize
                    StrikeSim = fixingS*resetSchedule.strike(i);
                end
                
                if i== scheduleSize
                    ccc = 1.0;
                end
                indexValue.payoffStates.cashflow(:,currentNodeIdx) = indexSim;
                indexValue.payoffStates.cashflow_npv(:,currentNodeIdx) = indexSim;
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = eqFwd;
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = eqFwd ;
                nMFuture.payoff = eqFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
           
           indexValue.payoff = indexValue.payoffStates.cashflow(:,end);
           
           dfOption = eqCOMDupireSpotGF.zeroCurve.DF(maturity/365.0);
           optionValue.payoff = dfOption*nominal*max(indexValue.payoff/basePrice - callStrike,0)*gearing;
           
%            for ii=1:length(modelStatesSize)
%                 optionValue.payoff(ii) = dfOption*nominal*max(indexValue.payoff(ii)/basePrice - callStrike,0)*gearing;
%            end

           indexValue.npv = mean(indexValue.payoff);
           optionValue.npv = mean(optionValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
           out.indexValue = indexValue;
           out.optionValue = optionValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
            expiry = params.expiry;
            
            initX = ones(NMC,1);
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            %divide time interval into volExpiry
            for i=2:length(timeStep)
                startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
                endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
                dZStep = dZ(:,startIdx+1:endIdx);
                nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
            expiry = params.expiry;
            Ks = params.Ks;
            initX = ones(NMC,1);
            initXIdx = ones(NMC,1)*params.Pricingidx;
            
            payoff = zeros(NMC,length(strikes));
            volExpiry = expiry(find(expiry <= maturity));
            volExpiry = [volExpiry; 0];
            timeStep = sort(union(volExpiry,maturity),'ascend');
            nextX = initX;
            nextXIdx = initXIdx;
            for i=2:length(timeStep)
                mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
                nextX = mcmcOut.nextX;
                nextXIdx = mcmcOut.nextXIdx;
            end
            
            
            for j=1:length(strikes)
                if(strikes(j) <= 1)
                    payoff(:,j) = max(strikes(j) - nextX,0);
                else
                    payoff(:,j) = max(nextX - strikes(j),0);
                end
            end
            out = mean(payoff);
            
        end
        
        function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            pdeOTMPrices =  zeros(length(expiry),length(strikes));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
%                 strikePerExpiry = params.fwdMoneyness(i,:);
                for j=1:length(strikePerExpiry)
                    pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
                end
            end
            
            out = pdeOTMPrices;
            
        end
        
        
        % compute forward mc(euler scheme) for 8*21 vanilla otm products
        % generate time steps from the enddate backwards 
        
        function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            % we generate the schedule from the enddate
            timeStepSize = params.mcOneTimeStep;
            MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
            dummyTimeSchedule = zeros(MCTimeStep,1);
            
            dummyN = 0;
            for i=1:MCTimeStep
                dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
                if dummyTimeSchedule(i) < 0
                    break;
                end
                dummyN = dummyN +1;
                
            end
            
            timeSchedule = zeros(dummyN,1);
            for i=1:dummyN
                timeSchedule(i)= dummyTimeSchedule(i);
            end
            
            volExpiry = [expiry; 0];
            
            newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
            
            dTSchedule = zeros(length(newTimeSchedule)-1,1);
            dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
            for i=length(newTimeSchedule)-1:-1:1
                dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
                dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
            end
            
            % find the vol expiry points
            
            volKnotIdx = zeros(length(expiry),1);
            for i=1:length(expiry)
                volKnotIdx(i) = find(newTimeSchedule==expiry(i));
                
            end
            
            timeScheduleInfo.timeSchedule = newTimeSchedule;
            timeScheduleInfo.dTSchedule = dTSchedule;
            timeScheduleInfo.dTSchedule365 = dTSchedule365;
            timeScheduleInfo.volKnowIdx = volKnotIdx;
            
            
            % schedule create end
%             NMC = params.NMC;
            NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%             Ps = sobolset(length(dTSchedule));
            
            Ps = sobolset(length(dTSchedule),'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            Q.reset;
            U = qrand(Q,NMC);
            dZ0 = zeros(size(U,1),size(U,2));
%             dZ = norminv(U);
            % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
            % generate random number from the end
            for i=1:size(U,1)
                for j=1:size(U,2)
                    dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
                end
            end
            
           % apply brownian bridge
           dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
           
           % first dZ is dummy , 0
           dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
           dZ(:,2) = dZ1(:,1);
           for i=3:size(dZ1,2)+1
                dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
           end
           
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPrices =  zeros(length(expiry),length(strikes));
            
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        
        % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
        % generate time steps using only events date 
        
        function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
            expiry = params.expiry;
            strikes = params.marketStrikes;
            
            %mc random number initialize
%             NMC = 2^14;
            NMC = params.NMC;
            
            MCTimeStep = length(expiry); % -1 because 0

            Ps = sobolset(MCTimeStep,'Skip',NMC);
%             Ps = scramble(Ps,'MatousekAffineOwen');
            Q = qrandstream(Ps);
            
            reset(Q);
            U0 = qrand(Q,NMC);
            U = zeros(NMC,MCTimeStep);
            
            % first random numbers goes to the enddate..
            for i=1:size(U,2)
                U(:,i) = U0(:,size(U,2)-(i-1));
            end            
            
            mcOTMPrices =  zeros(length(expiry),size(strikes,2));
            mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
            for i=1:length(expiry)
                strikePerExpiry = strikes(i,:);
                mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
                for j=1:size(strikes,2)
                    mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
                end
            end
            
            out = mcOTMPrices;
            
        end
        
        function out = computeForwardImpVolPerExpirySimple(eqCOMDupireSpotGF,maturity,strikes)
            expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
%             expiry = params.expiry;
            lastP = zeros(1,length(Ks));
            lastP(Pricingidx) = 1.0;
            newP = zeros(1,length(Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = Ks;
            modelXSize = length(Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
            
            expiry = params.expiry;
            lastP = zeros(1,length(params.Ks));
            lastP(params.Pricingidx) = 1.0;
            newP = zeros(1,length(params.Ks));
            
            
            idx = find(maturity <= expiry);
            if isempty(idx)
                idxNow = length(expiry);
            else
                idxNow = min(idx);
            end
            
            
            %relevent volProxy
            volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
            
            %relevent probMesh and dTDays
            dTDays = 0;
            if idxNow~=1
                probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
                lastP = probMesh;
                dTDays = (maturity - expiry(idxNow-1));
            else % idxNow == 1
                dTDays = maturity;
            end
            
            
            ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
            Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
            %filling proxy grid vol
            proxy = zeros(length(lastP),1);
            proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);

            %filling end
            size = length(lastP);
            a = zeros(size,1);
            b = zeros(size,1);
            c = zeros(size,1);
            b_rg = zeros(size,1);

            dK = Ks(2) - Ks(1);
            dT = dTDays/365.0;

            dK2 = dK*dK;
            for i=1:size
                a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
                c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
            end

            a(1) = 0;
            c(size) = 0;

            % absorbing boundary condition
            c(1) = 0;
            a(size) = 0;

            b(1) = 1;
            b(size) = 1;

            % to create tridiagonal matrix

            a1 = a(2:size);
            c1 = c(1:size-1);

            b_rg(1) = 0;
            b_rg(size) = 0;

            A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
            A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
            newP = lastP*A_exp1;
            
            
            impVol = zeros(1,length(strikes));
            
            %meshmatrixC(i,j) : max(x_i - k_j,0)
            %meshmatrixP(i,j) : max(-x_i + k_j,0)
            % payoff at maturity
            modelX = params.Ks;
            modelXSize = length(params.Ks);
            strikeSize = length(strikes);
            meshC = zeros(modelXSize,strikeSize);
            meshP = zeros(modelXSize,strikeSize);
            
            for i=1:modelXSize
                for j=1:strikeSize
                    meshC(i,j) = max(modelX(i) - strikes(j),0);
                    meshP(i,j) = max(strikes(j) - modelX(i),0);
                end
            end
            
            cValue = zeros(1,strikeSize);
            pValue = zeros(1,strikeSize);
            
            impVolC = zeros(1,strikeSize);
            impVolP = zeros(1,strikeSize);
            impVol  = zeros(1,strikeSize);
            cValue = newP*meshC;
            pValue = newP*meshP;
            
            out.cValue = cValue;
            out.pValue = pValue;
            % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
            % blackVolLBR
            % blackPrice
            
            for i=1:strikeSize
                K = strikes(i);
                impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
                impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
                if K <= 1.0
                     impVol(i) = impVolP(i);
                else
                     impVol(i) = impVolC(i);
                end
            end
            
            out.impVolP = impVolP;
            out.impVolC = impVolC;
            out.impVol = impVol;
            out.probMeshInt = newP;
            
        end
        
        function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
            targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
            probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
            targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
            cValue = zeros(1,size(targetStrikes,2));
            pValue = zeros(1,size(targetStrikes,2));
            probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
            
            for i=1:length(targetExpiry)
                strikePerExpiry = targetStrikes(i,:);
                outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
                targetImpVolPerExpiry = outDummy.impVol;
                cValue = outDummy.cValue;
                pValue = outDummy.pValue;
                probMeshIntPerExpiry = outDummy.probMeshInt;
                
                for j=1:size(targetStrikes,2)
                    targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
                    cValueSurface(i,j) = cValue(j);
                    pValueSurface(i,j) = pValue(j);
                    probMeshInt(i,j) = probMeshIntPerExpiry(j);
                end
            end
            
            out.impVolSurface = targetImpVolSurface;
            out.cValueSurface = cValueSurface;
            out.pValueSurface = pValueSurface;
            out.probMeshInt = probMeshInt;
        end
        
        %generate marginal(cumulative distribution) at each event
        %date(per expiry)
        %using conditional transition Probability matrix
        function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
%             Ks = params.Ks;
%             nPoints = length(params.Ks);
            
            nPoints = length(Ks);
            condProb = condProbPerExpiry;
            Qa = zeros(length(Ks),length(Ks));
            cumM = 0;
            cumM2 = 0;
            %first row dealt separately
            Qa(1,1) = condProb(1,1);
            cumM2 = 0.0;
            for j=2:nPoints
                cumM2 = cumM2 + condProb(1,j);
                Qa(1,j) = Qa(1,1) + cumM2;
            end
            
            for i=2:nPoints
                cumM = 0.0;
                Qa(i,i) = condProb(i,i);
                for j=i-1:-1:1
                    Qa(i,i) = Qa(i,i) + condProb(i,j);
                end

                cumM = 0.0;
                for j=i-1:-1:1
                    cumM = cumM + condProb(i,j+1);
                    Qa(i,j) = Qa(i,i) - cumM;
                end

                cumM2 = 0.0;
                for j=i+1:nPoints
                    cumM2 = cumM2 + condProb(i,j);
                    Qa(i,j) = Qa(i,i) + cumM2;
                end
            end
            
            out = Qa;
        end
        
        %generate marginal(cumulative distribution) for all eventdates
        %date(per expiry)
        %using conditional transition Probability matrix
        
        function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
            Ks = params.Ks;
            marginal = zeros(params.expirySize,length(Ks),length(Ks));
            condProbPerExpiry = zeros(length(Ks),length(Ks));
            marginalPerExpiry = zeros(length(Ks),length(Ks));
            for i=1:params.expirySize
                condProbPerExpiry(:,:) = condProb(i,:,:);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
                marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
                marginal(i,:,:) = marginalPerExpiry(:,:);
            end
        end
        
        
        % Target function for FixedPointIteration Optimizer
        
        function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
            idxNow = optParams.idxNow;
            if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
            mkt_dT=mkt_dT/365.0;
            
            
          %% Solve  Forward PDE(GF) One time
            outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
            
            newProb = outGF.newProb;
            optParams.meshProb = newProb;
           %% PDE(GF) Solve End
           % find the implied vol of dupire model
           if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
               target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           else
               target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
           end
           marketImpVol = optParams.params.blackVol;
           expiry = optParams.params.expiry;
           strike = optParams.params.strike;
           
           out.newLocalVol = zeros(length(optParams.strikeSize),1);
           out.localVolError = zeros(length(optParams.strikeSize),1);
           out.impVolError =  zeros(length(optParams.strikeSize),1);
           out.modelImpVol = zeros(length(optParams.strikeSize),1);
           out.marketImpVol = zeros(length(optParams.strikeSize),1);
           out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
           out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
           out.priceRe = zeros(length(optParams.strikeSize),1);
           
           for i=1:optParams.strikeSize
                out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
                out.localVolError(i) = out.newLocalVol(i) - tvar(i);
                out.modelImpVol(i) = target.vols(i);
                out.marketImpVol(i) =  marketImpVol(idxNow,i);
                out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
                out.interpOTMPrices(i) = target.interpOTMPrices(i);
                out.interpBlackPrices(i) = target.interpBlackPrices(i);
                out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
           end
           
           % for shorter tenor we only fit for the restricted interval
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=1:optParams.lowerIdx-1
                    out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
                end

                for i=optParams.upperIdx+1:optParams.strikeSize
                    out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
                end
            end
            
            out.volErrorTotal = 0.0;
            out.priceReTotal = 0.0;
            %for one month we only fit betwee lower and upper
            dummyN = 0;
            if idxNow <= optParams.params.numOfCutoffTenor
                for i=optParams.lowerIdx:optParams.upperIdx
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            else
                for i=1:length(out.impVolError)
                    out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
                    out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
                    dummyN = dummyN +1;
                end
            end
            out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
            out.priceReTotal = sqrt(out.priceReTotal/dummyN);
            
        end
        
         % Fixed Point Iteration Optimizer
         % start from implied vol find the local vol using Reghai's fixed
         % point iteration optimization
         % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
         
        function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
            
            n= 0;
            errorTry = 1000.0; % Initial ErrorBound make it big enough
            tvarOld = tvar;
            if optParams.params.targetType == 1
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce implied vol error
                    errorTry = out.volErrorTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            else
                while (errorTry > tol) && (n < maxIter)
                    out = targetfunc(tvar);
                    tvarOld = tvar;
                    % reduce price relative error
                    errorTry = out.priceReTotal;
                    tvar = out.newLocalVol;
                    n = n+1;
                end
                x = tvar;
                finalOut = targetfunc(tvarOld);
                fval = finalOut.volErrorTotal;
                marketImpVol = finalOut.marketImpVol;
                modelImpVol  = finalOut.modelImpVol;
                interpBlackPrices = finalOut.interpBlackPrices;
                interpOTMPrices = finalOut.interpOTMPrices;
                volError     = finalOut.impVolError;
                volErrorTotal = finalOut.volErrorTotal;
                priceRe      = finalOut.priceRe;
                priceReTotal = finalOut.priceReTotal;
                nIter = n;
            end
            
        end
        
        % Fixed Local Vol per expiry by bootstrapping
        
        function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
            %initialize calibration parameters
            
            optParams.idxNow = idxNow;
            optParams.dT = dT;
            optParams.params = params;
            optParams.ipos = zeros(params.strikeSize,1);
            optParams.Ks = zeros(params.Ns,1);
            optParams.Ks = params.Ks;
            optParams.strikeSize = params.strikeSize;
            optParams.meshSize = params.Ns;
            
            optParams.meshProb = zeros(1,params.Ns);
            
            optParams.proxy = zeros(params.Ns,1);
            optParams.dK = params.Ks(2) - params.Ks(1);
            
            optParams.marketStrikes = zeros(params.strikeSize,1);
            optParams.blackOTMPrices = params.blackOTMPrices;
            optParams.callCurve = zeros(params.strikeSize,1);
            optParams.putCurve = zeros(params.strikeSize,1);
            
            for i=1:params.strikeSize
                optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
            end
                
%             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
%                 % market strike points in fwd PDE grid
%                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
%                 end
%             else
%                 for i=1:params.strikeSize
%                     optParams.marketStrikes(i) = params.strike(i);
%                 end
%             end
            
            % positions to separate vol proxy regimes
            for i=1:params.strikeSize
                optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
            end
%             optParams.ipos(params.strikeSize) = optParams.meshSize;
            
            optParams.lastProb = lastProb;
            
            % exception for one month maturity for restricted interval
            
            lowerCutoff = params.lowerCutoffVector(idxNow);
            upperCutoff = params.upperCutoffVector(idxNow);
            numOfCutoffTenor = params.numOfCutoffTenor; 
            
            

            lowerIdx = 1;
            upperIdx = params.strikeSize;
            
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end

            % fixedStrikeTest
            if eqCOMDupireSpotGF.deltaCutOff > 0
                useYN = eqCOMDupireSpotGF.useYN(idxNow,:);
                if idxNow <= numOfCutoffTenor
                    lowerIdx = min(find(useYN > 0));
                    upperIdx = max(find(useYN > 0));
                end
            end

            optParams.lowerIdx = lowerIdx;
            optParams.upperIdx = upperIdx;
            
            for i=1:params.strikeSize
                tvar(i) = params.blackVol(idxNow,i);
                lb(i) = 0.01;
                ub(i) = 1.0;
            end
            
            for i=1:lowerIdx-1
                tvar(i) = tvar(lowerIdx);
            end
            
            for i=upperIdx+1:params.strikeSize
                tvar(i) = tvar(upperIdx);
            end

            targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
                
            
%             tol = 0.0001;
%             maxIter = 1000;
            
            tol = params.tol;
            maxIter = params.maxIter;
            
            [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
            
            for i=1:params.strikeSize
                bootStrapOut.volProxy(i) = x(i);
                bootStrapOut.ipos(i) = optParams.ipos(i);
            end
            
            % for later use
            bootStrapOut.marketStrikes = optParams.marketStrikes;
            
            bootStrapOut.volError = volError;
            bootStrapOut.marketImpVol = marketImpVol;
            bootStrapOut.modelImpVol = modelImpVol;
            bootStrapOut.interpBlackPrices = interpBlackPrices;
            bootStrapOut.dupireOTMPrices = interpOTMPrices;
            
            bootStrapOut.nIter = nIter;
            
            out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
            
            bootStrapOut.lastProb = out.newProb;
            bootStrapOut.condProb = out.condProb;
            
        end
        function out = CDF(eqCOMDupireSpotGF,k,term)
            
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dk = 1e-3*k;
            dummyUp = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k + dk,term);
            blackVolUp = dummyUp.vol;
            
            dummyDn = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k - dk,term);
            blackVolDn = dummyDn.vol;
            
            dVol_dk = (blackVolUp - blackVolDn)/(2*dk);
            
            dR = eqCOMDupireSpotGF.zeroCurve.DF(term);
            dD = eqCOMDupireSpotGF.GetFwdDividends(term*365.0);
            
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,k,term);
            blackVol = dummy.vol;
            
            stdDev = sqrt(blackVol*blackVol*term);
            variance = stdDev*stdDev;
            d1 = log(fwd/k)/stdDev + 0.5*stdDev;
            d2 = d1 - stdDev;
            
            if fwd <= k
                alpha = H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = -1.0*H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 1.0 + (strikeSens + vega * dVol_dk)/dR;
            else
                alpha = -1.0 + H_ncdf(d1);
                DalphaDd1 = H_ndf(d1);
                beta = 1.0 - H_ncdf(d2);
                DbetaDd2 = -1.0*H_ndf(d2);
                
                % strikeSensitivity
                temp = stdDev * k;
                DalphaDstrike = -1.0*DalphaDd1/temp;
                DbetaDstrike  = -1.0*DbetaDd2/temp;
                temp2 = DalphaDstrike*fwd +DbetaDstrike*k + beta*1.0;
                strikeSens = dR * temp2;
                
                %vega
                temp_vega = log(k/fwd)/variance;
                DalphaDsigma = DalphaDd1*(temp_vega + 0.5);
                DbetaDsigma  = DbetaDd2 *(temp_vega - 0.5);
                temp_vega2 = DalphaDsigma * fwd + DbetaDsigma * k;
                vega = dR * sqrt(term) * temp_vega2;
                
                out = 0.0 + (strikeSens + vega * dVol_dk)/dR;
            end
            
        end
        function out =  InvCDF(eqCOMDupireSpotGF,y,term)
            atmX = norminv(y);
            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(term*365.0);
            dummy = eqCOMDupireSpotGF.SABR_ImpVol(fwd,fwd,term);
            blackVol = dummy.vol;
            guess = fwd*exp(blackVol * sqrt(term) *atmX);
            
%             targetF = @(x) norm(CDF(eqCOMDupireSpotGF,x,term) - y);
            targetF = @(x) CDF(eqCOMDupireSpotGF,x,term) - y;
            
%             [B,fval,exitflag,outputCalib] = fminsearch(targetF,guess);
            [B,fval,exitflag,outputCalib] = fzero(targetF,guess);
            
            out= B;
            
        end
        
        function out = SABR_ImpVol(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            inrangeYN = 0;
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
                inrangeYN = 1;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
            
        end
        
        function out = SABR_ImpVol2(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            if Xz < 0.000001 && Xz > -0.000001
                multiplier = 1.0;
            else
                multiplier = z/Xz;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.model_vol = model_vol;
            out.multiplier = multiplier;
        end
        
        function out = SABR_ImpVol_Qlib(eqCOMDupireSpotGF,Forward,Strike,term)
            beta = eqCOMDupireSpotGF.beta;
            lambda = eqCOMDupireSpotGF.lambda;
            omega = eqCOMDupireSpotGF.omega;
            time_shift = eqCOMDupireSpotGF.time_shift;
            
            alpha_0 = eqCOMDupireSpotGF.alpha_0;
            alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            rho = eqCOMDupireSpotGF.rho;
            nu = eqCOMDupireSpotGF.nu;
            
            alpha_adj = (alpha_0 - alpha_inf)*exp(-lambda*term) + alpha_inf;
            term_adj = term + time_shift;
            nu_adj = nu/power(term_adj,omega);
            
            z = nu_adj/alpha_adj*power(Forward*Strike, 0.5*(1.0-beta))*log(Forward/Strike);
            Xz = log((sqrt(1.0-2.0*rho*z + z*z) + z - rho)/(1.0-rho));
            
            multiplier = 0.0;
            
            dummyQLEpsilon = 2.22045e-16;
            thresHold = dummyQLEpsilon*10.0;
            dummyz2 = abs(z*z);
            
             inrangeYN = 0;
            if dummyz2 > thresHold
                multiplier = z/Xz;
                 inrangeYN = 1;
            else
                multiplier = 1.0 -0.5*rho*z -(3.0*rho*rho-2.0)*z*z/12.0;
            end
            
            model_vol = multiplier * alpha_adj/(power(Forward*Strike, 0.5*(1.0-beta))*(1.0  + power(1.0-beta, 2.0)/24.0*power(log(Forward/Strike), 2.0) + power(1.0-beta, 4.0)/1920.0*power(log(Forward/Strike),4.0))) ...
					* (1.0 + (power(1.0-beta,2.0)/24.0*power(alpha_adj, 2.0)/power(Forward*Strike, 1.0-beta) ...
							+ 0.25*rho*beta*nu_adj*alpha_adj/power(Forward*Strike, 0.5*(1.0-beta)) + (2.0-3.0*power(rho, 2.0))/24.0*power(nu_adj,2.0) )*term);
                        
            if ~isreal(model_vol)
                disp('error! imaginary vol')
            end
            
            out.vol = model_vol;
            out.z = z;
            out.Xz = Xz;
            out.multiplier = multiplier;
            out.inrangeYN = inrangeYN;
%             out = model_vol;
%             out.multiplier = multiplier;
        end
        
        function out = CalibrateToVolSurfaceSSSABR(eqCOMDupireSpotGF,black,calibrationFlag)
%             out = 0.0;
            
            % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
          %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
            
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end
            
            % calibration Params
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            params.blackVol = blackVol;
           
            params.fwdMoneyness = fwdMoneyness;
            params.marketStrike = params.fwdMoneyness;
            params.calibrationYN = eqCOMDupireSpotGF.useYN;
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            params.lb = eqCOMDupireSpotGF.modelParams('lb');
            params.ub = eqCOMDupireSpotGF.modelParams('ub');
            
            % model params
            params.alpha_0 = eqCOMDupireSpotGF.alpha_0;
            params.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
            params.rho = eqCOMDupireSpotGF.rho;
            params.nu = eqCOMDupireSpotGF.nu;
            
            params.beta = eqCOMDupireSpotGF.beta;
            params.lambda = eqCOMDupireSpotGF.lambda;
            params.omega =eqCOMDupireSpotGF.omega;
            params.time_shift =eqCOMDupireSpotGF.time_shift;
            
            params.calibrationErrorType = eqCOMDupireSpotGF.modelParams('calibrationErrorType');
            
            %output 
            out.marketStrikes = params.fwdMoneyness;
            out.fwdMoneyness = params.fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.blackOTMPrice = blackOTMPrices;
            out.SSSABROTMPrice = zeros(expirySize,strikeSize);
            out.SSSABRBlackVol = zeros(expirySize,strikeSize);
            out.re = zeros(expirySize,strikeSize);
            out.voldiff = zeros(expirySize,strikeSize);
            out.rmseTotal=0.0;
            out.volmseTotal = 0.0;
            
            
            % computeSmileReferenceRefelctedVol
            
            out.SRRVol = zeros(expirySize,strikeSize);
            out.SRRStrike = zeros(expirySize,strikeSize);
            
            out.SRRVolRef = zeros(expirySize,strikeSize);
            out.SRRStrikeRef = zeros(expirySize,strikeSize);
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global4'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'global4'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                outDummy = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                model_vol = outDummy.vol;
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global4bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 4;  % alpha_0, alpha_inf, rho, nu
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    % initial parameters costval
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal4bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.paramsOut = zeros(4,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
%                     lb = [1e-8,1e-8,-0.9999,1e-8,1e-8,1e-8];
%                     ub = [100.0,100.0,0.9999,100.0,100.0,100.0];
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');

                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global6'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 6;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
%                     tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal6',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
%                     eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
%                     out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(6,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
%                     out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = [0.0001,0.0001,-0.9999,0.0001];
                    ub = [100.0,100.0,0.9999,100.0];
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7',tvar,x, params.maxIter, options,'unc',eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'global7bc'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 7;  % alpha_0, alpha_inf, rho, nu, lambda, omega, time_shift
                    optParams.numOfEquation = params.expirySize*params.strikeSize + ...  % target opm options
                                              0;   % constraint
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    
                    tvar(1) = eqCOMDupireSpotGF.alpha_0;
                    tvar(2) = eqCOMDupireSpotGF.alpha_inf;
                    tvar(3) = eqCOMDupireSpotGF.rho;
                    tvar(4) = eqCOMDupireSpotGF.nu;
                    
                    tvar(5) = eqCOMDupireSpotGF.lambda;
                    tvar(6) = eqCOMDupireSpotGF.omega;
                    tvar(7) = eqCOMDupireSpotGF.time_shift;
                    
                    % initial parameters costval
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    x=  zeros(optParams.numOfEquation,1);
                    lb = params.lb;
                    ub = params.ub;
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionSSSABRGlobal7bc',tvar,x, params.maxIter, options,'bc',lb,ub,eqCOMDupireSpotGF,optParams);
                    
                    if ret == -1
                        error('unimplemnted');
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                    
%                     elseif ~BoundaryConditionCheck(eqHeston,popt,lb,ub)
                        
%                         tvar(1) = eqHeston.modelParams('kappa');
%                         tvar(2) = eqHeston.modelParams('theta');
%                         tvar(3) = eqHeston.modelParams('rho');
%                         tvar(4) = eqHeston.modelParams('sigma');
%                         tvar(5) = eqHeston.modelParams('v0');
%                         x=  zeros(optParams.numOfEquation,1);
%                         
%                         [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, 10, options,'bc',lb,ub,eqHeston,optParams);
                            
                    end
%                     [ret, popt, info, covar]=levmar('TargetFunctionHestonGlobal',tvar,x, eqHeston.maxIter, options,'bc',lb,ub,eqHeston,optParams);
                    
                    toc
                    
                    eqCOMDupireSpotGF.alpha_0 = popt(1);
                    eqCOMDupireSpotGF.alpha_inf = popt(2);
                    eqCOMDupireSpotGF.rho   = popt(3);
                    eqCOMDupireSpotGF.nu = popt(4);
                    
                    eqCOMDupireSpotGF.lambda = popt(5);
                    eqCOMDupireSpotGF.omega = popt(6);
                    eqCOMDupireSpotGF.time_shift = popt(7);
                    
                    out.alpha_0 = popt(1);
                    out.alpha_inf = popt(2);
                    out.rho = popt(3);
                    out.nu = popt(4);
                    
                    out.lambda = popt(5);
                    out.omega = popt(6);
                    out.time_shift = popt(7);
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                case 'justCalculateError'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    if  strcmp(params.calibrationErrorType,'impVolError')
                        for i=1:params.expirySize
%                            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrices(i,j);
                                marketStrike = params.marketStrike(i,j);
                                outDummyVol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                                model_vol = outDummyVol.vol;
                                
                                if marketStrike <= 1.0
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                                else
                                    model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                                end
                                out.re(i,j) = model_otmPrice/market_otmPrice -1.0;
                                out.SSSABRBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.SSSABROTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                    elseif strcmp(params.calibrationErrorType,'priceError')
                        for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                            for j=1:params.strikeSize
                                market_vol = params.blackVol(i,j);
                                market_otmPrice = params.blackOTMPrice(i,j);
                                marketStrike = params.marketStrikes(i,j);
                                model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                                model_vol = eqCOMDupireSpotGF.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                                out.re(i,j) = model_otmPrice- market_otmPrice;
                                out.hestonBlackVol(i,j) = model_vol;
                                out.voldiff(i,j) = model_vol - market_vol;
                                out.hestonOTMPrice(i,j) = model_otmPrice;
                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                if params.calibrationYN(j,k) == 1
                                    out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
                                    out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
                                    N=N+1;
                                end
                            end
                        end

                        out.rmseTotal = sqrt(out.rmseTotal/N);
                        out.volmseTotal = sqrt(out.volmseTotal/N);
                    end
                    
                case 'computeSlopeReferenceReflectedVol'
                    
                    out.alpha_0 = eqCOMDupireSpotGF.alpha_0;
                    out.alpha_inf = eqCOMDupireSpotGF.alpha_inf;
                    out.rho = eqCOMDupireSpotGF.rho;
                    out.nu = eqCOMDupireSpotGF.nu;
                    
                    out.lambda = eqCOMDupireSpotGF.lambda;
                    out.omega = eqCOMDupireSpotGF.omega;
                    out.time_shift = eqCOMDupireSpotGF.time_shift;
                    
                    
                    out.paramsOut = zeros(7,1);
                    out.paramsOut(1) = out.alpha_0;
                    out.paramsOut(2) = out.alpha_inf;
                    out.paramsOut(3) = out.rho;
                    out.paramsOut(4) = out.nu;
                    out.paramsOut(5) = out.lambda;
                    out.paramsOut(6) = out.omega;
                    out.paramsOut(7) = out.time_shift;
                    
                    sRef = eqCOMDupireSpotGF.modelParams('reference');
                    slope = eqCOMDupireSpotGF.modelParams('slope');
                    slopeAdjustFactor = 1.0/(1.0 + slope*(spot/sRef - 1.0));
                    
                    for i=1:params.expirySize
                           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
                           % slope adjusted vol
                           fwdAdj = fwd * slopeAdjustFactor;
                        for j=1:params.strikeSize
                            market_vol = params.blackVol(i,j);
                            market_otmPrice = params.blackOTMPrices(i,j);
                            marketStrike = params.marketStrike(i,j);
                            out.SRRStrike(i,j)  = spot * strike(j);
                            
                            outDummyVol = eqCOMDupireSpotGF.SABR_ImpVol(fwdAdj, out.SRRStrike(i,j), expiry(i)/365.0);
                            model_vol = outDummyVol.vol;
                            out.SRRVol(i,j) =  model_vol;
                            
                            out.SRRStrikeRef(i,j)  = sRef * strike(j);
                            
                            outDummyVol2 = eqCOMDupireSpotGF.SABR_ImpVol(fwdAdj, out.SRRStrikeRef(i,j), expiry(i)/365.0);
                            model_vol2 = outDummyVol2.vol;
                            out.SRRVolRef(i,j) = model_vol2;
                            
                            
                        end
                    end

%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             if params.calibrationYN(j,k) == 1
%                                 out.rmseTotal = out.rmseTotal + out.re(j,k)*out.re(j,k);
%                                 out.volmseTotal = out.volmseTotal + out.voldiff(j,k)*out.voldiff(j,k);
%                                 N=N+1;
%                             end
%                         end
%                     end
% 
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.volmseTotal = sqrt(out.volmseTotal/N);
                    
                otherwise
                        disp('unimplemented')
            end
            
            eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
            eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
            
            
        end
        
        function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
            
          % Calculate Black OTM Prices as calibration target
            volSurface = black.volSurface;
            
            expiry = volSurface.maturities;
            strike  = volSurface.strikes;
            spot =  black.spot.params('rawData');
            expirySize = length(expiry);
            strikeSize = length(strike);
            
            blackOTMPrices = zeros(expirySize,strikeSize);
            blackOTMPrices2 = zeros(expirySize,strikeSize);
            blackVol = zeros(expirySize,strikeSize);
            blackOTMVega = zeros(expirySize,strikeSize);
            blackOTMVega2 = zeros(expirySize,strikeSize);
            fwdMoneyness = zeros(expirySize,strikeSize);
            deltaH = zeros(expirySize,strikeSize);
            

            dfOut = zeros(expirySize,1);
            % we add blackvol as matrix so that in the calibation we don't
            % need to do 2D interpolation again and again!!
            % also for the target black price too!!
            
            %% useYN is a static variables for fixedStrike calibration
            % if moneynessType is fixedStrike then useYN is meaningful
            % otherwise it is id matrix
            if isKey(eqCOMDupireSpotGF.modelParams,'useYN')
                eqCOMDupireSpotGF.useYN = eqCOMDupireSpotGF.modelParams('useYN');
            else
                eqCOMDupireSpotGF.useYN = ones(expirySize,strikeSize);
            end
            
            if isKey(eqCOMDupireSpotGF.modelParams,'deltaCutOff')
                eqCOMDupireSpotGF.deltaCutOff = eqCOMDupireSpotGF.modelParams('deltaCutOff');
            else
                eqCOMDupireSpotGF.deltaCutOff = 0.0;
            end
            
            if strcmp(volSurface.params('moneyNessType'),'fixedStrike')
                settleFutures = eqCOMDupireSpotGF.modelParams('settleFutures');
                eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFutures(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fixedStrikeOneStep')
                settleFuturesSpot = eqCOMDupireSpotGF.modelParams('settleFuturesSpot');
%                 eqCOMDupireSpotGF.settleFutures = settleFutures;
                
                for k=1:expirySize
                    fwd = settleFuturesSpot(k);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j)/fwd;
                        
                        if fwdMoneyness(k,j) <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                   end
                end
                
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
                for k=1:expirySize
    %                 fwd = black.Fwd(spot,expiry(k));
    %                 df = black.zeroCurve.DF(expiry(k)/365.0);
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = strike(j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')  %% SSR
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdMoneyness(k,j) = volSurface.fwdMoneyness(k,j);
                        
                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                        
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                    end
                end
            elseif strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
                for k=1:expirySize
                    fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        fwdStrike = strike(j)/fwdFactor;
                        fwdMoneyness(k,j) = fwdStrike;
                        if fwdStrike <= 1.0
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                        end
                    end
                end
            else
                for k=1:expirySize
                    for j=1:strikeSize
                        blackVol(k,j) = volSurface.volSurfaceR(k,j);
                        if strike(j) <= 0
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        else
                            dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
                            fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
                        end

                        if fwdMoneyness(k,j) <= 1
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'P');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        else
                            blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
                            blackOTMPrices2(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j) + 0.0001,'C');
                            blackOTMVega2(k,j) = (blackOTMPrices2(k,j) - blackOTMPrices(k,j))/0.0001;
                        end
                        blackOTMVega(k,j) = BlackFwdVega(1,fwdMoneyness(k,j),blackVol(k,j),expiry(k)/365.0);
                         
                    end
                    dfOut(k) = black.zeroCurve.DF(expiry(k)/365.0);
                end
                
            end

%             eqCOMDupireSpotGF.useYN = useYN;
            
            params.volSurface = volSurface;
            params.expiry = expiry;
            params.strike = strike;
            
            params.blackOTMPrices = blackOTMPrices;
            
            params.blackVol = blackVol;
          %% Due to SSR we change initial localVol
            
            
            params.fwdMoneyness = fwdMoneyness;
            
            % additional model params
            eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
            
            
            params.expirySize = expirySize;
            params.strikeSize =  strikeSize;
            
            % optimizer setting
            params.tol = eqCOMDupireSpotGF.modelParams('tol');
            params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
            
            % vol proxy calibration grid setting
            
            params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
            params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
            params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
            params.dK   = eqCOMDupireSpotGF.modelParams('dK');
            params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
            
            % current spot on the grid
            params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
            
            % targetType : 1 = implied vol diff
            %              2 = price relative error  
            params.targetType = 1;      
            
            % GFType : 1 = matrix exponential moler higham
            %          2 = one step matrix exponential moler higham & multiplication from
            %          3 = one step GF & multiplication from right Andreasen & Huge
            
            params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
            
            params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
            
            params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
            
            params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
            params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
            
            % backGFType : 1 = forward FD(GF)
            %              2 = implicit FD
            
            params.oneStepDt = 1.0/365.0;
            
            
            
            params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
            
            params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
            params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
            
            
            %output vol proxy
            out.fwdMoneyness = fwdMoneyness;
            out.expiry = params.expiry;
            out.strike = params.strike;
            
            out.volProxy = zeros(params.expirySize,params.strikeSize);
            out.volProxyOrig = zeros(params.expirySize,params.strikeSize);
            out.ipos = zeros(params.expirySize,params.strikeSize);
            out.probMesh = zeros(params.expirySize,params.Ns);
            
            out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
            
            out.useYN = ones(params.expirySize,params.strikeSize);

            if strcmp(volSurface.params('moneyNessType'),'fixedStrike') || strcmp(volSurface.params('moneyNessType'),'fwdMoneyNessSSR')
                out.useYN = eqCOMDupireSpotGF.useYN;
            end

            %initialze the intial probability distribution in the grid
            initProb = zeros(1,params.Ns);
            initProb(params.Pricingidx) = 1.0;
            
            out.blackOTMPrices = blackOTMPrices;
            out.blackOTMPrices2= blackOTMPrices2;
            out.blackOTMVega = blackOTMVega;
            out.blackOTMVega2 = blackOTMVega2;
            out.dfOut = dfOut;
            
            out.blackVol = blackVol;
            out.blackOTMVega = blackOTMVega;
            
            out.marketImpVol = zeros(expirySize,strikeSize);
            out.modelImpVol  = zeros(expirySize,strikeSize);
            
            out.interpBlackPrices = zeros(expirySize,strikeSize);
            out.dupireOTMPrices = zeros(expirySize,strikeSize);
            
            out.dupireBlackVol = zeros(expirySize,strikeSize);
            
            out.volError = zeros(expirySize,strikeSize);
            out.priceRe = zeros(expirySize,strikeSize);
            out.priceReOrig = zeros(expirySize,strikeSize);
            
            out.marketStrikes = zeros(expirySize,strikeSize);
            
            out.Ks =  params.Ks;
            out.Pricingidx = params.Pricingidx;
            out.nIter = zeros(expirySize,1);
            out.localVol = zeros(expirySize,length(out.Ks));
            out.rmseTotal=0.0;
            out.fwdVolMseTotal = 0.0;

            out.rmseTotal1 = 0;
            out.fwdVolMseTotal1 = 0;

            
            localVolFloor = eqCOMDupireSpotGF.modelParams('localVolFloor');
            localVolCap   = eqCOMDupireSpotGF.modelParams('localVolCap');
            
            calibrationType = 'bootstrap';
            if nargin < 3
                calibrationType = 'bootstrap'; 
            else
                calibrationType = calibrationFlag;
            end
            
            switch calibrationType
                case 'bootstrap'
                    
%                     tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        
                        % we apply floor and cap for the volProxy
                        
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                            out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
                    
%                     tic
                    % assigning the calibrated volProxy to the local vol
                    % surface
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
                    
                    % generate marginal(cumulative) distribution for mcmc
                    % simulation
                    
                    eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                    
                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    % calibration gives model schedule, for later use
                    % general products such as autocallables..
                    
                    eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
                    
                    params.marketStrikes = out.marketStrikes;
                    
%                     toc
                    
%                     calculate backward scheme for given target
%                     compareforwardMCFlag = 1 for one step euler scheme
%                     compareforwardMCFlag = 2 for daily euler scheme
%                     
%                     compareforwardMCFlag = 3 for markov chain monte carlo
                     
                    
                    pricingSchemeType = params.pricingSchemeType; 
                    
%                     tic
                    
                    if pricingSchemeType == 1
                        out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backReOrig = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backPriceOrigRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
  
                    elseif pricingSchemeType == 3
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);

                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                    
                    elseif pricingSchemeType == 4
                        out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
                        out.backRe = zeros(expirySize,strikeSize);
                        out.backwardVol = zeros(expirySize,strikeSize);
                        out.backVolError = zeros(expirySize,strikeSize);

                        out.backPriceRmseTotal = 0.0;
                        out.backVolMseTotal = 0.0;
                        for i=1:expirySize
                            for j=1:strikeSize
                                out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
                                if out.marketStrikes(i,j) <=1.0
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
                                else
                                    out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
                                end
                                out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);

                            end
                        end

                        N=0;
                        for j=1:expirySize
                            for k=1:strikeSize
                                out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
                                out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
                                N=N+1;
                            end
                        end

                        out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
                        out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
                        
                    end

%                     toc
                case 'bootstrapForwardSSR'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
                    %% compute implied surface for targetStrikes;
                    
                    targetExpiry = params.expiry;
                    targetStrikes = out.fwdMoneyness;
                    slope = eqCOMDupireSpotGF.slope;
                    for i=1:length(targetExpiry)
                        nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
                        referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
                        for j=1:size(targetStrikes,2)
                            targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
                        end
                    end
                    
                    eqCOMDupireSpotGF.targetStrikes = targetStrikes;
                    
                    outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
                    out.forwardImpVol = outDummy.impVolSurface;
                    out.cValueSurface = outDummy.cValueSurface;
                    out.pValueSurface = outDummy.pValueSurface;
                    out.probMeshInt = outDummy.probMeshInt;
                    
                    % construct new implied volSurface and calibrate on
                    % this surface
                    origMktData = eqCOMDupireSpotGF.mktData;
                    
                    originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
                    
                    
                    
                    newVolSurfaceData = originalVolSurfaceData;
                    newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
                    
                    mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
                    
                    if isKey(mktData,'parametrization')
                        mktData('parametrization') = 'nonParametric';
                    end
                    
                    mktData('rawData') = newVolSurfaceData;
                    mktData('fwdMoneyness') = out.fwdMoneyness;
                    impliedVolSurfaceMktData = MktData(mktData);
                    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
                    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
                    origMktData('impliedVolSurface') = impliedVolSurface;
                    
                    newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
                    
                    newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
                    
                    % 1st step calibration info
                    newOut.rmseTotal1 = out.rmseTotal1;
                    newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
                    newOut.originalUseYN = out.useYN;
                    
                    % bumped volSurface as output
                    % originalOut 20181226
                    newOut.originalImpVolSurface = out.marketImpVol;
                    newOut.originalModelImpVol = out.modelImpVol;
                    newOut.originalVolProxy = out.volProxy;
                    newOut.originalMarketStrikes = out.marketStrikes;
                    
                    newOut.shiftedImpVolSurface = outDummy.impVolSurface;
                    newOut.targetStrikes = targetStrikes;
                    
                    out = newOut;
                    
                    
                    
                    out = newOut;
                    aaa = 1.0;
                    
                case 'bootstrapFixedStrikeOneStep'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    
                    for j=1:expirySize
                        for k=1:strikeSize
                            % we report only those calibrated target only
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal1 = out.rmseTotal1 + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal1 = out.fwdVolMseTotal1 + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal1 = sqrt(out.rmseTotal1/N);
                    out.fwdVolMseTotal1 = sqrt(out.fwdVolMseTotal1/N);
                    toc
                    
                    % update calibration result for eqComDupire
                    % compute Target implied vol
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                  %% compute implied surface for targetStrikes;
%                     
%                     targetExpiry = params.expiry;
%                     targetStrikes = out.fwdMoneyness;
%                     slope = eqCOMDupireSpotGF.slope;
%                     for i=1:length(targetExpiry)
%                         nmFwd = eqCOMDupireSpotGF.FwdNM(targetExpiry(i));
%                         referenceNmFwd = eqCOMDupireSpotGF.ReferenceFwdNM(targetExpiry(i));
%                         for j=1:size(targetStrikes,2)
%                             targetStrikes(i,j) = out.fwdMoneyness(i,j)*(1.0 + slope*(nmFwd/referenceNmFwd - 1.0));
%                         end
%                     end
%                     
%                     eqCOMDupireSpotGF.targetStrikes = targetStrikes;
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                     % construct new implied volSurface and calibrate on
%                     % this surface
%                     origMktData = eqCOMDupireSpotGF.mktData;
%                     
%                     originalVolSurfaceData = origMktData('impliedVolSurface').params('rawData');
%                     
%                     
%                     
%                     newVolSurfaceData = originalVolSurfaceData;
%                     newVolSurfaceData(2:size(newVolSurfaceData,1),2:size(newVolSurfaceData,2)) = outDummy.impVolSurface(:,:);
%                     
%                     mktData = eqCOMDupireSpotGF.mktData('impliedVolSurface').params;
%                     
%                     if isKey(mktData,'parametrization')
%                         mktData('parametrization') = 'nonParametric';
%                     end
%                     
%                     mktData('rawData') = newVolSurfaceData;
%                     mktData('fwdMoneyness') = out.fwdMoneyness;
%                     impliedVolSurfaceMktData = MktData(mktData);
%                     impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%                     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNessSSR';
%                     origMktData('impliedVolSurface') = impliedVolSurface;
%                     
%                     newBlack =  EQBlack(EQModel(Model(origMktData,black.modelParams)));
%                     
%                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrapForwardOnly');
% %                     newOut = eqCOMDupireSpotGF.CalibrateToVolSurface(newBlack,'bootstrap');
%                     
%                     % 1st step calibration info
%                     newOut.rmseTotal1 = out.rmseTotal1;
%                     newOut.fwdVolMseTotal1 = out.fwdVolMseTotal1;
%                     newOut.originalUseYN = out.useYN;
%                     
%                     % bumped volSurface as output
%                     % originalOut 20181226
%                     newOut.originalImpVolSurface = out.marketImpVol;
%                     newOut.originalModelImpVol = out.modelImpVol;
%                     newOut.originalVolProxy = out.volProxy;
%                     newOut.originalMarketStrikes = out.marketStrikes;
%                     
%                     newOut.shiftedImpVolSurface = outDummy.impVolSurface;
%                     newOut.targetStrikes = targetStrikes;
%                     
%                     out = newOut;
%                     
%                     
%                     
%                     out = newOut;
%                     aaa = 1.0;
%                     
                case 'bootstrapForwardOnly'
                    
                    tic
                    %vol proxy calibration
                    lastProb = initProb;
                    
                    for i=1:expirySize
                        if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
                        dT = dT/365.0;
                        bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
                        for j=1:strikeSize
                            out.volProxyOrig(i,j) = bootStrapOut.volProxy(j);
                            out.volProxy(i,j) = min(max(bootStrapOut.volProxy(j),localVolFloor),localVolCap);
                            out.ipos(i,j)     = bootStrapOut.ipos(j);
                            out.volError(i,j) = bootStrapOut.volError(j);
                            out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
                            out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
                            out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
                            out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
                            out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
                            out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
                            out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
                        end
                        
                        out.nIter(i) = bootStrapOut.nIter;
                        tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
                        for j=1:params.Ns
                            out.probMesh(i,j) = bootStrapOut.lastProb(j);
                            out.localVol(i,j) = tempLocalVol(j);
                        end
                        
                        out.condProb(i,:,:) = bootStrapOut.condProb;
                        % Initial Payoff for the next period
                        lastProb = bootStrapOut.lastProb;
                        
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:strikeSize
                            if eqCOMDupireSpotGF.useYN(j,k) > 0
                                out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
                                out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
                    toc
                    
                    %% SSR
                    out.targetStrikes = eqCOMDupireSpotGF.targetStrikes;
                    % update calibration result for eqComDupire
                    
                    eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
                    eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
                    eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
                    eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
                    eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;

                    eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
                    eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
                    eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
                    eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
                    eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
                    
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
                  
                  
                    aaa = 1.0;
                    
                    
                      

                otherwise
                    disp('unImplemented')
                    
            end
        end
             
    end
    
end



In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
